In [19]:
'''set up for 5 cycles finding and triangular insertion
    full process from c7k4 strong product to all graphs with 5 cycle and 3 cycle both triangulated
'''

#from pysat.solvers import *
from itertools import *
from sys import argv
import sage.graphs.graph as graph
import copy
import time
import datetime 
import os


    
def fourcycles_equals(tup1, tup2):
    m, n, o, p = tup2[0], tup2[1], tup2[2], tup2[3]

    if tup1 == (m, n, o, p):
        return True
    elif tup1 == (p, m, n, o):
        return True
    elif tup1 == (o, p, m, n):
        return True
    elif tup1 == (n, o, p, m):
        return True
    else:
        return False
    
def fivecycles_equals(tup1, tup2):
    m, n, o, p, q = tup2[0], tup2[1], tup2[2], tup2[3], tup2[4]

    if tup1 == (m, n, o, p, q):
        return True
    elif tup1 == (q, m, n, o, p):
        return True
    elif tup1 == (p, q, m, n, o):
        return True
    elif tup1 == (o, p, q, m, n):
        return True
    elif tup1 == (n, o, p, q, m):
        return True
    else:
        return False

def adj_list_to_dict(adj_list):
    adj_dict = {}
    for node, neighbors in enumerate(adj_list):
        adj_dict[node] = neighbors

    return adj_dict

def dict_to_adj_list(dict):
    adj_list = []
    for i in range(0, len(dict)):
        adj_list.append(dict[i])
    return adj_list

def all_different(*args):
    """Checks if all arguments are different."""
    return len(set(args)) == len(args)


def main():

    #input: 
    #n = int(input("how long is the desired cycle?"))

    #this is c7k4 strong product
    g = [[1, 6, 7, 8, 13, 14, 15, 20, 21, 22, 27], [0, 2, 7, 8, 9, 14, 15, 16, 21, 22, 23], 
         [1, 3, 8, 9, 10, 15, 16, 17, 22, 23, 24], [2, 4, 9, 10, 11, 16, 17, 18, 23, 24, 25],  #3
         [3, 5, 10, 11, 12, 17, 18, 19, 24, 25, 26], [4, 6, 11, 12, 13, 18, 19, 20, 25, 26, 27], #5
         [0, 5, 7, 12, 13, 14, 19, 20, 21, 26, 27], [0, 1, 6, 8, 13, 14, 15, 20, 21, 22, 27],  #7
         [0, 1, 2, 7, 9, 14, 15, 16, 21, 22, 23], [1, 2, 3, 8, 10, 15, 16, 17, 22, 23, 24],   #9    
         [2, 3, 4, 9, 11, 16, 17, 18, 23, 24, 25], [3, 4, 5, 10, 12, 17, 18, 19, 24, 25, 26], #11
         [4, 5, 6, 11, 13, 18, 19, 20, 25, 26, 27], [0, 5, 6, 7, 12, 14, 19, 20, 21, 26, 27], #13
         [0, 1, 6, 7, 8, 13, 15, 20, 21, 22, 27], [0, 1, 2, 7, 8, 9, 14, 16, 21, 22, 23], #15
         [1, 2, 3, 8, 9, 10, 15, 17, 22, 23, 24], [2, 3, 4, 9, 10, 11, 16, 18, 23, 24, 25], #17
         [3, 4, 5, 10, 11, 12, 17, 19, 24, 25, 26], [4, 5, 6, 11, 12, 13, 18, 20, 25, 26, 27], #19
         [0, 5, 6, 7, 12, 13, 14, 19, 21, 26, 27], [0, 1, 6, 7, 8, 13, 14, 15, 20, 22, 27], #21
         [0, 1, 2, 7, 8, 9, 14, 15, 16, 21, 23], [1, 2, 3, 8, 9, 10, 15, 16, 17, 22, 24], #23
         [2, 3, 4, 9, 10, 11, 16, 17, 18, 23, 25], [3, 4, 5, 10, 11, 12, 17, 18, 19, 24, 26], #25 
         [4, 5, 6, 11, 12, 13, 18, 19, 20, 25, 27], [0, 5, 6, 7, 12, 13, 14, 19, 20, 21, 26]] #27
    
     #list of cycles of length 3 in the inflated c7k4 strong product graph
    
    #all 3 cycles in the c7k4 strong product
    tricycles = [(0, 1, 7), (0, 1, 8), (0, 1, 14), (0, 1, 15), (0, 1, 21), (0, 1, 22), (0, 6, 7), (0, 6, 13), (0, 6, 14), (0, 6, 20), (0, 6, 21), (0, 6, 27), (0, 7, 1), (0, 7, 6), (0, 7, 8), (0, 7, 13), (0, 7, 14), (0, 7, 15), (0, 7, 20), (0, 7, 21), (0, 7, 22), (0, 7, 27), (0, 8, 1), (0, 8, 7), (0, 8, 14), (0, 8, 15), (0, 8, 21), (0, 8, 22), (0, 13, 6), (0, 13, 7), (0, 13, 14), (0, 13, 20), (0, 13, 21), (0, 13, 27), (0, 14, 1), (0, 14, 6), (0, 14, 7), (0, 14, 8), (0, 14, 13), (0, 14, 15), (0, 14, 20), (0, 14, 21), (0, 14, 22), (0, 14, 27), (0, 15, 1), (0, 15, 7), (0, 15, 8), (0, 15, 14), (0, 15, 21), (0, 15, 22), (0, 20, 6), (0, 20, 7), (0, 20, 13), (0, 20, 14), (0, 20, 21), (0, 20, 27), (0, 21, 1), (0, 21, 6), (0, 21, 7), (0, 21, 8), (0, 21, 13), (0, 21, 14), (0, 21, 15), (0, 21, 20), (0, 21, 22), (0, 21, 27), (0, 22, 1), (0, 22, 7), (0, 22, 8), (0, 22, 14), (0, 22, 15), (0, 22, 21), (0, 27, 6), (0, 27, 7), (0, 27, 13), (0, 27, 14), (0, 27, 20), (0, 27, 21), (1, 0, 7), (1, 0, 8), (1, 0, 14), (1, 0, 15), (1, 0, 21), (1, 0, 22), (1, 2, 8), (1, 2, 9), (1, 2, 15), (1, 2, 16), (1, 2, 22), (1, 2, 23), (1, 7, 0), (1, 7, 8), (1, 7, 14), (1, 7, 15), (1, 7, 21), (1, 7, 22), (1, 8, 0), (1, 8, 2), (1, 8, 7), (1, 8, 9), (1, 8, 14), (1, 8, 15), (1, 8, 16), (1, 8, 21), (1, 8, 22), (1, 8, 23), (1, 9, 2), (1, 9, 8), (1, 9, 15), (1, 9, 16), (1, 9, 22), (1, 9, 23), (1, 14, 0), (1, 14, 7), (1, 14, 8), (1, 14, 15), (1, 14, 21), (1, 14, 22), (1, 15, 0), (1, 15, 2), (1, 15, 7), (1, 15, 8), (1, 15, 9), (1, 15, 14), (1, 15, 16), (1, 15, 21), (1, 15, 22), (1, 15, 23), (1, 16, 2), (1, 16, 8), (1, 16, 9), (1, 16, 15), (1, 16, 22), (1, 16, 23), (1, 21, 0), (1, 21, 7), (1, 21, 8), (1, 21, 14), (1, 21, 15), (1, 21, 22), (1, 22, 0), (1, 22, 2), (1, 22, 7), (1, 22, 8), (1, 22, 9), (1, 22, 14), (1, 22, 15), (1, 22, 16), (1, 22, 21), (1, 22, 23), (1, 23, 2), (1, 23, 8), (1, 23, 9), (1, 23, 15), (1, 23, 16), (1, 23, 22), (2, 1, 8), (2, 1, 9), (2, 1, 15), (2, 1, 16), (2, 1, 22), (2, 1, 23), (2, 3, 9), (2, 3, 10), (2, 3, 16), (2, 3, 17), (2, 3, 23), (2, 3, 24), (2, 8, 1), (2, 8, 9), (2, 8, 15), (2, 8, 16), (2, 8, 22), (2, 8, 23), (2, 9, 1), (2, 9, 3), (2, 9, 8), (2, 9, 10), (2, 9, 15), (2, 9, 16), (2, 9, 17), (2, 9, 22), (2, 9, 23), (2, 9, 24), (2, 10, 3), (2, 10, 9), (2, 10, 16), (2, 10, 17), (2, 10, 23), (2, 10, 24), (2, 15, 1), (2, 15, 8), (2, 15, 9), (2, 15, 16), (2, 15, 22), (2, 15, 23), (2, 16, 1), (2, 16, 3), (2, 16, 8), (2, 16, 9), (2, 16, 10), (2, 16, 15), (2, 16, 17), (2, 16, 22), (2, 16, 23), (2, 16, 24), (2, 17, 3), (2, 17, 9), (2, 17, 10), (2, 17, 16), (2, 17, 23), (2, 17, 24), (2, 22, 1), (2, 22, 8), (2, 22, 9), (2, 22, 15), (2, 22, 16), (2, 22, 23), (2, 23, 1), (2, 23, 3), (2, 23, 8), (2, 23, 9), (2, 23, 10), (2, 23, 15), (2, 23, 16), (2, 23, 17), (2, 23, 22), (2, 23, 24), (2, 24, 3), (2, 24, 9), (2, 24, 10), (2, 24, 16), (2, 24, 17), (2, 24, 23), (3, 2, 9), (3, 2, 10), (3, 2, 16), (3, 2, 17), (3, 2, 23), (3, 2, 24), (3, 4, 10), (3, 4, 11), (3, 4, 17), (3, 4, 18), (3, 4, 24), (3, 4, 25), (3, 9, 2), (3, 9, 10), (3, 9, 16), (3, 9, 17), (3, 9, 23), (3, 9, 24), (3, 10, 2), (3, 10, 4), (3, 10, 9), (3, 10, 11), (3, 10, 16), (3, 10, 17), (3, 10, 18), (3, 10, 23), (3, 10, 24), (3, 10, 25), (3, 11, 4), (3, 11, 10), (3, 11, 17), (3, 11, 18), (3, 11, 24), (3, 11, 25), (3, 16, 2), (3, 16, 9), (3, 16, 10), (3, 16, 17), (3, 16, 23), (3, 16, 24), (3, 17, 2), (3, 17, 4), (3, 17, 9), (3, 17, 10), (3, 17, 11), (3, 17, 16), (3, 17, 18), (3, 17, 23), (3, 17, 24), (3, 17, 25), (3, 18, 4), (3, 18, 10), (3, 18, 11), (3, 18, 17), (3, 18, 24), (3, 18, 25), (3, 23, 2), (3, 23, 9), (3, 23, 10), (3, 23, 16), (3, 23, 17), (3, 23, 24), (3, 24, 2), (3, 24, 4), (3, 24, 9), (3, 24, 10), (3, 24, 11), (3, 24, 16), (3, 24, 17), (3, 24, 18), (3, 24, 23), (3, 24, 25), (3, 25, 4), (3, 25, 10), (3, 25, 11), (3, 25, 17), (3, 25, 18), (3, 25, 24), (4, 3, 10), (4, 3, 11), (4, 3, 17), (4, 3, 18), (4, 3, 24), (4, 3, 25), (4, 5, 11), (4, 5, 12), (4, 5, 18), (4, 5, 19), (4, 5, 25), (4, 5, 26), (4, 10, 3), (4, 10, 11), (4, 10, 17), (4, 10, 18), (4, 10, 24), (4, 10, 25), (4, 11, 3), (4, 11, 5), (4, 11, 10), (4, 11, 12), (4, 11, 17), (4, 11, 18), (4, 11, 19), (4, 11, 24), (4, 11, 25), (4, 11, 26), (4, 12, 5), (4, 12, 11), (4, 12, 18), (4, 12, 19), (4, 12, 25), (4, 12, 26), (4, 17, 3), (4, 17, 10), (4, 17, 11), (4, 17, 18), (4, 17, 24), (4, 17, 25), (4, 18, 3), (4, 18, 5), (4, 18, 10), (4, 18, 11), (4, 18, 12), (4, 18, 17), (4, 18, 19), (4, 18, 24), (4, 18, 25), (4, 18, 26), (4, 19, 5), (4, 19, 11), (4, 19, 12), (4, 19, 18), (4, 19, 25), (4, 19, 26), (4, 24, 3), (4, 24, 10), (4, 24, 11), (4, 24, 17), (4, 24, 18), (4, 24, 25), (4, 25, 3), (4, 25, 5), (4, 25, 10), (4, 25, 11), (4, 25, 12), (4, 25, 17), (4, 25, 18), (4, 25, 19), (4, 25, 24), (4, 25, 26), (4, 26, 5), (4, 26, 11), (4, 26, 12), (4, 26, 18), (4, 26, 19), (4, 26, 25), (5, 4, 11), (5, 4, 12), (5, 4, 18), (5, 4, 19), (5, 4, 25), (5, 4, 26), (5, 6, 12), (5, 6, 13), (5, 6, 19), (5, 6, 20), (5, 6, 26), (5, 6, 27), (5, 11, 4), (5, 11, 12), (5, 11, 18), (5, 11, 19), (5, 11, 25), (5, 11, 26), (5, 12, 4), (5, 12, 6), (5, 12, 11), (5, 12, 13), (5, 12, 18), (5, 12, 19), (5, 12, 20), (5, 12, 25), (5, 12, 26), (5, 12, 27), (5, 13, 6), (5, 13, 12), (5, 13, 19), (5, 13, 20), (5, 13, 26), (5, 13, 27), (5, 18, 4), (5, 18, 11), (5, 18, 12), (5, 18, 19), (5, 18, 25), (5, 18, 26), (5, 19, 4), (5, 19, 6), (5, 19, 11), (5, 19, 12), (5, 19, 13), (5, 19, 18), (5, 19, 20), (5, 19, 25), (5, 19, 26), (5, 19, 27), (5, 20, 6), (5, 20, 12), (5, 20, 13), (5, 20, 19), (5, 20, 26), (5, 20, 27), (5, 25, 4), (5, 25, 11), (5, 25, 12), (5, 25, 18), (5, 25, 19), (5, 25, 26), (5, 26, 4), (5, 26, 6), (5, 26, 11), (5, 26, 12), (5, 26, 13), (5, 26, 18), (5, 26, 19), (5, 26, 20), (5, 26, 25), (5, 26, 27), (5, 27, 6), (5, 27, 12), (5, 27, 13), (5, 27, 19), (5, 27, 20), (5, 27, 26), (6, 0, 7), (6, 0, 13), (6, 0, 14), (6, 0, 20), (6, 0, 21), (6, 0, 27), (6, 5, 12), (6, 5, 13), (6, 5, 19), (6, 5, 20), (6, 5, 26), (6, 5, 27), (6, 7, 0), (6, 7, 13), (6, 7, 14), (6, 7, 20), (6, 7, 21), (6, 7, 27), (6, 12, 5), (6, 12, 13), (6, 12, 19), (6, 12, 20), (6, 12, 26), (6, 12, 27), (6, 13, 0), (6, 13, 5), (6, 13, 7), (6, 13, 12), (6, 13, 14), (6, 13, 19), (6, 13, 20), (6, 13, 21), (6, 13, 26), (6, 13, 27), (6, 14, 0), (6, 14, 7), (6, 14, 13), (6, 14, 20), (6, 14, 21), (6, 14, 27), (6, 19, 5), (6, 19, 12), (6, 19, 13), (6, 19, 20), (6, 19, 26), (6, 19, 27), (6, 20, 0), (6, 20, 5), (6, 20, 7), (6, 20, 12), (6, 20, 13), (6, 20, 14), (6, 20, 19), (6, 20, 21), (6, 20, 26), (6, 20, 27), (6, 21, 0), (6, 21, 7), (6, 21, 13), (6, 21, 14), (6, 21, 20), (6, 21, 27), (6, 26, 5), (6, 26, 12), (6, 26, 13), (6, 26, 19), (6, 26, 20), (6, 26, 27), (6, 27, 0), (6, 27, 5), (6, 27, 7), (6, 27, 12), (6, 27, 13), (6, 27, 14), (6, 27, 19), (6, 27, 20), (6, 27, 21), (6, 27, 26), (7, 0, 1), (7, 0, 6), (7, 0, 8), (7, 0, 13), (7, 0, 14), (7, 0, 15), (7, 0, 20), (7, 0, 21), (7, 0, 22), (7, 0, 27), (7, 1, 0), (7, 1, 8), (7, 1, 14), (7, 1, 15), (7, 1, 21), (7, 1, 22), (7, 6, 0), (7, 6, 13), (7, 6, 14), (7, 6, 20), (7, 6, 21), (7, 6, 27), (7, 8, 0), (7, 8, 1), (7, 8, 14), (7, 8, 15), (7, 8, 21), (7, 8, 22), (7, 13, 0), (7, 13, 6), (7, 13, 14), (7, 13, 20), (7, 13, 21), (7, 13, 27), (7, 14, 0), (7, 14, 1), (7, 14, 6), (7, 14, 8), (7, 14, 13), (7, 14, 15), (7, 14, 20), (7, 14, 21), (7, 14, 22), (7, 14, 27), (7, 15, 0), (7, 15, 1), (7, 15, 8), (7, 15, 14), (7, 15, 21), (7, 15, 22), (7, 20, 0), (7, 20, 6), (7, 20, 13), (7, 20, 14), (7, 20, 21), (7, 20, 27), (7, 21, 0), (7, 21, 1), (7, 21, 6), (7, 21, 8), (7, 21, 13), (7, 21, 14), (7, 21, 15), (7, 21, 20), (7, 21, 22), (7, 21, 27), (7, 22, 0), (7, 22, 1), (7, 22, 8), (7, 22, 14), (7, 22, 15), (7, 22, 21), (7, 27, 0), (7, 27, 6), (7, 27, 13), (7, 27, 14), (7, 27, 20), (7, 27, 21), (8, 0, 1), (8, 0, 7), (8, 0, 14), (8, 0, 15), (8, 0, 21), (8, 0, 22), (8, 1, 0), (8, 1, 2), (8, 1, 7), (8, 1, 9), (8, 1, 14), (8, 1, 15), (8, 1, 16), (8, 1, 21), (8, 1, 22), (8, 1, 23), (8, 2, 1), (8, 2, 9), (8, 2, 15), (8, 2, 16), (8, 2, 22), (8, 2, 23), (8, 7, 0), (8, 7, 1), (8, 7, 14), (8, 7, 15), (8, 7, 21), (8, 7, 22), (8, 9, 1), (8, 9, 2), (8, 9, 15), (8, 9, 16), (8, 9, 22), (8, 9, 23), (8, 14, 0), (8, 14, 1), (8, 14, 7), (8, 14, 15), (8, 14, 21), (8, 14, 22), (8, 15, 0), (8, 15, 1), (8, 15, 2), (8, 15, 7), (8, 15, 9), (8, 15, 14), (8, 15, 16), (8, 15, 21), (8, 15, 22), (8, 15, 23), (8, 16, 1), (8, 16, 2), (8, 16, 9), (8, 16, 15), (8, 16, 22), (8, 16, 23), (8, 21, 0), (8, 21, 1), (8, 21, 7), (8, 21, 14), (8, 21, 15), (8, 21, 22), (8, 22, 0), (8, 22, 1), (8, 22, 2), (8, 22, 7), (8, 22, 9), (8, 22, 14), (8, 22, 15), (8, 22, 16), (8, 22, 21), (8, 22, 23), (8, 23, 1), (8, 23, 2), (8, 23, 9), (8, 23, 15), (8, 23, 16), (8, 23, 22), (9, 1, 2), (9, 1, 8), (9, 1, 15), (9, 1, 16), (9, 1, 22), (9, 1, 23), (9, 2, 1), (9, 2, 3), (9, 2, 8), (9, 2, 10), (9, 2, 15), (9, 2, 16), (9, 2, 17), (9, 2, 22), (9, 2, 23), (9, 2, 24), (9, 3, 2), (9, 3, 10), (9, 3, 16), (9, 3, 17), (9, 3, 23), (9, 3, 24), (9, 8, 1), (9, 8, 2), (9, 8, 15), (9, 8, 16), (9, 8, 22), (9, 8, 23), (9, 10, 2), (9, 10, 3), (9, 10, 16), (9, 10, 17), (9, 10, 23), (9, 10, 24), (9, 15, 1), (9, 15, 2), (9, 15, 8), (9, 15, 16), (9, 15, 22), (9, 15, 23), (9, 16, 1), (9, 16, 2), (9, 16, 3), (9, 16, 8), (9, 16, 10), (9, 16, 15), (9, 16, 17), (9, 16, 22), (9, 16, 23), (9, 16, 24), (9, 17, 2), (9, 17, 3), (9, 17, 10), (9, 17, 16), (9, 17, 23), (9, 17, 24), (9, 22, 1), (9, 22, 2), (9, 22, 8), (9, 22, 15), (9, 22, 16), (9, 22, 23), (9, 23, 1), (9, 23, 2), (9, 23, 3), (9, 23, 8), (9, 23, 10), (9, 23, 15), (9, 23, 16), (9, 23, 17), (9, 23, 22), (9, 23, 24), (9, 24, 2), (9, 24, 3), (9, 24, 10), (9, 24, 16), (9, 24, 17), (9, 24, 23), (10, 2, 3), (10, 2, 9), (10, 2, 16), (10, 2, 17), (10, 2, 23), (10, 2, 24), (10, 3, 2), (10, 3, 4), (10, 3, 9), (10, 3, 11), (10, 3, 16), (10, 3, 17), (10, 3, 18), (10, 3, 23), (10, 3, 24), (10, 3, 25), (10, 4, 3), (10, 4, 11), (10, 4, 17), (10, 4, 18), (10, 4, 24), (10, 4, 25), (10, 9, 2), (10, 9, 3), (10, 9, 16), (10, 9, 17), (10, 9, 23), (10, 9, 24), (10, 11, 3), (10, 11, 4), (10, 11, 17), (10, 11, 18), (10, 11, 24), (10, 11, 25), (10, 16, 2), (10, 16, 3), (10, 16, 9), (10, 16, 17), (10, 16, 23), (10, 16, 24), (10, 17, 2), (10, 17, 3), (10, 17, 4), (10, 17, 9), (10, 17, 11), (10, 17, 16), (10, 17, 18), (10, 17, 23), (10, 17, 24), (10, 17, 25), (10, 18, 3), (10, 18, 4), (10, 18, 11), (10, 18, 17), (10, 18, 24), (10, 18, 25), (10, 23, 2), (10, 23, 3), (10, 23, 9), (10, 23, 16), (10, 23, 17), (10, 23, 24), (10, 24, 2), (10, 24, 3), (10, 24, 4), (10, 24, 9), (10, 24, 11), (10, 24, 16), (10, 24, 17), (10, 24, 18), (10, 24, 23), (10, 24, 25), (10, 25, 3), (10, 25, 4), (10, 25, 11), (10, 25, 17), (10, 25, 18), (10, 25, 24), (11, 3, 4), (11, 3, 10), (11, 3, 17), (11, 3, 18), (11, 3, 24), (11, 3, 25), (11, 4, 3), (11, 4, 5), (11, 4, 10), (11, 4, 12), (11, 4, 17), (11, 4, 18), (11, 4, 19), (11, 4, 24), (11, 4, 25), (11, 4, 26), (11, 5, 4), (11, 5, 12), (11, 5, 18), (11, 5, 19), (11, 5, 25), (11, 5, 26), (11, 10, 3), (11, 10, 4), (11, 10, 17), (11, 10, 18), (11, 10, 24), (11, 10, 25), (11, 12, 4), (11, 12, 5), (11, 12, 18), (11, 12, 19), (11, 12, 25), (11, 12, 26), (11, 17, 3), (11, 17, 4), (11, 17, 10), (11, 17, 18), (11, 17, 24), (11, 17, 25), (11, 18, 3), (11, 18, 4), (11, 18, 5), (11, 18, 10), (11, 18, 12), (11, 18, 17), (11, 18, 19), (11, 18, 24), (11, 18, 25), (11, 18, 26), (11, 19, 4), (11, 19, 5), (11, 19, 12), (11, 19, 18), (11, 19, 25), (11, 19, 26), (11, 24, 3), (11, 24, 4), (11, 24, 10), (11, 24, 17), (11, 24, 18), (11, 24, 25), (11, 25, 3), (11, 25, 4), (11, 25, 5), (11, 25, 10), (11, 25, 12), (11, 25, 17), (11, 25, 18), (11, 25, 19), (11, 25, 24), (11, 25, 26), (11, 26, 4), (11, 26, 5), (11, 26, 12), (11, 26, 18), (11, 26, 19), (11, 26, 25), (12, 4, 5), (12, 4, 11), (12, 4, 18), (12, 4, 19), (12, 4, 25), (12, 4, 26), (12, 5, 4), (12, 5, 6), (12, 5, 11), (12, 5, 13), (12, 5, 18), (12, 5, 19), (12, 5, 20), (12, 5, 25), (12, 5, 26), (12, 5, 27), (12, 6, 5), (12, 6, 13), (12, 6, 19), (12, 6, 20), (12, 6, 26), (12, 6, 27), (12, 11, 4), (12, 11, 5), (12, 11, 18), (12, 11, 19), (12, 11, 25), (12, 11, 26), (12, 13, 5), (12, 13, 6), (12, 13, 19), (12, 13, 20), (12, 13, 26), (12, 13, 27), (12, 18, 4), (12, 18, 5), (12, 18, 11), (12, 18, 19), (12, 18, 25), (12, 18, 26), (12, 19, 4), (12, 19, 5), (12, 19, 6), (12, 19, 11), (12, 19, 13), (12, 19, 18), (12, 19, 20), (12, 19, 25), (12, 19, 26), (12, 19, 27), (12, 20, 5), (12, 20, 6), (12, 20, 13), (12, 20, 19), (12, 20, 26), (12, 20, 27), (12, 25, 4), (12, 25, 5), (12, 25, 11), (12, 25, 18), (12, 25, 19), (12, 25, 26), (12, 26, 4), (12, 26, 5), (12, 26, 6), (12, 26, 11), (12, 26, 13), (12, 26, 18), (12, 26, 19), (12, 26, 20), (12, 26, 25), (12, 26, 27), (12, 27, 5), (12, 27, 6), (12, 27, 13), (12, 27, 19), (12, 27, 20), (12, 27, 26), (13, 0, 6), (13, 0, 7), (13, 0, 14), (13, 0, 20), (13, 0, 21), (13, 0, 27), (13, 5, 6), (13, 5, 12), (13, 5, 19), (13, 5, 20), (13, 5, 26), (13, 5, 27), (13, 6, 0), (13, 6, 5), (13, 6, 7), (13, 6, 12), (13, 6, 14), (13, 6, 19), (13, 6, 20), (13, 6, 21), (13, 6, 26), (13, 6, 27), (13, 7, 0), (13, 7, 6), (13, 7, 14), (13, 7, 20), (13, 7, 21), (13, 7, 27), (13, 12, 5), (13, 12, 6), (13, 12, 19), (13, 12, 20), (13, 12, 26), (13, 12, 27), (13, 14, 0), (13, 14, 6), (13, 14, 7), (13, 14, 20), (13, 14, 21), (13, 14, 27), (13, 19, 5), (13, 19, 6), (13, 19, 12), (13, 19, 20), (13, 19, 26), (13, 19, 27), (13, 20, 0), (13, 20, 5), (13, 20, 6), (13, 20, 7), (13, 20, 12), (13, 20, 14), (13, 20, 19), (13, 20, 21), (13, 20, 26), (13, 20, 27), (13, 21, 0), (13, 21, 6), (13, 21, 7), (13, 21, 14), (13, 21, 20), (13, 21, 27), (13, 26, 5), (13, 26, 6), (13, 26, 12), (13, 26, 19), (13, 26, 20), (13, 26, 27), (13, 27, 0), (13, 27, 5), (13, 27, 6), (13, 27, 7), (13, 27, 12), (13, 27, 14), (13, 27, 19), (13, 27, 20), (13, 27, 21), (13, 27, 26), (14, 0, 1), (14, 0, 6), (14, 0, 7), (14, 0, 8), (14, 0, 13), (14, 0, 15), (14, 0, 20), (14, 0, 21), (14, 0, 22), (14, 0, 27), (14, 1, 0), (14, 1, 7), (14, 1, 8), (14, 1, 15), (14, 1, 21), (14, 1, 22), (14, 6, 0), (14, 6, 7), (14, 6, 13), (14, 6, 20), (14, 6, 21), (14, 6, 27), (14, 7, 0), (14, 7, 1), (14, 7, 6), (14, 7, 8), (14, 7, 13), (14, 7, 15), (14, 7, 20), (14, 7, 21), (14, 7, 22), (14, 7, 27), (14, 8, 0), (14, 8, 1), (14, 8, 7), (14, 8, 15), (14, 8, 21), (14, 8, 22), (14, 13, 0), (14, 13, 6), (14, 13, 7), (14, 13, 20), (14, 13, 21), (14, 13, 27), (14, 15, 0), (14, 15, 1), (14, 15, 7), (14, 15, 8), (14, 15, 21), (14, 15, 22), (14, 20, 0), (14, 20, 6), (14, 20, 7), (14, 20, 13), (14, 20, 21), (14, 20, 27), (14, 21, 0), (14, 21, 1), (14, 21, 6), (14, 21, 7), (14, 21, 8), (14, 21, 13), (14, 21, 15), (14, 21, 20), (14, 21, 22), (14, 21, 27), (14, 22, 0), (14, 22, 1), (14, 22, 7), (14, 22, 8), (14, 22, 15), (14, 22, 21), (14, 27, 0), (14, 27, 6), (14, 27, 7), (14, 27, 13), (14, 27, 20), (14, 27, 21), (15, 0, 1), (15, 0, 7), (15, 0, 8), (15, 0, 14), (15, 0, 21), (15, 0, 22), (15, 1, 0), (15, 1, 2), (15, 1, 7), (15, 1, 8), (15, 1, 9), (15, 1, 14), (15, 1, 16), (15, 1, 21), (15, 1, 22), (15, 1, 23), (15, 2, 1), (15, 2, 8), (15, 2, 9), (15, 2, 16), (15, 2, 22), (15, 2, 23), (15, 7, 0), (15, 7, 1), (15, 7, 8), (15, 7, 14), (15, 7, 21), (15, 7, 22), (15, 8, 0), (15, 8, 1), (15, 8, 2), (15, 8, 7), (15, 8, 9), (15, 8, 14), (15, 8, 16), (15, 8, 21), (15, 8, 22), (15, 8, 23), (15, 9, 1), (15, 9, 2), (15, 9, 8), (15, 9, 16), (15, 9, 22), (15, 9, 23), (15, 14, 0), (15, 14, 1), (15, 14, 7), (15, 14, 8), (15, 14, 21), (15, 14, 22), (15, 16, 1), (15, 16, 2), (15, 16, 8), (15, 16, 9), (15, 16, 22), (15, 16, 23), (15, 21, 0), (15, 21, 1), (15, 21, 7), (15, 21, 8), (15, 21, 14), (15, 21, 22), (15, 22, 0), (15, 22, 1), (15, 22, 2), (15, 22, 7), (15, 22, 8), (15, 22, 9), (15, 22, 14), (15, 22, 16), (15, 22, 21), (15, 22, 23), (15, 23, 1), (15, 23, 2), (15, 23, 8), (15, 23, 9), (15, 23, 16), (15, 23, 22), (16, 1, 2), (16, 1, 8), (16, 1, 9), (16, 1, 15), (16, 1, 22), (16, 1, 23), (16, 2, 1), (16, 2, 3), (16, 2, 8), (16, 2, 9), (16, 2, 10), (16, 2, 15), (16, 2, 17), (16, 2, 22), (16, 2, 23), (16, 2, 24), (16, 3, 2), (16, 3, 9), (16, 3, 10), (16, 3, 17), (16, 3, 23), (16, 3, 24), (16, 8, 1), (16, 8, 2), (16, 8, 9), (16, 8, 15), (16, 8, 22), (16, 8, 23), (16, 9, 1), (16, 9, 2), (16, 9, 3), (16, 9, 8), (16, 9, 10), (16, 9, 15), (16, 9, 17), (16, 9, 22), (16, 9, 23), (16, 9, 24), (16, 10, 2), (16, 10, 3), (16, 10, 9), (16, 10, 17), (16, 10, 23), (16, 10, 24), (16, 15, 1), (16, 15, 2), (16, 15, 8), (16, 15, 9), (16, 15, 22), (16, 15, 23), (16, 17, 2), (16, 17, 3), (16, 17, 9), (16, 17, 10), (16, 17, 23), (16, 17, 24), (16, 22, 1), (16, 22, 2), (16, 22, 8), (16, 22, 9), (16, 22, 15), (16, 22, 23), (16, 23, 1), (16, 23, 2), (16, 23, 3), (16, 23, 8), (16, 23, 9), (16, 23, 10), (16, 23, 15), (16, 23, 17), (16, 23, 22), (16, 23, 24), (16, 24, 2), (16, 24, 3), (16, 24, 9), (16, 24, 10), (16, 24, 17), (16, 24, 23), (17, 2, 3), (17, 2, 9), (17, 2, 10), (17, 2, 16), (17, 2, 23), (17, 2, 24), (17, 3, 2), (17, 3, 4), (17, 3, 9), (17, 3, 10), (17, 3, 11), (17, 3, 16), (17, 3, 18), (17, 3, 23), (17, 3, 24), (17, 3, 25), (17, 4, 3), (17, 4, 10), (17, 4, 11), (17, 4, 18), (17, 4, 24), (17, 4, 25), (17, 9, 2), (17, 9, 3), (17, 9, 10), (17, 9, 16), (17, 9, 23), (17, 9, 24), (17, 10, 2), (17, 10, 3), (17, 10, 4), (17, 10, 9), (17, 10, 11), (17, 10, 16), (17, 10, 18), (17, 10, 23), (17, 10, 24), (17, 10, 25), (17, 11, 3), (17, 11, 4), (17, 11, 10), (17, 11, 18), (17, 11, 24), (17, 11, 25), (17, 16, 2), (17, 16, 3), (17, 16, 9), (17, 16, 10), (17, 16, 23), (17, 16, 24), (17, 18, 3), (17, 18, 4), (17, 18, 10), (17, 18, 11), (17, 18, 24), (17, 18, 25), (17, 23, 2), (17, 23, 3), (17, 23, 9), (17, 23, 10), (17, 23, 16), (17, 23, 24), (17, 24, 2), (17, 24, 3), (17, 24, 4), (17, 24, 9), (17, 24, 10), (17, 24, 11), (17, 24, 16), (17, 24, 18), (17, 24, 23), (17, 24, 25), (17, 25, 3), (17, 25, 4), (17, 25, 10), (17, 25, 11), (17, 25, 18), (17, 25, 24), (18, 3, 4), (18, 3, 10), (18, 3, 11), (18, 3, 17), (18, 3, 24), (18, 3, 25), (18, 4, 3), (18, 4, 5), (18, 4, 10), (18, 4, 11), (18, 4, 12), (18, 4, 17), (18, 4, 19), (18, 4, 24), (18, 4, 25), (18, 4, 26), (18, 5, 4), (18, 5, 11), (18, 5, 12), (18, 5, 19), (18, 5, 25), (18, 5, 26), (18, 10, 3), (18, 10, 4), (18, 10, 11), (18, 10, 17), (18, 10, 24), (18, 10, 25), (18, 11, 3), (18, 11, 4), (18, 11, 5), (18, 11, 10), (18, 11, 12), (18, 11, 17), (18, 11, 19), (18, 11, 24), (18, 11, 25), (18, 11, 26), (18, 12, 4), (18, 12, 5), (18, 12, 11), (18, 12, 19), (18, 12, 25), (18, 12, 26), (18, 17, 3), (18, 17, 4), (18, 17, 10), (18, 17, 11), (18, 17, 24), (18, 17, 25), (18, 19, 4), (18, 19, 5), (18, 19, 11), (18, 19, 12), (18, 19, 25), (18, 19, 26), (18, 24, 3), (18, 24, 4), (18, 24, 10), (18, 24, 11), (18, 24, 17), (18, 24, 25), (18, 25, 3), (18, 25, 4), (18, 25, 5), (18, 25, 10), (18, 25, 11), (18, 25, 12), (18, 25, 17), (18, 25, 19), (18, 25, 24), (18, 25, 26), (18, 26, 4), (18, 26, 5), (18, 26, 11), (18, 26, 12), (18, 26, 19), (18, 26, 25), (19, 4, 5), (19, 4, 11), (19, 4, 12), (19, 4, 18), (19, 4, 25), (19, 4, 26), (19, 5, 4), (19, 5, 6), (19, 5, 11), (19, 5, 12), (19, 5, 13), (19, 5, 18), (19, 5, 20), (19, 5, 25), (19, 5, 26), (19, 5, 27), (19, 6, 5), (19, 6, 12), (19, 6, 13), (19, 6, 20), (19, 6, 26), (19, 6, 27), (19, 11, 4), (19, 11, 5), (19, 11, 12), (19, 11, 18), (19, 11, 25), (19, 11, 26), (19, 12, 4), (19, 12, 5), (19, 12, 6), (19, 12, 11), (19, 12, 13), (19, 12, 18), (19, 12, 20), (19, 12, 25), (19, 12, 26), (19, 12, 27), (19, 13, 5), (19, 13, 6), (19, 13, 12), (19, 13, 20), (19, 13, 26), (19, 13, 27), (19, 18, 4), (19, 18, 5), (19, 18, 11), (19, 18, 12), (19, 18, 25), (19, 18, 26), (19, 20, 5), (19, 20, 6), (19, 20, 12), (19, 20, 13), (19, 20, 26), (19, 20, 27), (19, 25, 4), (19, 25, 5), (19, 25, 11), (19, 25, 12), (19, 25, 18), (19, 25, 26), (19, 26, 4), (19, 26, 5), (19, 26, 6), (19, 26, 11), (19, 26, 12), (19, 26, 13), (19, 26, 18), (19, 26, 20), (19, 26, 25), (19, 26, 27), (19, 27, 5), (19, 27, 6), (19, 27, 12), (19, 27, 13), (19, 27, 20), (19, 27, 26), (20, 0, 6), (20, 0, 7), (20, 0, 13), (20, 0, 14), (20, 0, 21), (20, 0, 27), (20, 5, 6), (20, 5, 12), (20, 5, 13), (20, 5, 19), (20, 5, 26), (20, 5, 27), (20, 6, 0), (20, 6, 5), (20, 6, 7), (20, 6, 12), (20, 6, 13), (20, 6, 14), (20, 6, 19), (20, 6, 21), (20, 6, 26), (20, 6, 27), (20, 7, 0), (20, 7, 6), (20, 7, 13), (20, 7, 14), (20, 7, 21), (20, 7, 27), (20, 12, 5), (20, 12, 6), (20, 12, 13), (20, 12, 19), (20, 12, 26), (20, 12, 27), (20, 13, 0), (20, 13, 5), (20, 13, 6), (20, 13, 7), (20, 13, 12), (20, 13, 14), (20, 13, 19), (20, 13, 21), (20, 13, 26), (20, 13, 27), (20, 14, 0), (20, 14, 6), (20, 14, 7), (20, 14, 13), (20, 14, 21), (20, 14, 27), (20, 19, 5), (20, 19, 6), (20, 19, 12), (20, 19, 13), (20, 19, 26), (20, 19, 27), (20, 21, 0), (20, 21, 6), (20, 21, 7), (20, 21, 13), (20, 21, 14), (20, 21, 27), (20, 26, 5), (20, 26, 6), (20, 26, 12), (20, 26, 13), (20, 26, 19), (20, 26, 27), (20, 27, 0), (20, 27, 5), (20, 27, 6), (20, 27, 7), (20, 27, 12), (20, 27, 13), (20, 27, 14), (20, 27, 19), (20, 27, 21), (20, 27, 26), (21, 0, 1), (21, 0, 6), (21, 0, 7), (21, 0, 8), (21, 0, 13), (21, 0, 14), (21, 0, 15), (21, 0, 20), (21, 0, 22), (21, 0, 27), (21, 1, 0), (21, 1, 7), (21, 1, 8), (21, 1, 14), (21, 1, 15), (21, 1, 22), (21, 6, 0), (21, 6, 7), (21, 6, 13), (21, 6, 14), (21, 6, 20), (21, 6, 27), (21, 7, 0), (21, 7, 1), (21, 7, 6), (21, 7, 8), (21, 7, 13), (21, 7, 14), (21, 7, 15), (21, 7, 20), (21, 7, 22), (21, 7, 27), (21, 8, 0), (21, 8, 1), (21, 8, 7), (21, 8, 14), (21, 8, 15), (21, 8, 22), (21, 13, 0), (21, 13, 6), (21, 13, 7), (21, 13, 14), (21, 13, 20), (21, 13, 27), (21, 14, 0), (21, 14, 1), (21, 14, 6), (21, 14, 7), (21, 14, 8), (21, 14, 13), (21, 14, 15), (21, 14, 20), (21, 14, 22), (21, 14, 27), (21, 15, 0), (21, 15, 1), (21, 15, 7), (21, 15, 8), (21, 15, 14), (21, 15, 22), (21, 20, 0), (21, 20, 6), (21, 20, 7), (21, 20, 13), (21, 20, 14), (21, 20, 27), (21, 22, 0), (21, 22, 1), (21, 22, 7), (21, 22, 8), (21, 22, 14), (21, 22, 15), (21, 27, 0), (21, 27, 6), (21, 27, 7), (21, 27, 13), (21, 27, 14), (21, 27, 20), (22, 0, 1), (22, 0, 7), (22, 0, 8), (22, 0, 14), (22, 0, 15), (22, 0, 21), (22, 1, 0), (22, 1, 2), (22, 1, 7), (22, 1, 8), (22, 1, 9), (22, 1, 14), (22, 1, 15), (22, 1, 16), (22, 1, 21), (22, 1, 23), (22, 2, 1), (22, 2, 8), (22, 2, 9), (22, 2, 15), (22, 2, 16), (22, 2, 23), (22, 7, 0), (22, 7, 1), (22, 7, 8), (22, 7, 14), (22, 7, 15), (22, 7, 21), (22, 8, 0), (22, 8, 1), (22, 8, 2), (22, 8, 7), (22, 8, 9), (22, 8, 14), (22, 8, 15), (22, 8, 16), (22, 8, 21), (22, 8, 23), (22, 9, 1), (22, 9, 2), (22, 9, 8), (22, 9, 15), (22, 9, 16), (22, 9, 23), (22, 14, 0), (22, 14, 1), (22, 14, 7), (22, 14, 8), (22, 14, 15), (22, 14, 21), (22, 15, 0), (22, 15, 1), (22, 15, 2), (22, 15, 7), (22, 15, 8), (22, 15, 9), (22, 15, 14), (22, 15, 16), (22, 15, 21), (22, 15, 23), (22, 16, 1), (22, 16, 2), (22, 16, 8), (22, 16, 9), (22, 16, 15), (22, 16, 23), (22, 21, 0), (22, 21, 1), (22, 21, 7), (22, 21, 8), (22, 21, 14), (22, 21, 15), (22, 23, 1), (22, 23, 2), (22, 23, 8), (22, 23, 9), (22, 23, 15), (22, 23, 16), (23, 1, 2), (23, 1, 8), (23, 1, 9), (23, 1, 15), (23, 1, 16), (23, 1, 22), (23, 2, 1), (23, 2, 3), (23, 2, 8), (23, 2, 9), (23, 2, 10), (23, 2, 15), (23, 2, 16), (23, 2, 17), (23, 2, 22), (23, 2, 24), (23, 3, 2), (23, 3, 9), (23, 3, 10), (23, 3, 16), (23, 3, 17), (23, 3, 24), (23, 8, 1), (23, 8, 2), (23, 8, 9), (23, 8, 15), (23, 8, 16), (23, 8, 22), (23, 9, 1), (23, 9, 2), (23, 9, 3), (23, 9, 8), (23, 9, 10), (23, 9, 15), (23, 9, 16), (23, 9, 17), (23, 9, 22), (23, 9, 24), (23, 10, 2), (23, 10, 3), (23, 10, 9), (23, 10, 16), (23, 10, 17), (23, 10, 24), (23, 15, 1), (23, 15, 2), (23, 15, 8), (23, 15, 9), (23, 15, 16), (23, 15, 22), (23, 16, 1), (23, 16, 2), (23, 16, 3), (23, 16, 8), (23, 16, 9), (23, 16, 10), (23, 16, 15), (23, 16, 17), (23, 16, 22), (23, 16, 24), (23, 17, 2), (23, 17, 3), (23, 17, 9), (23, 17, 10), (23, 17, 16), (23, 17, 24), (23, 22, 1), (23, 22, 2), (23, 22, 8), (23, 22, 9), (23, 22, 15), (23, 22, 16), (23, 24, 2), (23, 24, 3), (23, 24, 9), (23, 24, 10), (23, 24, 16), (23, 24, 17), (24, 2, 3), (24, 2, 9), (24, 2, 10), (24, 2, 16), (24, 2, 17), (24, 2, 23), (24, 3, 2), (24, 3, 4), (24, 3, 9), (24, 3, 10), (24, 3, 11), (24, 3, 16), (24, 3, 17), (24, 3, 18), (24, 3, 23), (24, 3, 25), (24, 4, 3), (24, 4, 10), (24, 4, 11), (24, 4, 17), (24, 4, 18), (24, 4, 25), (24, 9, 2), (24, 9, 3), (24, 9, 10), (24, 9, 16), (24, 9, 17), (24, 9, 23), (24, 10, 2), (24, 10, 3), (24, 10, 4), (24, 10, 9), (24, 10, 11), (24, 10, 16), (24, 10, 17), (24, 10, 18), (24, 10, 23), (24, 10, 25), (24, 11, 3), (24, 11, 4), (24, 11, 10), (24, 11, 17), (24, 11, 18), (24, 11, 25), (24, 16, 2), (24, 16, 3), (24, 16, 9), (24, 16, 10), (24, 16, 17), (24, 16, 23), (24, 17, 2), (24, 17, 3), (24, 17, 4), (24, 17, 9), (24, 17, 10), (24, 17, 11), (24, 17, 16), (24, 17, 18), (24, 17, 23), (24, 17, 25), (24, 18, 3), (24, 18, 4), (24, 18, 10), (24, 18, 11), (24, 18, 17), (24, 18, 25), (24, 23, 2), (24, 23, 3), (24, 23, 9), (24, 23, 10), (24, 23, 16), (24, 23, 17), (24, 25, 3), (24, 25, 4), (24, 25, 10), (24, 25, 11), (24, 25, 17), (24, 25, 18), (25, 3, 4), (25, 3, 10), (25, 3, 11), (25, 3, 17), (25, 3, 18), (25, 3, 24), (25, 4, 3), (25, 4, 5), (25, 4, 10), (25, 4, 11), (25, 4, 12), (25, 4, 17), (25, 4, 18), (25, 4, 19), (25, 4, 24), (25, 4, 26), (25, 5, 4), (25, 5, 11), (25, 5, 12), (25, 5, 18), (25, 5, 19), (25, 5, 26), (25, 10, 3), (25, 10, 4), (25, 10, 11), (25, 10, 17), (25, 10, 18), (25, 10, 24), (25, 11, 3), (25, 11, 4), (25, 11, 5), (25, 11, 10), (25, 11, 12), (25, 11, 17), (25, 11, 18), (25, 11, 19), (25, 11, 24), (25, 11, 26), (25, 12, 4), (25, 12, 5), (25, 12, 11), (25, 12, 18), (25, 12, 19), (25, 12, 26), (25, 17, 3), (25, 17, 4), (25, 17, 10), (25, 17, 11), (25, 17, 18), (25, 17, 24), (25, 18, 3), (25, 18, 4), (25, 18, 5), (25, 18, 10), (25, 18, 11), (25, 18, 12), (25, 18, 17), (25, 18, 19), (25, 18, 24), (25, 18, 26), (25, 19, 4), (25, 19, 5), (25, 19, 11), (25, 19, 12), (25, 19, 18), (25, 19, 26), (25, 24, 3), (25, 24, 4), (25, 24, 10), (25, 24, 11), (25, 24, 17), (25, 24, 18), (25, 26, 4), (25, 26, 5), (25, 26, 11), (25, 26, 12), (25, 26, 18), (25, 26, 19), (26, 4, 5), (26, 4, 11), (26, 4, 12), (26, 4, 18), (26, 4, 19), (26, 4, 25), (26, 5, 4), (26, 5, 6), (26, 5, 11), (26, 5, 12), (26, 5, 13), (26, 5, 18), (26, 5, 19), (26, 5, 20), (26, 5, 25), (26, 5, 27), (26, 6, 5), (26, 6, 12), (26, 6, 13), (26, 6, 19), (26, 6, 20), (26, 6, 27), (26, 11, 4), (26, 11, 5), (26, 11, 12), (26, 11, 18), (26, 11, 19), (26, 11, 25), (26, 12, 4), (26, 12, 5), (26, 12, 6), (26, 12, 11), (26, 12, 13), (26, 12, 18), (26, 12, 19), (26, 12, 20), (26, 12, 25), (26, 12, 27), (26, 13, 5), (26, 13, 6), (26, 13, 12), (26, 13, 19), (26, 13, 20), (26, 13, 27), (26, 18, 4), (26, 18, 5), (26, 18, 11), (26, 18, 12), (26, 18, 19), (26, 18, 25), (26, 19, 4), (26, 19, 5), (26, 19, 6), (26, 19, 11), (26, 19, 12), (26, 19, 13), (26, 19, 18), (26, 19, 20), (26, 19, 25), (26, 19, 27), (26, 20, 5), (26, 20, 6), (26, 20, 12), (26, 20, 13), (26, 20, 19), (26, 20, 27), (26, 25, 4), (26, 25, 5), (26, 25, 11), (26, 25, 12), (26, 25, 18), (26, 25, 19), (26, 27, 5), (26, 27, 6), (26, 27, 12), (26, 27, 13), (26, 27, 19), (26, 27, 20), (27, 0, 6), (27, 0, 7), (27, 0, 13), (27, 0, 14), (27, 0, 20), (27, 0, 21), (27, 5, 6), (27, 5, 12), (27, 5, 13), (27, 5, 19), (27, 5, 20), (27, 5, 26), (27, 6, 0), (27, 6, 5), (27, 6, 7), (27, 6, 12), (27, 6, 13), (27, 6, 14), (27, 6, 19), (27, 6, 20), (27, 6, 21), (27, 6, 26), (27, 7, 0), (27, 7, 6), (27, 7, 13), (27, 7, 14), (27, 7, 20), (27, 7, 21), (27, 12, 5), (27, 12, 6), (27, 12, 13), (27, 12, 19), (27, 12, 20), (27, 12, 26), (27, 13, 0), (27, 13, 5), (27, 13, 6), (27, 13, 7), (27, 13, 12), (27, 13, 14), (27, 13, 19), (27, 13, 20), (27, 13, 21), (27, 13, 26), (27, 14, 0), (27, 14, 6), (27, 14, 7), (27, 14, 13), (27, 14, 20), (27, 14, 21), (27, 19, 5), (27, 19, 6), (27, 19, 12), (27, 19, 13), (27, 19, 20), (27, 19, 26), (27, 20, 0), (27, 20, 5), (27, 20, 6), (27, 20, 7), (27, 20, 12), (27, 20, 13), (27, 20, 14), (27, 20, 19), (27, 20, 21), (27, 20, 26), (27, 21, 0), (27, 21, 6), (27, 21, 7), (27, 21, 13), (27, 21, 14), (27, 21, 20), (27, 26, 5), (27, 26, 6), (27, 26, 12), (27, 26, 13), (27, 26, 19), (27, 26, 20)]

    #all 5 cycles in the c7k4 strong product with starting node 0
    fivecycles_list = [(0, 1, 2, 8, 7),(0, 1, 2, 8, 14),(0, 1, 2, 8, 15),(0, 1, 2, 8, 21),(0, 1, 2, 8, 22),(0, 1, 2, 9, 8),(0, 1, 2, 9, 15),(0, 1, 2, 9, 22),(0, 1, 2, 15, 7),(0, 1, 2, 15, 8),(0, 1, 2, 15, 14),(0, 1, 2, 15, 21),(0, 1, 2, 15, 22),(0, 1, 2, 16, 8),(0, 1, 2, 16, 15),(0, 1, 2, 16, 22),(0, 1, 2, 22, 7),(0, 1, 2, 22, 8),(0, 1, 2, 22, 14),(0, 1, 2, 22, 15),(0, 1, 2, 22, 21),(0, 1, 2, 23, 8),(0, 1, 2, 23, 15),(0, 1, 2, 23, 22),(0, 1, 7, 6, 13),(0, 1, 7, 6, 14),(0, 1, 7, 6, 20),(0, 1, 7, 6, 21),(0, 1, 7, 6, 27),(0, 1, 7, 8, 14),(0, 1, 7, 8, 15),(0, 1, 7, 8, 21),(0, 1, 7, 8, 22),(0, 1, 7, 13, 6),(0, 1, 7, 13, 14),(0, 1, 7, 13, 20),(0, 1, 7, 13, 21),(0, 1, 7, 13, 27),(0, 1, 7, 14, 6),(0, 1, 7, 14, 8),(0, 1, 7, 14, 13),(0, 1, 7, 14, 15),(0, 1, 7, 14, 20),(0, 1, 7, 14, 21),(0, 1, 7, 14, 22),(0, 1, 7, 14, 27),(0, 1, 7, 15, 8),(0, 1, 7, 15, 14),(0, 1, 7, 15, 21),(0, 1, 7, 15, 22),(0, 1, 7, 20, 6),(0, 1, 7, 20, 13),(0, 1, 7, 20, 14),(0, 1, 7, 20, 21),(0, 1, 7, 20, 27),(0, 1, 7, 21, 6),(0, 1, 7, 21, 8),(0, 1, 7, 21, 13),(0, 1, 7, 21, 14),(0, 1, 7, 21, 15),(0, 1, 7, 21, 20),(0, 1, 7, 21, 22),(0, 1, 7, 21, 27),(0, 1, 7, 22, 8),(0, 1, 7, 22, 14),(0, 1, 7, 22, 15),(0, 1, 7, 22, 21),(0, 1, 7, 27, 6),(0, 1, 7, 27, 13),(0, 1, 7, 27, 14),(0, 1, 7, 27, 20),(0, 1, 7, 27, 21),(0, 1, 8, 2, 15),(0, 1, 8, 2, 22),(0, 1, 8, 7, 6),(0, 1, 8, 7, 13),(0, 1, 8, 7, 14),(0, 1, 8, 7, 15),(0, 1, 8, 7, 20),(0, 1, 8, 7, 21),(0, 1, 8, 7, 22),(0, 1, 8, 7, 27),(0, 1, 8, 9, 15),(0, 1, 8, 9, 22),(0, 1, 8, 14, 6),(0, 1, 8, 14, 7),(0, 1, 8, 14, 13),(0, 1, 8, 14, 15),(0, 1, 8, 14, 20),(0, 1, 8, 14, 21),(0, 1, 8, 14, 22),(0, 1, 8, 14, 27),(0, 1, 8, 15, 7),(0, 1, 8, 15, 14),(0, 1, 8, 15, 21),(0, 1, 8, 15, 22),(0, 1, 8, 16, 15),(0, 1, 8, 16, 22),(0, 1, 8, 21, 6),(0, 1, 8, 21, 7),(0, 1, 8, 21, 13),(0, 1, 8, 21, 14),(0, 1, 8, 21, 15),(0, 1, 8, 21, 20),(0, 1, 8, 21, 22),(0, 1, 8, 21, 27),(0, 1, 8, 22, 7),(0, 1, 8, 22, 14),(0, 1, 8, 22, 15),(0, 1, 8, 22, 21),(0, 1, 8, 23, 15),(0, 1, 8, 23, 22),(0, 1, 9, 2, 8),(0, 1, 9, 2, 15),(0, 1, 9, 2, 22),(0, 1, 9, 8, 7),(0, 1, 9, 8, 14),(0, 1, 9, 8, 15),(0, 1, 9, 8, 21),(0, 1, 9, 8, 22),(0, 1, 9, 15, 7),(0, 1, 9, 15, 8),(0, 1, 9, 15, 14),(0, 1, 9, 15, 21),(0, 1, 9, 15, 22),(0, 1, 9, 16, 8),(0, 1, 9, 16, 15),(0, 1, 9, 16, 22),(0, 1, 9, 22, 7),(0, 1, 9, 22, 8),(0, 1, 9, 22, 14),(0, 1, 9, 22, 15),(0, 1, 9, 22, 21),(0, 1, 9, 23, 8),(0, 1, 9, 23, 15),(0, 1, 9, 23, 22),(0, 1, 14, 6, 7),(0, 1, 14, 6, 13),(0, 1, 14, 6, 20),(0, 1, 14, 6, 21),(0, 1, 14, 6, 27),(0, 1, 14, 7, 6),(0, 1, 14, 7, 8),(0, 1, 14, 7, 13),(0, 1, 14, 7, 15),(0, 1, 14, 7, 20),(0, 1, 14, 7, 21),(0, 1, 14, 7, 22),(0, 1, 14, 7, 27),(0, 1, 14, 8, 7),(0, 1, 14, 8, 15),(0, 1, 14, 8, 21),(0, 1, 14, 8, 22),(0, 1, 14, 13, 6),(0, 1, 14, 13, 7),(0, 1, 14, 13, 20),(0, 1, 14, 13, 21),(0, 1, 14, 13, 27),(0, 1, 14, 15, 7),(0, 1, 14, 15, 8),(0, 1, 14, 15, 21),(0, 1, 14, 15, 22),(0, 1, 14, 20, 6),(0, 1, 14, 20, 7),(0, 1, 14, 20, 13),(0, 1, 14, 20, 21),(0, 1, 14, 20, 27),(0, 1, 14, 21, 6),(0, 1, 14, 21, 7),(0, 1, 14, 21, 8),(0, 1, 14, 21, 13),(0, 1, 14, 21, 15),(0, 1, 14, 21, 20),(0, 1, 14, 21, 22),(0, 1, 14, 21, 27),(0, 1, 14, 22, 7),(0, 1, 14, 22, 8),(0, 1, 14, 22, 15),(0, 1, 14, 22, 21),(0, 1, 14, 27, 6),(0, 1, 14, 27, 7),(0, 1, 14, 27, 13),(0, 1, 14, 27, 20),(0, 1, 14, 27, 21),(0, 1, 15, 2, 8),(0, 1, 15, 2, 22),(0, 1, 15, 7, 6),(0, 1, 15, 7, 8),(0, 1, 15, 7, 13),(0, 1, 15, 7, 14),(0, 1, 15, 7, 20),(0, 1, 15, 7, 21),(0, 1, 15, 7, 22),(0, 1, 15, 7, 27),(0, 1, 15, 8, 7),(0, 1, 15, 8, 14),(0, 1, 15, 8, 21),(0, 1, 15, 8, 22),(0, 1, 15, 9, 8),(0, 1, 15, 9, 22),(0, 1, 15, 14, 6),(0, 1, 15, 14, 7),(0, 1, 15, 14, 8),(0, 1, 15, 14, 13),(0, 1, 15, 14, 20),(0, 1, 15, 14, 21),(0, 1, 15, 14, 22),(0, 1, 15, 14, 27),(0, 1, 15, 16, 8),(0, 1, 15, 16, 22),(0, 1, 15, 21, 6),(0, 1, 15, 21, 7),(0, 1, 15, 21, 8),(0, 1, 15, 21, 13),(0, 1, 15, 21, 14),(0, 1, 15, 21, 20),(0, 1, 15, 21, 22),(0, 1, 15, 21, 27),(0, 1, 15, 22, 7),(0, 1, 15, 22, 8),(0, 1, 15, 22, 14),(0, 1, 15, 22, 21),(0, 1, 15, 23, 8),(0, 1, 15, 23, 22),(0, 1, 16, 2, 8),(0, 1, 16, 2, 15),(0, 1, 16, 2, 22),(0, 1, 16, 8, 7),(0, 1, 16, 8, 14),(0, 1, 16, 8, 15),(0, 1, 16, 8, 21),(0, 1, 16, 8, 22),(0, 1, 16, 9, 8),(0, 1, 16, 9, 15),(0, 1, 16, 9, 22),(0, 1, 16, 15, 7),(0, 1, 16, 15, 8),(0, 1, 16, 15, 14),(0, 1, 16, 15, 21),(0, 1, 16, 15, 22),(0, 1, 16, 22, 7),(0, 1, 16, 22, 8),(0, 1, 16, 22, 14),(0, 1, 16, 22, 15),(0, 1, 16, 22, 21),(0, 1, 16, 23, 8),(0, 1, 16, 23, 15),(0, 1, 16, 23, 22),(0, 1, 21, 6, 7),(0, 1, 21, 6, 13),(0, 1, 21, 6, 14),(0, 1, 21, 6, 20),(0, 1, 21, 6, 27),(0, 1, 21, 7, 6),(0, 1, 21, 7, 8),(0, 1, 21, 7, 13),(0, 1, 21, 7, 14),(0, 1, 21, 7, 15),(0, 1, 21, 7, 20),(0, 1, 21, 7, 22),(0, 1, 21, 7, 27),(0, 1, 21, 8, 7),(0, 1, 21, 8, 14),(0, 1, 21, 8, 15),(0, 1, 21, 8, 22),(0, 1, 21, 13, 6),(0, 1, 21, 13, 7),(0, 1, 21, 13, 14),(0, 1, 21, 13, 20),(0, 1, 21, 13, 27),(0, 1, 21, 14, 6),(0, 1, 21, 14, 7),(0, 1, 21, 14, 8),(0, 1, 21, 14, 13),(0, 1, 21, 14, 15),(0, 1, 21, 14, 20),(0, 1, 21, 14, 22),(0, 1, 21, 14, 27),(0, 1, 21, 15, 7),(0, 1, 21, 15, 8),(0, 1, 21, 15, 14),(0, 1, 21, 15, 22),(0, 1, 21, 20, 6),(0, 1, 21, 20, 7),(0, 1, 21, 20, 13),(0, 1, 21, 20, 14),(0, 1, 21, 20, 27),(0, 1, 21, 22, 7),(0, 1, 21, 22, 8),(0, 1, 21, 22, 14),(0, 1, 21, 22, 15),(0, 1, 21, 27, 6),(0, 1, 21, 27, 7),(0, 1, 21, 27, 13),(0, 1, 21, 27, 14),(0, 1, 21, 27, 20),(0, 1, 22, 2, 8),(0, 1, 22, 2, 15),(0, 1, 22, 7, 6),(0, 1, 22, 7, 8),(0, 1, 22, 7, 13),(0, 1, 22, 7, 14),(0, 1, 22, 7, 15),(0, 1, 22, 7, 20),(0, 1, 22, 7, 21),(0, 1, 22, 7, 27),(0, 1, 22, 8, 7),(0, 1, 22, 8, 14),(0, 1, 22, 8, 15),(0, 1, 22, 8, 21),(0, 1, 22, 9, 8),(0, 1, 22, 9, 15),(0, 1, 22, 14, 6),(0, 1, 22, 14, 7),(0, 1, 22, 14, 8),(0, 1, 22, 14, 13),(0, 1, 22, 14, 15),(0, 1, 22, 14, 20),(0, 1, 22, 14, 21),(0, 1, 22, 14, 27),(0, 1, 22, 15, 7),(0, 1, 22, 15, 8),(0, 1, 22, 15, 14),(0, 1, 22, 15, 21),(0, 1, 22, 16, 8),(0, 1, 22, 16, 15),(0, 1, 22, 21, 6),(0, 1, 22, 21, 7),(0, 1, 22, 21, 8),(0, 1, 22, 21, 13),(0, 1, 22, 21, 14),(0, 1, 22, 21, 15),(0, 1, 22, 21, 20),(0, 1, 22, 21, 27),(0, 1, 22, 23, 8),(0, 1, 22, 23, 15),(0, 1, 23, 2, 8),(0, 1, 23, 2, 15),(0, 1, 23, 2, 22),(0, 1, 23, 8, 7),(0, 1, 23, 8, 14),(0, 1, 23, 8, 15),(0, 1, 23, 8, 21),(0, 1, 23, 8, 22),(0, 1, 23, 9, 8),(0, 1, 23, 9, 15),(0, 1, 23, 9, 22),(0, 1, 23, 15, 7),(0, 1, 23, 15, 8),(0, 1, 23, 15, 14),(0, 1, 23, 15, 21),(0, 1, 23, 15, 22),(0, 1, 23, 16, 8),(0, 1, 23, 16, 15),(0, 1, 23, 16, 22),(0, 1, 23, 22, 7),(0, 1, 23, 22, 8),(0, 1, 23, 22, 14),(0, 1, 23, 22, 15),(0, 1, 23, 22, 21),(0, 6, 5, 12, 13),(0, 6, 5, 12, 20),(0, 6, 5, 12, 27),(0, 6, 5, 13, 7),(0, 6, 5, 13, 14),(0, 6, 5, 13, 20),(0, 6, 5, 13, 21),(0, 6, 5, 13, 27),(0, 6, 5, 19, 13),(0, 6, 5, 19, 20),(0, 6, 5, 19, 27),(0, 6, 5, 20, 7),(0, 6, 5, 20, 13),(0, 6, 5, 20, 14),(0, 6, 5, 20, 21),(0, 6, 5, 20, 27),(0, 6, 5, 26, 13),(0, 6, 5, 26, 20),(0, 6, 5, 26, 27),(0, 6, 5, 27, 7),(0, 6, 5, 27, 13),(0, 6, 5, 27, 14),(0, 6, 5, 27, 20),(0, 6, 5, 27, 21),(0, 6, 7, 1, 8),(0, 6, 7, 1, 14),(0, 6, 7, 1, 15),(0, 6, 7, 1, 21),(0, 6, 7, 1, 22),(0, 6, 7, 8, 1),(0, 6, 7, 8, 14),(0, 6, 7, 8, 15),(0, 6, 7, 8, 21),(0, 6, 7, 8, 22),(0, 6, 7, 13, 14),(0, 6, 7, 13, 20),(0, 6, 7, 13, 21),(0, 6, 7, 13, 27),(0, 6, 7, 14, 1),(0, 6, 7, 14, 8),(0, 6, 7, 14, 13),(0, 6, 7, 14, 15),(0, 6, 7, 14, 20),(0, 6, 7, 14, 21),(0, 6, 7, 14, 22),(0, 6, 7, 14, 27),(0, 6, 7, 15, 1),(0, 6, 7, 15, 8),(0, 6, 7, 15, 14),(0, 6, 7, 15, 21),(0, 6, 7, 15, 22),(0, 6, 7, 20, 13),(0, 6, 7, 20, 14),(0, 6, 7, 20, 21),(0, 6, 7, 20, 27),(0, 6, 7, 21, 1),(0, 6, 7, 21, 8),(0, 6, 7, 21, 13),(0, 6, 7, 21, 14),(0, 6, 7, 21, 15),(0, 6, 7, 21, 20),(0, 6, 7, 21, 22),(0, 6, 7, 21, 27),(0, 6, 7, 22, 1),(0, 6, 7, 22, 8),(0, 6, 7, 22, 14),(0, 6, 7, 22, 15),(0, 6, 7, 22, 21),(0, 6, 7, 27, 13),(0, 6, 7, 27, 14),(0, 6, 7, 27, 20),(0, 6, 7, 27, 21),(0, 6, 12, 5, 13),(0, 6, 12, 5, 20),(0, 6, 12, 5, 27),(0, 6, 12, 13, 7),(0, 6, 12, 13, 14),(0, 6, 12, 13, 20),(0, 6, 12, 13, 21),(0, 6, 12, 13, 27),(0, 6, 12, 19, 13),(0, 6, 12, 19, 20),(0, 6, 12, 19, 27),(0, 6, 12, 20, 7),(0, 6, 12, 20, 13),(0, 6, 12, 20, 14),(0, 6, 12, 20, 21),(0, 6, 12, 20, 27),(0, 6, 12, 26, 13),(0, 6, 12, 26, 20),(0, 6, 12, 26, 27),(0, 6, 12, 27, 7),(0, 6, 12, 27, 13),(0, 6, 12, 27, 14),(0, 6, 12, 27, 20),(0, 6, 12, 27, 21),(0, 6, 13, 5, 20),(0, 6, 13, 5, 27),(0, 6, 13, 7, 1),(0, 6, 13, 7, 8),(0, 6, 13, 7, 14),(0, 6, 13, 7, 15),(0, 6, 13, 7, 20),(0, 6, 13, 7, 21),(0, 6, 13, 7, 22),(0, 6, 13, 7, 27),(0, 6, 13, 12, 20),(0, 6, 13, 12, 27),(0, 6, 13, 14, 1),(0, 6, 13, 14, 7),(0, 6, 13, 14, 8),(0, 6, 13, 14, 15),(0, 6, 13, 14, 20),(0, 6, 13, 14, 21),(0, 6, 13, 14, 22),(0, 6, 13, 14, 27),(0, 6, 13, 19, 20),(0, 6, 13, 19, 27),(0, 6, 13, 20, 7),(0, 6, 13, 20, 14),(0, 6, 13, 20, 21),(0, 6, 13, 20, 27),(0, 6, 13, 21, 1),(0, 6, 13, 21, 7),(0, 6, 13, 21, 8),(0, 6, 13, 21, 14),(0, 6, 13, 21, 15),(0, 6, 13, 21, 20),(0, 6, 13, 21, 22),(0, 6, 13, 21, 27),(0, 6, 13, 26, 20),(0, 6, 13, 26, 27),(0, 6, 13, 27, 7),(0, 6, 13, 27, 14),(0, 6, 13, 27, 20),(0, 6, 13, 27, 21),(0, 6, 14, 1, 7),(0, 6, 14, 1, 8),(0, 6, 14, 1, 15),(0, 6, 14, 1, 21),(0, 6, 14, 1, 22),(0, 6, 14, 7, 1),(0, 6, 14, 7, 8),(0, 6, 14, 7, 13),(0, 6, 14, 7, 15),(0, 6, 14, 7, 20),(0, 6, 14, 7, 21),(0, 6, 14, 7, 22),(0, 6, 14, 7, 27),(0, 6, 14, 8, 1),(0, 6, 14, 8, 7),(0, 6, 14, 8, 15),(0, 6, 14, 8, 21),(0, 6, 14, 8, 22),(0, 6, 14, 13, 7),(0, 6, 14, 13, 20),(0, 6, 14, 13, 21),(0, 6, 14, 13, 27),(0, 6, 14, 15, 1),(0, 6, 14, 15, 7),(0, 6, 14, 15, 8),(0, 6, 14, 15, 21),(0, 6, 14, 15, 22),(0, 6, 14, 20, 7),(0, 6, 14, 20, 13),(0, 6, 14, 20, 21),(0, 6, 14, 20, 27),(0, 6, 14, 21, 1),(0, 6, 14, 21, 7),(0, 6, 14, 21, 8),(0, 6, 14, 21, 13),(0, 6, 14, 21, 15),(0, 6, 14, 21, 20),(0, 6, 14, 21, 22),(0, 6, 14, 21, 27),(0, 6, 14, 22, 1),(0, 6, 14, 22, 7),(0, 6, 14, 22, 8),(0, 6, 14, 22, 15),(0, 6, 14, 22, 21),(0, 6, 14, 27, 7),(0, 6, 14, 27, 13),(0, 6, 14, 27, 20),(0, 6, 14, 27, 21),(0, 6, 19, 5, 13),(0, 6, 19, 5, 20),(0, 6, 19, 5, 27),(0, 6, 19, 12, 13),(0, 6, 19, 12, 20),(0, 6, 19, 12, 27),(0, 6, 19, 13, 7),(0, 6, 19, 13, 14),(0, 6, 19, 13, 20),(0, 6, 19, 13, 21),(0, 6, 19, 13, 27),(0, 6, 19, 20, 7),(0, 6, 19, 20, 13),(0, 6, 19, 20, 14),(0, 6, 19, 20, 21),(0, 6, 19, 20, 27),(0, 6, 19, 26, 13),(0, 6, 19, 26, 20),(0, 6, 19, 26, 27),(0, 6, 19, 27, 7),(0, 6, 19, 27, 13),(0, 6, 19, 27, 14),(0, 6, 19, 27, 20),(0, 6, 19, 27, 21),(0, 6, 20, 5, 13),(0, 6, 20, 5, 27),(0, 6, 20, 7, 1),(0, 6, 20, 7, 8),(0, 6, 20, 7, 13),(0, 6, 20, 7, 14),(0, 6, 20, 7, 15),(0, 6, 20, 7, 21),(0, 6, 20, 7, 22),(0, 6, 20, 7, 27),(0, 6, 20, 12, 13),(0, 6, 20, 12, 27),(0, 6, 20, 13, 7),(0, 6, 20, 13, 14),(0, 6, 20, 13, 21),(0, 6, 20, 13, 27),(0, 6, 20, 14, 1),(0, 6, 20, 14, 7),(0, 6, 20, 14, 8),(0, 6, 20, 14, 13),(0, 6, 20, 14, 15),(0, 6, 20, 14, 21),(0, 6, 20, 14, 22),(0, 6, 20, 14, 27),(0, 6, 20, 19, 13),(0, 6, 20, 19, 27),(0, 6, 20, 21, 1),(0, 6, 20, 21, 7),(0, 6, 20, 21, 8),(0, 6, 20, 21, 13),(0, 6, 20, 21, 14),(0, 6, 20, 21, 15),(0, 6, 20, 21, 22),(0, 6, 20, 21, 27),(0, 6, 20, 26, 13),(0, 6, 20, 26, 27),(0, 6, 20, 27, 7),(0, 6, 20, 27, 13),(0, 6, 20, 27, 14),(0, 6, 20, 27, 21),(0, 6, 21, 1, 7),(0, 6, 21, 1, 8),(0, 6, 21, 1, 14),(0, 6, 21, 1, 15),(0, 6, 21, 1, 22),(0, 6, 21, 7, 1),(0, 6, 21, 7, 8),(0, 6, 21, 7, 13),(0, 6, 21, 7, 14),(0, 6, 21, 7, 15),(0, 6, 21, 7, 20),(0, 6, 21, 7, 22),(0, 6, 21, 7, 27),(0, 6, 21, 8, 1),(0, 6, 21, 8, 7),(0, 6, 21, 8, 14),(0, 6, 21, 8, 15),(0, 6, 21, 8, 22),(0, 6, 21, 13, 7),(0, 6, 21, 13, 14),(0, 6, 21, 13, 20),(0, 6, 21, 13, 27),(0, 6, 21, 14, 1),(0, 6, 21, 14, 7),(0, 6, 21, 14, 8),(0, 6, 21, 14, 13),(0, 6, 21, 14, 15),(0, 6, 21, 14, 20),(0, 6, 21, 14, 22),(0, 6, 21, 14, 27),(0, 6, 21, 15, 1),(0, 6, 21, 15, 7),(0, 6, 21, 15, 8),(0, 6, 21, 15, 14),(0, 6, 21, 15, 22),(0, 6, 21, 20, 7),(0, 6, 21, 20, 13),(0, 6, 21, 20, 14),(0, 6, 21, 20, 27),(0, 6, 21, 22, 1),(0, 6, 21, 22, 7),(0, 6, 21, 22, 8),(0, 6, 21, 22, 14),(0, 6, 21, 22, 15),(0, 6, 21, 27, 7),(0, 6, 21, 27, 13),(0, 6, 21, 27, 14),(0, 6, 21, 27, 20),(0, 6, 26, 5, 13),(0, 6, 26, 5, 20),(0, 6, 26, 5, 27),(0, 6, 26, 12, 13),(0, 6, 26, 12, 20),(0, 6, 26, 12, 27),(0, 6, 26, 13, 7),(0, 6, 26, 13, 14),(0, 6, 26, 13, 20),(0, 6, 26, 13, 21),(0, 6, 26, 13, 27),(0, 6, 26, 19, 13),(0, 6, 26, 19, 20),(0, 6, 26, 19, 27),(0, 6, 26, 20, 7),(0, 6, 26, 20, 13),(0, 6, 26, 20, 14),(0, 6, 26, 20, 21),(0, 6, 26, 20, 27),(0, 6, 26, 27, 7),(0, 6, 26, 27, 13),(0, 6, 26, 27, 14),(0, 6, 26, 27, 20),(0, 6, 26, 27, 21),(0, 6, 27, 5, 13),(0, 6, 27, 5, 20),(0, 6, 27, 7, 1),(0, 6, 27, 7, 8),(0, 6, 27, 7, 13),(0, 6, 27, 7, 14),(0, 6, 27, 7, 15),(0, 6, 27, 7, 20),(0, 6, 27, 7, 21),(0, 6, 27, 7, 22),(0, 6, 27, 12, 13),(0, 6, 27, 12, 20),(0, 6, 27, 13, 7),(0, 6, 27, 13, 14),(0, 6, 27, 13, 20),(0, 6, 27, 13, 21),(0, 6, 27, 14, 1),(0, 6, 27, 14, 7),(0, 6, 27, 14, 8),(0, 6, 27, 14, 13),(0, 6, 27, 14, 15),(0, 6, 27, 14, 20),(0, 6, 27, 14, 21),(0, 6, 27, 14, 22),(0, 6, 27, 19, 13),(0, 6, 27, 19, 20),(0, 6, 27, 20, 7),(0, 6, 27, 20, 13),(0, 6, 27, 20, 14),(0, 6, 27, 20, 21),(0, 6, 27, 21, 1),(0, 6, 27, 21, 7),(0, 6, 27, 21, 8),(0, 6, 27, 21, 13),(0, 6, 27, 21, 14),(0, 6, 27, 21, 15),(0, 6, 27, 21, 20),(0, 6, 27, 21, 22),(0, 6, 27, 26, 13),(0, 6, 27, 26, 20),(0, 7, 1, 2, 8),(0, 7, 1, 2, 15),(0, 7, 1, 2, 22),(0, 7, 1, 8, 14),(0, 7, 1, 8, 15),(0, 7, 1, 8, 21),(0, 7, 1, 8, 22),(0, 7, 1, 9, 8),(0, 7, 1, 9, 15),(0, 7, 1, 9, 22),(0, 7, 1, 14, 6),(0, 7, 1, 14, 8),(0, 7, 1, 14, 13),(0, 7, 1, 14, 15),(0, 7, 1, 14, 20),(0, 7, 1, 14, 21),(0, 7, 1, 14, 22),(0, 7, 1, 14, 27),(0, 7, 1, 15, 8),(0, 7, 1, 15, 14),(0, 7, 1, 15, 21),(0, 7, 1, 15, 22),(0, 7, 1, 16, 8),(0, 7, 1, 16, 15),(0, 7, 1, 16, 22),(0, 7, 1, 21, 6),(0, 7, 1, 21, 8),(0, 7, 1, 21, 13),(0, 7, 1, 21, 14),(0, 7, 1, 21, 15),(0, 7, 1, 21, 20),(0, 7, 1, 21, 22),(0, 7, 1, 21, 27),(0, 7, 1, 22, 8),(0, 7, 1, 22, 14),(0, 7, 1, 22, 15),(0, 7, 1, 22, 21),(0, 7, 1, 23, 8),(0, 7, 1, 23, 15),(0, 7, 1, 23, 22),(0, 7, 6, 5, 13),(0, 7, 6, 5, 20),(0, 7, 6, 5, 27),(0, 7, 6, 12, 13),(0, 7, 6, 12, 20),(0, 7, 6, 12, 27),(0, 7, 6, 13, 14),(0, 7, 6, 13, 20),(0, 7, 6, 13, 21),(0, 7, 6, 13, 27),(0, 7, 6, 14, 1),(0, 7, 6, 14, 8),(0, 7, 6, 14, 13),(0, 7, 6, 14, 15),(0, 7, 6, 14, 20),(0, 7, 6, 14, 21),(0, 7, 6, 14, 22),(0, 7, 6, 14, 27),(0, 7, 6, 19, 13),(0, 7, 6, 19, 20),(0, 7, 6, 19, 27),(0, 7, 6, 20, 13),(0, 7, 6, 20, 14),(0, 7, 6, 20, 21),(0, 7, 6, 20, 27),(0, 7, 6, 21, 1),(0, 7, 6, 21, 8),(0, 7, 6, 21, 13),(0, 7, 6, 21, 14),(0, 7, 6, 21, 15),(0, 7, 6, 21, 20),(0, 7, 6, 21, 22),(0, 7, 6, 21, 27),(0, 7, 6, 26, 13),(0, 7, 6, 26, 20),(0, 7, 6, 26, 27),(0, 7, 6, 27, 13),(0, 7, 6, 27, 14),(0, 7, 6, 27, 20),(0, 7, 6, 27, 21),(0, 7, 8, 1, 14),(0, 7, 8, 1, 15),(0, 7, 8, 1, 21),(0, 7, 8, 1, 22),(0, 7, 8, 2, 1),(0, 7, 8, 2, 15),(0, 7, 8, 2, 22),(0, 7, 8, 9, 1),(0, 7, 8, 9, 15),(0, 7, 8, 9, 22),(0, 7, 8, 14, 1),(0, 7, 8, 14, 6),(0, 7, 8, 14, 13),(0, 7, 8, 14, 15),(0, 7, 8, 14, 20),(0, 7, 8, 14, 21),(0, 7, 8, 14, 22),(0, 7, 8, 14, 27),(0, 7, 8, 15, 1),(0, 7, 8, 15, 14),(0, 7, 8, 15, 21),(0, 7, 8, 15, 22),(0, 7, 8, 16, 1),(0, 7, 8, 16, 15),(0, 7, 8, 16, 22),(0, 7, 8, 21, 1),(0, 7, 8, 21, 6),(0, 7, 8, 21, 13),(0, 7, 8, 21, 14),(0, 7, 8, 21, 15),(0, 7, 8, 21, 20),(0, 7, 8, 21, 22),(0, 7, 8, 21, 27),(0, 7, 8, 22, 1),(0, 7, 8, 22, 14),(0, 7, 8, 22, 15),(0, 7, 8, 22, 21),(0, 7, 8, 23, 1),(0, 7, 8, 23, 15),(0, 7, 8, 23, 22),(0, 7, 13, 5, 6),(0, 7, 13, 5, 20),(0, 7, 13, 5, 27),(0, 7, 13, 6, 14),(0, 7, 13, 6, 20),(0, 7, 13, 6, 21),(0, 7, 13, 6, 27),(0, 7, 13, 12, 6),(0, 7, 13, 12, 20),(0, 7, 13, 12, 27),(0, 7, 13, 14, 1),(0, 7, 13, 14, 6),(0, 7, 13, 14, 8),(0, 7, 13, 14, 15),(0, 7, 13, 14, 20),(0, 7, 13, 14, 21),(0, 7, 13, 14, 22),(0, 7, 13, 14, 27),(0, 7, 13, 19, 6),(0, 7, 13, 19, 20),(0, 7, 13, 19, 27),(0, 7, 13, 20, 6),(0, 7, 13, 20, 14),(0, 7, 13, 20, 21),(0, 7, 13, 20, 27),(0, 7, 13, 21, 1),(0, 7, 13, 21, 6),(0, 7, 13, 21, 8),(0, 7, 13, 21, 14),(0, 7, 13, 21, 15),(0, 7, 13, 21, 20),(0, 7, 13, 21, 22),(0, 7, 13, 21, 27),(0, 7, 13, 26, 6),(0, 7, 13, 26, 20),(0, 7, 13, 26, 27),(0, 7, 13, 27, 6),(0, 7, 13, 27, 14),(0, 7, 13, 27, 20),(0, 7, 13, 27, 21),(0, 7, 14, 1, 8),(0, 7, 14, 1, 15),(0, 7, 14, 1, 21),(0, 7, 14, 1, 22),(0, 7, 14, 6, 13),(0, 7, 14, 6, 20),(0, 7, 14, 6, 21),(0, 7, 14, 6, 27),(0, 7, 14, 8, 1),(0, 7, 14, 8, 15),(0, 7, 14, 8, 21),(0, 7, 14, 8, 22),(0, 7, 14, 13, 6),(0, 7, 14, 13, 20),(0, 7, 14, 13, 21),(0, 7, 14, 13, 27),(0, 7, 14, 15, 1),(0, 7, 14, 15, 8),(0, 7, 14, 15, 21),(0, 7, 14, 15, 22),(0, 7, 14, 20, 6),(0, 7, 14, 20, 13),(0, 7, 14, 20, 21),(0, 7, 14, 20, 27),(0, 7, 14, 21, 1),(0, 7, 14, 21, 6),(0, 7, 14, 21, 8),(0, 7, 14, 21, 13),(0, 7, 14, 21, 15),(0, 7, 14, 21, 20),(0, 7, 14, 21, 22),(0, 7, 14, 21, 27),(0, 7, 14, 22, 1),(0, 7, 14, 22, 8),(0, 7, 14, 22, 15),(0, 7, 14, 22, 21),(0, 7, 14, 27, 6),(0, 7, 14, 27, 13),(0, 7, 14, 27, 20),(0, 7, 14, 27, 21),(0, 7, 15, 1, 8),(0, 7, 15, 1, 14),(0, 7, 15, 1, 21),(0, 7, 15, 1, 22),(0, 7, 15, 2, 1),(0, 7, 15, 2, 8),(0, 7, 15, 2, 22),(0, 7, 15, 8, 1),(0, 7, 15, 8, 14),(0, 7, 15, 8, 21),(0, 7, 15, 8, 22),(0, 7, 15, 9, 1),(0, 7, 15, 9, 8),(0, 7, 15, 9, 22),(0, 7, 15, 14, 1),(0, 7, 15, 14, 6),(0, 7, 15, 14, 8),(0, 7, 15, 14, 13),(0, 7, 15, 14, 20),(0, 7, 15, 14, 21),(0, 7, 15, 14, 22),(0, 7, 15, 14, 27),(0, 7, 15, 16, 1),(0, 7, 15, 16, 8),(0, 7, 15, 16, 22),(0, 7, 15, 21, 1),(0, 7, 15, 21, 6),(0, 7, 15, 21, 8),(0, 7, 15, 21, 13),(0, 7, 15, 21, 14),(0, 7, 15, 21, 20),(0, 7, 15, 21, 22),(0, 7, 15, 21, 27),(0, 7, 15, 22, 1),(0, 7, 15, 22, 8),(0, 7, 15, 22, 14),(0, 7, 15, 22, 21),(0, 7, 15, 23, 1),(0, 7, 15, 23, 8),(0, 7, 15, 23, 22),(0, 7, 20, 5, 6),(0, 7, 20, 5, 13),(0, 7, 20, 5, 27),(0, 7, 20, 6, 13),(0, 7, 20, 6, 14),(0, 7, 20, 6, 21),(0, 7, 20, 6, 27),(0, 7, 20, 12, 6),(0, 7, 20, 12, 13),(0, 7, 20, 12, 27),(0, 7, 20, 13, 6),(0, 7, 20, 13, 14),(0, 7, 20, 13, 21),(0, 7, 20, 13, 27),(0, 7, 20, 14, 1),(0, 7, 20, 14, 6),(0, 7, 20, 14, 8),(0, 7, 20, 14, 13),(0, 7, 20, 14, 15),(0, 7, 20, 14, 21),(0, 7, 20, 14, 22),(0, 7, 20, 14, 27),(0, 7, 20, 19, 6),(0, 7, 20, 19, 13),(0, 7, 20, 19, 27),(0, 7, 20, 21, 1),(0, 7, 20, 21, 6),(0, 7, 20, 21, 8),(0, 7, 20, 21, 13),(0, 7, 20, 21, 14),(0, 7, 20, 21, 15),(0, 7, 20, 21, 22),(0, 7, 20, 21, 27),(0, 7, 20, 26, 6),(0, 7, 20, 26, 13),(0, 7, 20, 26, 27),(0, 7, 20, 27, 6),(0, 7, 20, 27, 13),(0, 7, 20, 27, 14),(0, 7, 20, 27, 21),(0, 7, 21, 1, 8),(0, 7, 21, 1, 14),(0, 7, 21, 1, 15),(0, 7, 21, 1, 22),(0, 7, 21, 6, 13),(0, 7, 21, 6, 14),(0, 7, 21, 6, 20),(0, 7, 21, 6, 27),(0, 7, 21, 8, 1),(0, 7, 21, 8, 14),(0, 7, 21, 8, 15),(0, 7, 21, 8, 22),(0, 7, 21, 13, 6),(0, 7, 21, 13, 14),(0, 7, 21, 13, 20),(0, 7, 21, 13, 27),(0, 7, 21, 14, 1),(0, 7, 21, 14, 6),(0, 7, 21, 14, 8),(0, 7, 21, 14, 13),(0, 7, 21, 14, 15),(0, 7, 21, 14, 20),(0, 7, 21, 14, 22),(0, 7, 21, 14, 27),(0, 7, 21, 15, 1),(0, 7, 21, 15, 8),(0, 7, 21, 15, 14),(0, 7, 21, 15, 22),(0, 7, 21, 20, 6),(0, 7, 21, 20, 13),(0, 7, 21, 20, 14),(0, 7, 21, 20, 27),(0, 7, 21, 22, 1),(0, 7, 21, 22, 8),(0, 7, 21, 22, 14),(0, 7, 21, 22, 15),(0, 7, 21, 27, 6),(0, 7, 21, 27, 13),(0, 7, 21, 27, 14),(0, 7, 21, 27, 20),(0, 7, 22, 1, 8),(0, 7, 22, 1, 14),(0, 7, 22, 1, 15),(0, 7, 22, 1, 21),(0, 7, 22, 2, 1),(0, 7, 22, 2, 8),(0, 7, 22, 2, 15),(0, 7, 22, 8, 1),(0, 7, 22, 8, 14),(0, 7, 22, 8, 15),(0, 7, 22, 8, 21),(0, 7, 22, 9, 1),(0, 7, 22, 9, 8),(0, 7, 22, 9, 15),(0, 7, 22, 14, 1),(0, 7, 22, 14, 6),(0, 7, 22, 14, 8),(0, 7, 22, 14, 13),(0, 7, 22, 14, 15),(0, 7, 22, 14, 20),(0, 7, 22, 14, 21),(0, 7, 22, 14, 27),(0, 7, 22, 15, 1),(0, 7, 22, 15, 8),(0, 7, 22, 15, 14),(0, 7, 22, 15, 21),(0, 7, 22, 16, 1),(0, 7, 22, 16, 8),(0, 7, 22, 16, 15),(0, 7, 22, 21, 1),(0, 7, 22, 21, 6),(0, 7, 22, 21, 8),(0, 7, 22, 21, 13),(0, 7, 22, 21, 14),(0, 7, 22, 21, 15),(0, 7, 22, 21, 20),(0, 7, 22, 21, 27),(0, 7, 22, 23, 1),(0, 7, 22, 23, 8),(0, 7, 22, 23, 15),(0, 7, 27, 5, 6),(0, 7, 27, 5, 13),(0, 7, 27, 5, 20),(0, 7, 27, 6, 13),(0, 7, 27, 6, 14),(0, 7, 27, 6, 20),(0, 7, 27, 6, 21),(0, 7, 27, 12, 6),(0, 7, 27, 12, 13),(0, 7, 27, 12, 20),(0, 7, 27, 13, 6),(0, 7, 27, 13, 14),(0, 7, 27, 13, 20),(0, 7, 27, 13, 21),(0, 7, 27, 14, 1),(0, 7, 27, 14, 6),(0, 7, 27, 14, 8),(0, 7, 27, 14, 13),(0, 7, 27, 14, 15),(0, 7, 27, 14, 20),(0, 7, 27, 14, 21),(0, 7, 27, 14, 22),(0, 7, 27, 19, 6),(0, 7, 27, 19, 13),(0, 7, 27, 19, 20),(0, 7, 27, 20, 6),(0, 7, 27, 20, 13),(0, 7, 27, 20, 14),(0, 7, 27, 20, 21),(0, 7, 27, 21, 1),(0, 7, 27, 21, 6),(0, 7, 27, 21, 8),(0, 7, 27, 21, 13),(0, 7, 27, 21, 14),(0, 7, 27, 21, 15),(0, 7, 27, 21, 20),(0, 7, 27, 21, 22),(0, 7, 27, 26, 6),(0, 7, 27, 26, 13),(0, 7, 27, 26, 20),(0, 8, 1, 2, 15),(0, 8, 1, 2, 22),(0, 8, 1, 7, 6),(0, 8, 1, 7, 13),(0, 8, 1, 7, 14),(0, 8, 1, 7, 15),(0, 8, 1, 7, 20),(0, 8, 1, 7, 21),(0, 8, 1, 7, 22),(0, 8, 1, 7, 27),(0, 8, 1, 9, 15),(0, 8, 1, 9, 22),(0, 8, 1, 14, 6),(0, 8, 1, 14, 7),(0, 8, 1, 14, 13),(0, 8, 1, 14, 15),(0, 8, 1, 14, 20),(0, 8, 1, 14, 21),(0, 8, 1, 14, 22),(0, 8, 1, 14, 27),(0, 8, 1, 15, 7),(0, 8, 1, 15, 14),(0, 8, 1, 15, 21),(0, 8, 1, 15, 22),(0, 8, 1, 16, 15),(0, 8, 1, 16, 22),(0, 8, 1, 21, 6),(0, 8, 1, 21, 7),(0, 8, 1, 21, 13),(0, 8, 1, 21, 14),(0, 8, 1, 21, 15),(0, 8, 1, 21, 20),(0, 8, 1, 21, 22),(0, 8, 1, 21, 27),(0, 8, 1, 22, 7),(0, 8, 1, 22, 14),(0, 8, 1, 22, 15),(0, 8, 1, 22, 21),(0, 8, 1, 23, 15),(0, 8, 1, 23, 22),(0, 8, 2, 1, 7),(0, 8, 2, 1, 14),(0, 8, 2, 1, 15),(0, 8, 2, 1, 21),(0, 8, 2, 1, 22),(0, 8, 2, 9, 1),(0, 8, 2, 9, 15),(0, 8, 2, 9, 22),(0, 8, 2, 15, 1),(0, 8, 2, 15, 7),(0, 8, 2, 15, 14),(0, 8, 2, 15, 21),(0, 8, 2, 15, 22),(0, 8, 2, 16, 1),(0, 8, 2, 16, 15),(0, 8, 2, 16, 22),(0, 8, 2, 22, 1),(0, 8, 2, 22, 7),(0, 8, 2, 22, 14),(0, 8, 2, 22, 15),(0, 8, 2, 22, 21),(0, 8, 2, 23, 1),(0, 8, 2, 23, 15),(0, 8, 2, 23, 22),(0, 8, 7, 1, 14),(0, 8, 7, 1, 15),(0, 8, 7, 1, 21),(0, 8, 7, 1, 22),(0, 8, 7, 6, 13),(0, 8, 7, 6, 14),(0, 8, 7, 6, 20),(0, 8, 7, 6, 21),(0, 8, 7, 6, 27),(0, 8, 7, 13, 6),(0, 8, 7, 13, 14),(0, 8, 7, 13, 20),(0, 8, 7, 13, 21),(0, 8, 7, 13, 27),(0, 8, 7, 14, 1),(0, 8, 7, 14, 6),(0, 8, 7, 14, 13),(0, 8, 7, 14, 15),(0, 8, 7, 14, 20),(0, 8, 7, 14, 21),(0, 8, 7, 14, 22),(0, 8, 7, 14, 27),(0, 8, 7, 15, 1),(0, 8, 7, 15, 14),(0, 8, 7, 15, 21),(0, 8, 7, 15, 22),(0, 8, 7, 20, 6),(0, 8, 7, 20, 13),(0, 8, 7, 20, 14),(0, 8, 7, 20, 21),(0, 8, 7, 20, 27),(0, 8, 7, 21, 1),(0, 8, 7, 21, 6),(0, 8, 7, 21, 13),(0, 8, 7, 21, 14),(0, 8, 7, 21, 15),(0, 8, 7, 21, 20),(0, 8, 7, 21, 22),(0, 8, 7, 21, 27),(0, 8, 7, 22, 1),(0, 8, 7, 22, 14),(0, 8, 7, 22, 15),(0, 8, 7, 22, 21),(0, 8, 7, 27, 6),(0, 8, 7, 27, 13),(0, 8, 7, 27, 14),(0, 8, 7, 27, 20),(0, 8, 7, 27, 21),(0, 8, 9, 1, 7),(0, 8, 9, 1, 14),(0, 8, 9, 1, 15),(0, 8, 9, 1, 21),(0, 8, 9, 1, 22),(0, 8, 9, 2, 1),(0, 8, 9, 2, 15),(0, 8, 9, 2, 22),(0, 8, 9, 15, 1),(0, 8, 9, 15, 7),(0, 8, 9, 15, 14),(0, 8, 9, 15, 21),(0, 8, 9, 15, 22),(0, 8, 9, 16, 1),(0, 8, 9, 16, 15),(0, 8, 9, 16, 22),(0, 8, 9, 22, 1),(0, 8, 9, 22, 7),(0, 8, 9, 22, 14),(0, 8, 9, 22, 15),(0, 8, 9, 22, 21),(0, 8, 9, 23, 1),(0, 8, 9, 23, 15),(0, 8, 9, 23, 22),(0, 8, 14, 1, 7),(0, 8, 14, 1, 15),(0, 8, 14, 1, 21),(0, 8, 14, 1, 22),(0, 8, 14, 6, 7),(0, 8, 14, 6, 13),(0, 8, 14, 6, 20),(0, 8, 14, 6, 21),(0, 8, 14, 6, 27),(0, 8, 14, 7, 1),(0, 8, 14, 7, 6),(0, 8, 14, 7, 13),(0, 8, 14, 7, 15),(0, 8, 14, 7, 20),(0, 8, 14, 7, 21),(0, 8, 14, 7, 22),(0, 8, 14, 7, 27),(0, 8, 14, 13, 6),(0, 8, 14, 13, 7),(0, 8, 14, 13, 20),(0, 8, 14, 13, 21),(0, 8, 14, 13, 27),(0, 8, 14, 15, 1),(0, 8, 14, 15, 7),(0, 8, 14, 15, 21),(0, 8, 14, 15, 22),(0, 8, 14, 20, 6),(0, 8, 14, 20, 7),(0, 8, 14, 20, 13),(0, 8, 14, 20, 21),(0, 8, 14, 20, 27),(0, 8, 14, 21, 1),(0, 8, 14, 21, 6),(0, 8, 14, 21, 7),(0, 8, 14, 21, 13),(0, 8, 14, 21, 15),(0, 8, 14, 21, 20),(0, 8, 14, 21, 22),(0, 8, 14, 21, 27),(0, 8, 14, 22, 1),(0, 8, 14, 22, 7),(0, 8, 14, 22, 15),(0, 8, 14, 22, 21),(0, 8, 14, 27, 6),(0, 8, 14, 27, 7),(0, 8, 14, 27, 13),(0, 8, 14, 27, 20),(0, 8, 14, 27, 21),(0, 8, 15, 1, 7),(0, 8, 15, 1, 14),(0, 8, 15, 1, 21),(0, 8, 15, 1, 22),(0, 8, 15, 2, 1),(0, 8, 15, 2, 22),(0, 8, 15, 7, 1),(0, 8, 15, 7, 6),(0, 8, 15, 7, 13),(0, 8, 15, 7, 14),(0, 8, 15, 7, 20),(0, 8, 15, 7, 21),(0, 8, 15, 7, 22),(0, 8, 15, 7, 27),(0, 8, 15, 9, 1),(0, 8, 15, 9, 22),(0, 8, 15, 14, 1),(0, 8, 15, 14, 6),(0, 8, 15, 14, 7),(0, 8, 15, 14, 13),(0, 8, 15, 14, 20),(0, 8, 15, 14, 21),(0, 8, 15, 14, 22),(0, 8, 15, 14, 27),(0, 8, 15, 16, 1),(0, 8, 15, 16, 22),(0, 8, 15, 21, 1),(0, 8, 15, 21, 6),(0, 8, 15, 21, 7),(0, 8, 15, 21, 13),(0, 8, 15, 21, 14),(0, 8, 15, 21, 20),(0, 8, 15, 21, 22),(0, 8, 15, 21, 27),(0, 8, 15, 22, 1),(0, 8, 15, 22, 7),(0, 8, 15, 22, 14),(0, 8, 15, 22, 21),(0, 8, 15, 23, 1),(0, 8, 15, 23, 22),(0, 8, 16, 1, 7),(0, 8, 16, 1, 14),(0, 8, 16, 1, 15),(0, 8, 16, 1, 21),(0, 8, 16, 1, 22),(0, 8, 16, 2, 1),(0, 8, 16, 2, 15),(0, 8, 16, 2, 22),(0, 8, 16, 9, 1),(0, 8, 16, 9, 15),(0, 8, 16, 9, 22),(0, 8, 16, 15, 1),(0, 8, 16, 15, 7),(0, 8, 16, 15, 14),(0, 8, 16, 15, 21),(0, 8, 16, 15, 22),(0, 8, 16, 22, 1),(0, 8, 16, 22, 7),(0, 8, 16, 22, 14),(0, 8, 16, 22, 15),(0, 8, 16, 22, 21),(0, 8, 16, 23, 1),(0, 8, 16, 23, 15),(0, 8, 16, 23, 22),(0, 8, 21, 1, 7),(0, 8, 21, 1, 14),(0, 8, 21, 1, 15),(0, 8, 21, 1, 22),(0, 8, 21, 6, 7),(0, 8, 21, 6, 13),(0, 8, 21, 6, 14),(0, 8, 21, 6, 20),(0, 8, 21, 6, 27),(0, 8, 21, 7, 1),(0, 8, 21, 7, 6),(0, 8, 21, 7, 13),(0, 8, 21, 7, 14),(0, 8, 21, 7, 15),(0, 8, 21, 7, 20),(0, 8, 21, 7, 22),(0, 8, 21, 7, 27),(0, 8, 21, 13, 6),(0, 8, 21, 13, 7),(0, 8, 21, 13, 14),(0, 8, 21, 13, 20),(0, 8, 21, 13, 27),(0, 8, 21, 14, 1),(0, 8, 21, 14, 6),(0, 8, 21, 14, 7),(0, 8, 21, 14, 13),(0, 8, 21, 14, 15),(0, 8, 21, 14, 20),(0, 8, 21, 14, 22),(0, 8, 21, 14, 27),(0, 8, 21, 15, 1),(0, 8, 21, 15, 7),(0, 8, 21, 15, 14),(0, 8, 21, 15, 22),(0, 8, 21, 20, 6),(0, 8, 21, 20, 7),(0, 8, 21, 20, 13),(0, 8, 21, 20, 14),(0, 8, 21, 20, 27),(0, 8, 21, 22, 1),(0, 8, 21, 22, 7),(0, 8, 21, 22, 14),(0, 8, 21, 22, 15),(0, 8, 21, 27, 6),(0, 8, 21, 27, 7),(0, 8, 21, 27, 13),(0, 8, 21, 27, 14),(0, 8, 21, 27, 20),(0, 8, 22, 1, 7),(0, 8, 22, 1, 14),(0, 8, 22, 1, 15),(0, 8, 22, 1, 21),(0, 8, 22, 2, 1),(0, 8, 22, 2, 15),(0, 8, 22, 7, 1),(0, 8, 22, 7, 6),(0, 8, 22, 7, 13),(0, 8, 22, 7, 14),(0, 8, 22, 7, 15),(0, 8, 22, 7, 20),(0, 8, 22, 7, 21),(0, 8, 22, 7, 27),(0, 8, 22, 9, 1),(0, 8, 22, 9, 15),(0, 8, 22, 14, 1),(0, 8, 22, 14, 6),(0, 8, 22, 14, 7),(0, 8, 22, 14, 13),(0, 8, 22, 14, 15),(0, 8, 22, 14, 20),(0, 8, 22, 14, 21),(0, 8, 22, 14, 27),(0, 8, 22, 15, 1),(0, 8, 22, 15, 7),(0, 8, 22, 15, 14),(0, 8, 22, 15, 21),(0, 8, 22, 16, 1),(0, 8, 22, 16, 15),(0, 8, 22, 21, 1),(0, 8, 22, 21, 6),(0, 8, 22, 21, 7),(0, 8, 22, 21, 13),(0, 8, 22, 21, 14),(0, 8, 22, 21, 15),(0, 8, 22, 21, 20),(0, 8, 22, 21, 27),(0, 8, 22, 23, 1),(0, 8, 22, 23, 15),(0, 8, 23, 1, 7),(0, 8, 23, 1, 14),(0, 8, 23, 1, 15),(0, 8, 23, 1, 21),(0, 8, 23, 1, 22),(0, 8, 23, 2, 1),(0, 8, 23, 2, 15),(0, 8, 23, 2, 22),(0, 8, 23, 9, 1),(0, 8, 23, 9, 15),(0, 8, 23, 9, 22),(0, 8, 23, 15, 1),(0, 8, 23, 15, 7),(0, 8, 23, 15, 14),(0, 8, 23, 15, 21),(0, 8, 23, 15, 22),(0, 8, 23, 16, 1),(0, 8, 23, 16, 15),(0, 8, 23, 16, 22),(0, 8, 23, 22, 1),(0, 8, 23, 22, 7),(0, 8, 23, 22, 14),(0, 8, 23, 22, 15),(0, 8, 23, 22, 21),(0, 13, 5, 6, 7),(0, 13, 5, 6, 14),(0, 13, 5, 6, 20),(0, 13, 5, 6, 21),(0, 13, 5, 6, 27),(0, 13, 5, 12, 6),(0, 13, 5, 12, 20),(0, 13, 5, 12, 27),(0, 13, 5, 19, 6),(0, 13, 5, 19, 20),(0, 13, 5, 19, 27),(0, 13, 5, 20, 6),(0, 13, 5, 20, 7),(0, 13, 5, 20, 14),(0, 13, 5, 20, 21),(0, 13, 5, 20, 27),(0, 13, 5, 26, 6),(0, 13, 5, 26, 20),(0, 13, 5, 26, 27),(0, 13, 5, 27, 6),(0, 13, 5, 27, 7),(0, 13, 5, 27, 14),(0, 13, 5, 27, 20),(0, 13, 5, 27, 21),(0, 13, 6, 5, 20),(0, 13, 6, 5, 27),(0, 13, 6, 7, 1),(0, 13, 6, 7, 8),(0, 13, 6, 7, 14),(0, 13, 6, 7, 15),(0, 13, 6, 7, 20),(0, 13, 6, 7, 21),(0, 13, 6, 7, 22),(0, 13, 6, 7, 27),(0, 13, 6, 12, 20),(0, 13, 6, 12, 27),(0, 13, 6, 14, 1),(0, 13, 6, 14, 7),(0, 13, 6, 14, 8),(0, 13, 6, 14, 15),(0, 13, 6, 14, 20),(0, 13, 6, 14, 21),(0, 13, 6, 14, 22),(0, 13, 6, 14, 27),(0, 13, 6, 19, 20),(0, 13, 6, 19, 27),(0, 13, 6, 20, 7),(0, 13, 6, 20, 14),(0, 13, 6, 20, 21),(0, 13, 6, 20, 27),(0, 13, 6, 21, 1),(0, 13, 6, 21, 7),(0, 13, 6, 21, 8),(0, 13, 6, 21, 14),(0, 13, 6, 21, 15),(0, 13, 6, 21, 20),(0, 13, 6, 21, 22),(0, 13, 6, 21, 27),(0, 13, 6, 26, 20),(0, 13, 6, 26, 27),(0, 13, 6, 27, 7),(0, 13, 6, 27, 14),(0, 13, 6, 27, 20),(0, 13, 6, 27, 21),(0, 13, 7, 1, 8),(0, 13, 7, 1, 14),(0, 13, 7, 1, 15),(0, 13, 7, 1, 21),(0, 13, 7, 1, 22),(0, 13, 7, 6, 14),(0, 13, 7, 6, 20),(0, 13, 7, 6, 21),(0, 13, 7, 6, 27),(0, 13, 7, 8, 1),(0, 13, 7, 8, 14),(0, 13, 7, 8, 15),(0, 13, 7, 8, 21),(0, 13, 7, 8, 22),(0, 13, 7, 14, 1),(0, 13, 7, 14, 6),(0, 13, 7, 14, 8),(0, 13, 7, 14, 15),(0, 13, 7, 14, 20),(0, 13, 7, 14, 21),(0, 13, 7, 14, 22),(0, 13, 7, 14, 27),(0, 13, 7, 15, 1),(0, 13, 7, 15, 8),(0, 13, 7, 15, 14),(0, 13, 7, 15, 21),(0, 13, 7, 15, 22),(0, 13, 7, 20, 6),(0, 13, 7, 20, 14),(0, 13, 7, 20, 21),(0, 13, 7, 20, 27),(0, 13, 7, 21, 1),(0, 13, 7, 21, 6),(0, 13, 7, 21, 8),(0, 13, 7, 21, 14),(0, 13, 7, 21, 15),(0, 13, 7, 21, 20),(0, 13, 7, 21, 22),(0, 13, 7, 21, 27),(0, 13, 7, 22, 1),(0, 13, 7, 22, 8),(0, 13, 7, 22, 14),(0, 13, 7, 22, 15),(0, 13, 7, 22, 21),(0, 13, 7, 27, 6),(0, 13, 7, 27, 14),(0, 13, 7, 27, 20),(0, 13, 7, 27, 21),(0, 13, 12, 5, 6),(0, 13, 12, 5, 20),(0, 13, 12, 5, 27),(0, 13, 12, 6, 7),(0, 13, 12, 6, 14),(0, 13, 12, 6, 20),(0, 13, 12, 6, 21),(0, 13, 12, 6, 27),(0, 13, 12, 19, 6),(0, 13, 12, 19, 20),(0, 13, 12, 19, 27),(0, 13, 12, 20, 6),(0, 13, 12, 20, 7),(0, 13, 12, 20, 14),(0, 13, 12, 20, 21),(0, 13, 12, 20, 27),(0, 13, 12, 26, 6),(0, 13, 12, 26, 20),(0, 13, 12, 26, 27),(0, 13, 12, 27, 6),(0, 13, 12, 27, 7),(0, 13, 12, 27, 14),(0, 13, 12, 27, 20),(0, 13, 12, 27, 21),(0, 13, 14, 1, 7),(0, 13, 14, 1, 8),(0, 13, 14, 1, 15),(0, 13, 14, 1, 21),(0, 13, 14, 1, 22),(0, 13, 14, 6, 7),(0, 13, 14, 6, 20),(0, 13, 14, 6, 21),(0, 13, 14, 6, 27),(0, 13, 14, 7, 1),(0, 13, 14, 7, 6),(0, 13, 14, 7, 8),(0, 13, 14, 7, 15),(0, 13, 14, 7, 20),(0, 13, 14, 7, 21),(0, 13, 14, 7, 22),(0, 13, 14, 7, 27),(0, 13, 14, 8, 1),(0, 13, 14, 8, 7),(0, 13, 14, 8, 15),(0, 13, 14, 8, 21),(0, 13, 14, 8, 22),(0, 13, 14, 15, 1),(0, 13, 14, 15, 7),(0, 13, 14, 15, 8),(0, 13, 14, 15, 21),(0, 13, 14, 15, 22),(0, 13, 14, 20, 6),(0, 13, 14, 20, 7),(0, 13, 14, 20, 21),(0, 13, 14, 20, 27),(0, 13, 14, 21, 1),(0, 13, 14, 21, 6),(0, 13, 14, 21, 7),(0, 13, 14, 21, 8),(0, 13, 14, 21, 15),(0, 13, 14, 21, 20),(0, 13, 14, 21, 22),(0, 13, 14, 21, 27),(0, 13, 14, 22, 1),(0, 13, 14, 22, 7),(0, 13, 14, 22, 8),(0, 13, 14, 22, 15),(0, 13, 14, 22, 21),(0, 13, 14, 27, 6),(0, 13, 14, 27, 7),(0, 13, 14, 27, 20),(0, 13, 14, 27, 21),(0, 13, 19, 5, 6),(0, 13, 19, 5, 20),(0, 13, 19, 5, 27),(0, 13, 19, 6, 7),(0, 13, 19, 6, 14),(0, 13, 19, 6, 20),(0, 13, 19, 6, 21),(0, 13, 19, 6, 27),(0, 13, 19, 12, 6),(0, 13, 19, 12, 20),(0, 13, 19, 12, 27),(0, 13, 19, 20, 6),(0, 13, 19, 20, 7),(0, 13, 19, 20, 14),(0, 13, 19, 20, 21),(0, 13, 19, 20, 27),(0, 13, 19, 26, 6),(0, 13, 19, 26, 20),(0, 13, 19, 26, 27),(0, 13, 19, 27, 6),(0, 13, 19, 27, 7),(0, 13, 19, 27, 14),(0, 13, 19, 27, 20),(0, 13, 19, 27, 21),(0, 13, 20, 5, 6),(0, 13, 20, 5, 27),(0, 13, 20, 6, 7),(0, 13, 20, 6, 14),(0, 13, 20, 6, 21),(0, 13, 20, 6, 27),(0, 13, 20, 7, 1),(0, 13, 20, 7, 6),(0, 13, 20, 7, 8),(0, 13, 20, 7, 14),(0, 13, 20, 7, 15),(0, 13, 20, 7, 21),(0, 13, 20, 7, 22),(0, 13, 20, 7, 27),(0, 13, 20, 12, 6),(0, 13, 20, 12, 27),(0, 13, 20, 14, 1),(0, 13, 20, 14, 6),(0, 13, 20, 14, 7),(0, 13, 20, 14, 8),(0, 13, 20, 14, 15),(0, 13, 20, 14, 21),(0, 13, 20, 14, 22),(0, 13, 20, 14, 27),(0, 13, 20, 19, 6),(0, 13, 20, 19, 27),(0, 13, 20, 21, 1),(0, 13, 20, 21, 6),(0, 13, 20, 21, 7),(0, 13, 20, 21, 8),(0, 13, 20, 21, 14),(0, 13, 20, 21, 15),(0, 13, 20, 21, 22),(0, 13, 20, 21, 27),(0, 13, 20, 26, 6),(0, 13, 20, 26, 27),(0, 13, 20, 27, 6),(0, 13, 20, 27, 7),(0, 13, 20, 27, 14),(0, 13, 20, 27, 21),(0, 13, 21, 1, 7),(0, 13, 21, 1, 8),(0, 13, 21, 1, 14),(0, 13, 21, 1, 15),(0, 13, 21, 1, 22),(0, 13, 21, 6, 7),(0, 13, 21, 6, 14),(0, 13, 21, 6, 20),(0, 13, 21, 6, 27),(0, 13, 21, 7, 1),(0, 13, 21, 7, 6),(0, 13, 21, 7, 8),(0, 13, 21, 7, 14),(0, 13, 21, 7, 15),(0, 13, 21, 7, 20),(0, 13, 21, 7, 22),(0, 13, 21, 7, 27),(0, 13, 21, 8, 1),(0, 13, 21, 8, 7),(0, 13, 21, 8, 14),(0, 13, 21, 8, 15),(0, 13, 21, 8, 22),(0, 13, 21, 14, 1),(0, 13, 21, 14, 6),(0, 13, 21, 14, 7),(0, 13, 21, 14, 8),(0, 13, 21, 14, 15),(0, 13, 21, 14, 20),(0, 13, 21, 14, 22),(0, 13, 21, 14, 27),(0, 13, 21, 15, 1),(0, 13, 21, 15, 7),(0, 13, 21, 15, 8),(0, 13, 21, 15, 14),(0, 13, 21, 15, 22),(0, 13, 21, 20, 6),(0, 13, 21, 20, 7),(0, 13, 21, 20, 14),(0, 13, 21, 20, 27),(0, 13, 21, 22, 1),(0, 13, 21, 22, 7),(0, 13, 21, 22, 8),(0, 13, 21, 22, 14),(0, 13, 21, 22, 15),(0, 13, 21, 27, 6),(0, 13, 21, 27, 7),(0, 13, 21, 27, 14),(0, 13, 21, 27, 20),(0, 13, 26, 5, 6),(0, 13, 26, 5, 20),(0, 13, 26, 5, 27),(0, 13, 26, 6, 7),(0, 13, 26, 6, 14),(0, 13, 26, 6, 20),(0, 13, 26, 6, 21),(0, 13, 26, 6, 27),(0, 13, 26, 12, 6),(0, 13, 26, 12, 20),(0, 13, 26, 12, 27),(0, 13, 26, 19, 6),(0, 13, 26, 19, 20),(0, 13, 26, 19, 27),(0, 13, 26, 20, 6),(0, 13, 26, 20, 7),(0, 13, 26, 20, 14),(0, 13, 26, 20, 21),(0, 13, 26, 20, 27),(0, 13, 26, 27, 6),(0, 13, 26, 27, 7),(0, 13, 26, 27, 14),(0, 13, 26, 27, 20),(0, 13, 26, 27, 21),(0, 13, 27, 5, 6),(0, 13, 27, 5, 20),(0, 13, 27, 6, 7),(0, 13, 27, 6, 14),(0, 13, 27, 6, 20),(0, 13, 27, 6, 21),(0, 13, 27, 7, 1),(0, 13, 27, 7, 6),(0, 13, 27, 7, 8),(0, 13, 27, 7, 14),(0, 13, 27, 7, 15),(0, 13, 27, 7, 20),(0, 13, 27, 7, 21),(0, 13, 27, 7, 22),(0, 13, 27, 12, 6),(0, 13, 27, 12, 20),(0, 13, 27, 14, 1),(0, 13, 27, 14, 6),(0, 13, 27, 14, 7),(0, 13, 27, 14, 8),(0, 13, 27, 14, 15),(0, 13, 27, 14, 20),(0, 13, 27, 14, 21),(0, 13, 27, 14, 22),(0, 13, 27, 19, 6),(0, 13, 27, 19, 20),(0, 13, 27, 20, 6),(0, 13, 27, 20, 7),(0, 13, 27, 20, 14),(0, 13, 27, 20, 21),(0, 13, 27, 21, 1),(0, 13, 27, 21, 6),(0, 13, 27, 21, 7),(0, 13, 27, 21, 8),(0, 13, 27, 21, 14),(0, 13, 27, 21, 15),(0, 13, 27, 21, 20),(0, 13, 27, 21, 22),(0, 13, 27, 26, 6),(0, 13, 27, 26, 20),(0, 14, 1, 2, 8),(0, 14, 1, 2, 15),(0, 14, 1, 2, 22),(0, 14, 1, 7, 6),(0, 14, 1, 7, 8),(0, 14, 1, 7, 13),(0, 14, 1, 7, 15),(0, 14, 1, 7, 20),(0, 14, 1, 7, 21),(0, 14, 1, 7, 22),(0, 14, 1, 7, 27),(0, 14, 1, 8, 7),(0, 14, 1, 8, 15),(0, 14, 1, 8, 21),(0, 14, 1, 8, 22),(0, 14, 1, 9, 8),(0, 14, 1, 9, 15),(0, 14, 1, 9, 22),(0, 14, 1, 15, 7),(0, 14, 1, 15, 8),(0, 14, 1, 15, 21),(0, 14, 1, 15, 22),(0, 14, 1, 16, 8),(0, 14, 1, 16, 15),(0, 14, 1, 16, 22),(0, 14, 1, 21, 6),(0, 14, 1, 21, 7),(0, 14, 1, 21, 8),(0, 14, 1, 21, 13),(0, 14, 1, 21, 15),(0, 14, 1, 21, 20),(0, 14, 1, 21, 22),(0, 14, 1, 21, 27),(0, 14, 1, 22, 7),(0, 14, 1, 22, 8),(0, 14, 1, 22, 15),(0, 14, 1, 22, 21),(0, 14, 1, 23, 8),(0, 14, 1, 23, 15),(0, 14, 1, 23, 22),(0, 14, 6, 5, 13),(0, 14, 6, 5, 20),(0, 14, 6, 5, 27),(0, 14, 6, 7, 1),(0, 14, 6, 7, 8),(0, 14, 6, 7, 13),(0, 14, 6, 7, 15),(0, 14, 6, 7, 20),(0, 14, 6, 7, 21),(0, 14, 6, 7, 22),(0, 14, 6, 7, 27),(0, 14, 6, 12, 13),(0, 14, 6, 12, 20),(0, 14, 6, 12, 27),(0, 14, 6, 13, 7),(0, 14, 6, 13, 20),(0, 14, 6, 13, 21),(0, 14, 6, 13, 27),(0, 14, 6, 19, 13),(0, 14, 6, 19, 20),(0, 14, 6, 19, 27),(0, 14, 6, 20, 7),(0, 14, 6, 20, 13),(0, 14, 6, 20, 21),(0, 14, 6, 20, 27),(0, 14, 6, 21, 1),(0, 14, 6, 21, 7),(0, 14, 6, 21, 8),(0, 14, 6, 21, 13),(0, 14, 6, 21, 15),(0, 14, 6, 21, 20),(0, 14, 6, 21, 22),(0, 14, 6, 21, 27),(0, 14, 6, 26, 13),(0, 14, 6, 26, 20),(0, 14, 6, 26, 27),(0, 14, 6, 27, 7),(0, 14, 6, 27, 13),(0, 14, 6, 27, 20),(0, 14, 6, 27, 21),(0, 14, 7, 1, 8),(0, 14, 7, 1, 15),(0, 14, 7, 1, 21),(0, 14, 7, 1, 22),(0, 14, 7, 6, 13),(0, 14, 7, 6, 20),(0, 14, 7, 6, 21),(0, 14, 7, 6, 27),(0, 14, 7, 8, 1),(0, 14, 7, 8, 15),(0, 14, 7, 8, 21),(0, 14, 7, 8, 22),(0, 14, 7, 13, 6),(0, 14, 7, 13, 20),(0, 14, 7, 13, 21),(0, 14, 7, 13, 27),(0, 14, 7, 15, 1),(0, 14, 7, 15, 8),(0, 14, 7, 15, 21),(0, 14, 7, 15, 22),(0, 14, 7, 20, 6),(0, 14, 7, 20, 13),(0, 14, 7, 20, 21),(0, 14, 7, 20, 27),(0, 14, 7, 21, 1),(0, 14, 7, 21, 6),(0, 14, 7, 21, 8),(0, 14, 7, 21, 13),(0, 14, 7, 21, 15),(0, 14, 7, 21, 20),(0, 14, 7, 21, 22),(0, 14, 7, 21, 27),(0, 14, 7, 22, 1),(0, 14, 7, 22, 8),(0, 14, 7, 22, 15),(0, 14, 7, 22, 21),(0, 14, 7, 27, 6),(0, 14, 7, 27, 13),(0, 14, 7, 27, 20),(0, 14, 7, 27, 21),(0, 14, 8, 1, 7),(0, 14, 8, 1, 15),(0, 14, 8, 1, 21),(0, 14, 8, 1, 22),(0, 14, 8, 2, 1),(0, 14, 8, 2, 15),(0, 14, 8, 2, 22),(0, 14, 8, 7, 1),(0, 14, 8, 7, 6),(0, 14, 8, 7, 13),(0, 14, 8, 7, 15),(0, 14, 8, 7, 20),(0, 14, 8, 7, 21),(0, 14, 8, 7, 22),(0, 14, 8, 7, 27),(0, 14, 8, 9, 1),(0, 14, 8, 9, 15),(0, 14, 8, 9, 22),(0, 14, 8, 15, 1),(0, 14, 8, 15, 7),(0, 14, 8, 15, 21),(0, 14, 8, 15, 22),(0, 14, 8, 16, 1),(0, 14, 8, 16, 15),(0, 14, 8, 16, 22),(0, 14, 8, 21, 1),(0, 14, 8, 21, 6),(0, 14, 8, 21, 7),(0, 14, 8, 21, 13),(0, 14, 8, 21, 15),(0, 14, 8, 21, 20),(0, 14, 8, 21, 22),(0, 14, 8, 21, 27),(0, 14, 8, 22, 1),(0, 14, 8, 22, 7),(0, 14, 8, 22, 15),(0, 14, 8, 22, 21),(0, 14, 8, 23, 1),(0, 14, 8, 23, 15),(0, 14, 8, 23, 22),(0, 14, 13, 5, 6),(0, 14, 13, 5, 20),(0, 14, 13, 5, 27),(0, 14, 13, 6, 7),(0, 14, 13, 6, 20),(0, 14, 13, 6, 21),(0, 14, 13, 6, 27),(0, 14, 13, 7, 1),(0, 14, 13, 7, 6),(0, 14, 13, 7, 8),(0, 14, 13, 7, 15),(0, 14, 13, 7, 20),(0, 14, 13, 7, 21),(0, 14, 13, 7, 22),(0, 14, 13, 7, 27),(0, 14, 13, 12, 6),(0, 14, 13, 12, 20),(0, 14, 13, 12, 27),(0, 14, 13, 19, 6),(0, 14, 13, 19, 20),(0, 14, 13, 19, 27),(0, 14, 13, 20, 6),(0, 14, 13, 20, 7),(0, 14, 13, 20, 21),(0, 14, 13, 20, 27),(0, 14, 13, 21, 1),(0, 14, 13, 21, 6),(0, 14, 13, 21, 7),(0, 14, 13, 21, 8),(0, 14, 13, 21, 15),(0, 14, 13, 21, 20),(0, 14, 13, 21, 22),(0, 14, 13, 21, 27),(0, 14, 13, 26, 6),(0, 14, 13, 26, 20),(0, 14, 13, 26, 27),(0, 14, 13, 27, 6),(0, 14, 13, 27, 7),(0, 14, 13, 27, 20),(0, 14, 13, 27, 21),(0, 14, 15, 1, 7),(0, 14, 15, 1, 8),(0, 14, 15, 1, 21),(0, 14, 15, 1, 22),(0, 14, 15, 2, 1),(0, 14, 15, 2, 8),(0, 14, 15, 2, 22),(0, 14, 15, 7, 1),(0, 14, 15, 7, 6),(0, 14, 15, 7, 8),(0, 14, 15, 7, 13),(0, 14, 15, 7, 20),(0, 14, 15, 7, 21),(0, 14, 15, 7, 22),(0, 14, 15, 7, 27),(0, 14, 15, 8, 1),(0, 14, 15, 8, 7),(0, 14, 15, 8, 21),(0, 14, 15, 8, 22),(0, 14, 15, 9, 1),(0, 14, 15, 9, 8),(0, 14, 15, 9, 22),(0, 14, 15, 16, 1),(0, 14, 15, 16, 8),(0, 14, 15, 16, 22),(0, 14, 15, 21, 1),(0, 14, 15, 21, 6),(0, 14, 15, 21, 7),(0, 14, 15, 21, 8),(0, 14, 15, 21, 13),(0, 14, 15, 21, 20),(0, 14, 15, 21, 22),(0, 14, 15, 21, 27),(0, 14, 15, 22, 1),(0, 14, 15, 22, 7),(0, 14, 15, 22, 8),(0, 14, 15, 22, 21),(0, 14, 15, 23, 1),(0, 14, 15, 23, 8),(0, 14, 15, 23, 22),(0, 14, 20, 5, 6),(0, 14, 20, 5, 13),(0, 14, 20, 5, 27),(0, 14, 20, 6, 7),(0, 14, 20, 6, 13),(0, 14, 20, 6, 21),(0, 14, 20, 6, 27),(0, 14, 20, 7, 1),(0, 14, 20, 7, 6),(0, 14, 20, 7, 8),(0, 14, 20, 7, 13),(0, 14, 20, 7, 15),(0, 14, 20, 7, 21),(0, 14, 20, 7, 22),(0, 14, 20, 7, 27),(0, 14, 20, 12, 6),(0, 14, 20, 12, 13),(0, 14, 20, 12, 27),(0, 14, 20, 13, 6),(0, 14, 20, 13, 7),(0, 14, 20, 13, 21),(0, 14, 20, 13, 27),(0, 14, 20, 19, 6),(0, 14, 20, 19, 13),(0, 14, 20, 19, 27),(0, 14, 20, 21, 1),(0, 14, 20, 21, 6),(0, 14, 20, 21, 7),(0, 14, 20, 21, 8),(0, 14, 20, 21, 13),(0, 14, 20, 21, 15),(0, 14, 20, 21, 22),(0, 14, 20, 21, 27),(0, 14, 20, 26, 6),(0, 14, 20, 26, 13),(0, 14, 20, 26, 27),(0, 14, 20, 27, 6),(0, 14, 20, 27, 7),(0, 14, 20, 27, 13),(0, 14, 20, 27, 21),(0, 14, 21, 1, 7),(0, 14, 21, 1, 8),(0, 14, 21, 1, 15),(0, 14, 21, 1, 22),(0, 14, 21, 6, 7),(0, 14, 21, 6, 13),(0, 14, 21, 6, 20),(0, 14, 21, 6, 27),(0, 14, 21, 7, 1),(0, 14, 21, 7, 6),(0, 14, 21, 7, 8),(0, 14, 21, 7, 13),(0, 14, 21, 7, 15),(0, 14, 21, 7, 20),(0, 14, 21, 7, 22),(0, 14, 21, 7, 27),(0, 14, 21, 8, 1),(0, 14, 21, 8, 7),(0, 14, 21, 8, 15),(0, 14, 21, 8, 22),(0, 14, 21, 13, 6),(0, 14, 21, 13, 7),(0, 14, 21, 13, 20),(0, 14, 21, 13, 27),(0, 14, 21, 15, 1),(0, 14, 21, 15, 7),(0, 14, 21, 15, 8),(0, 14, 21, 15, 22),(0, 14, 21, 20, 6),(0, 14, 21, 20, 7),(0, 14, 21, 20, 13),(0, 14, 21, 20, 27),(0, 14, 21, 22, 1),(0, 14, 21, 22, 7),(0, 14, 21, 22, 8),(0, 14, 21, 22, 15),(0, 14, 21, 27, 6),(0, 14, 21, 27, 7),(0, 14, 21, 27, 13),(0, 14, 21, 27, 20),(0, 14, 22, 1, 7),(0, 14, 22, 1, 8),(0, 14, 22, 1, 15),(0, 14, 22, 1, 21),(0, 14, 22, 2, 1),(0, 14, 22, 2, 8),(0, 14, 22, 2, 15),(0, 14, 22, 7, 1),(0, 14, 22, 7, 6),(0, 14, 22, 7, 8),(0, 14, 22, 7, 13),(0, 14, 22, 7, 15),(0, 14, 22, 7, 20),(0, 14, 22, 7, 21),(0, 14, 22, 7, 27),(0, 14, 22, 8, 1),(0, 14, 22, 8, 7),(0, 14, 22, 8, 15),(0, 14, 22, 8, 21),(0, 14, 22, 9, 1),(0, 14, 22, 9, 8),(0, 14, 22, 9, 15),(0, 14, 22, 15, 1),(0, 14, 22, 15, 7),(0, 14, 22, 15, 8),(0, 14, 22, 15, 21),(0, 14, 22, 16, 1),(0, 14, 22, 16, 8),(0, 14, 22, 16, 15),(0, 14, 22, 21, 1),(0, 14, 22, 21, 6),(0, 14, 22, 21, 7),(0, 14, 22, 21, 8),(0, 14, 22, 21, 13),(0, 14, 22, 21, 15),(0, 14, 22, 21, 20),(0, 14, 22, 21, 27),(0, 14, 22, 23, 1),(0, 14, 22, 23, 8),(0, 14, 22, 23, 15),(0, 14, 27, 5, 6),(0, 14, 27, 5, 13),(0, 14, 27, 5, 20),(0, 14, 27, 6, 7),(0, 14, 27, 6, 13),(0, 14, 27, 6, 20),(0, 14, 27, 6, 21),(0, 14, 27, 7, 1),(0, 14, 27, 7, 6),(0, 14, 27, 7, 8),(0, 14, 27, 7, 13),(0, 14, 27, 7, 15),(0, 14, 27, 7, 20),(0, 14, 27, 7, 21),(0, 14, 27, 7, 22),(0, 14, 27, 12, 6),(0, 14, 27, 12, 13),(0, 14, 27, 12, 20),(0, 14, 27, 13, 6),(0, 14, 27, 13, 7),(0, 14, 27, 13, 20),(0, 14, 27, 13, 21),(0, 14, 27, 19, 6),(0, 14, 27, 19, 13),(0, 14, 27, 19, 20),(0, 14, 27, 20, 6),(0, 14, 27, 20, 7),(0, 14, 27, 20, 13),(0, 14, 27, 20, 21),(0, 14, 27, 21, 1),(0, 14, 27, 21, 6),(0, 14, 27, 21, 7),(0, 14, 27, 21, 8),(0, 14, 27, 21, 13),(0, 14, 27, 21, 15),(0, 14, 27, 21, 20),(0, 14, 27, 21, 22),(0, 14, 27, 26, 6),(0, 14, 27, 26, 13),(0, 14, 27, 26, 20),(0, 15, 1, 2, 8),(0, 15, 1, 2, 22),(0, 15, 1, 7, 6),(0, 15, 1, 7, 8),(0, 15, 1, 7, 13),(0, 15, 1, 7, 14),(0, 15, 1, 7, 20),(0, 15, 1, 7, 21),(0, 15, 1, 7, 22),(0, 15, 1, 7, 27),(0, 15, 1, 8, 7),(0, 15, 1, 8, 14),(0, 15, 1, 8, 21),(0, 15, 1, 8, 22),(0, 15, 1, 9, 8),(0, 15, 1, 9, 22),(0, 15, 1, 14, 6),(0, 15, 1, 14, 7),(0, 15, 1, 14, 8),(0, 15, 1, 14, 13),(0, 15, 1, 14, 20),(0, 15, 1, 14, 21),(0, 15, 1, 14, 22),(0, 15, 1, 14, 27),(0, 15, 1, 16, 8),(0, 15, 1, 16, 22),(0, 15, 1, 21, 6),(0, 15, 1, 21, 7),(0, 15, 1, 21, 8),(0, 15, 1, 21, 13),(0, 15, 1, 21, 14),(0, 15, 1, 21, 20),(0, 15, 1, 21, 22),(0, 15, 1, 21, 27),(0, 15, 1, 22, 7),(0, 15, 1, 22, 8),(0, 15, 1, 22, 14),(0, 15, 1, 22, 21),(0, 15, 1, 23, 8),(0, 15, 1, 23, 22),(0, 15, 2, 1, 7),(0, 15, 2, 1, 8),(0, 15, 2, 1, 14),(0, 15, 2, 1, 21),(0, 15, 2, 1, 22),(0, 15, 2, 8, 1),(0, 15, 2, 8, 7),(0, 15, 2, 8, 14),(0, 15, 2, 8, 21),(0, 15, 2, 8, 22),(0, 15, 2, 9, 1),(0, 15, 2, 9, 8),(0, 15, 2, 9, 22),(0, 15, 2, 16, 1),(0, 15, 2, 16, 8),(0, 15, 2, 16, 22),(0, 15, 2, 22, 1),(0, 15, 2, 22, 7),(0, 15, 2, 22, 8),(0, 15, 2, 22, 14),(0, 15, 2, 22, 21),(0, 15, 2, 23, 1),(0, 15, 2, 23, 8),(0, 15, 2, 23, 22),(0, 15, 7, 1, 8),(0, 15, 7, 1, 14),(0, 15, 7, 1, 21),(0, 15, 7, 1, 22),(0, 15, 7, 6, 13),(0, 15, 7, 6, 14),(0, 15, 7, 6, 20),(0, 15, 7, 6, 21),(0, 15, 7, 6, 27),(0, 15, 7, 8, 1),(0, 15, 7, 8, 14),(0, 15, 7, 8, 21),(0, 15, 7, 8, 22),(0, 15, 7, 13, 6),(0, 15, 7, 13, 14),(0, 15, 7, 13, 20),(0, 15, 7, 13, 21),(0, 15, 7, 13, 27),(0, 15, 7, 14, 1),(0, 15, 7, 14, 6),(0, 15, 7, 14, 8),(0, 15, 7, 14, 13),(0, 15, 7, 14, 20),(0, 15, 7, 14, 21),(0, 15, 7, 14, 22),(0, 15, 7, 14, 27),(0, 15, 7, 20, 6),(0, 15, 7, 20, 13),(0, 15, 7, 20, 14),(0, 15, 7, 20, 21),(0, 15, 7, 20, 27),(0, 15, 7, 21, 1),(0, 15, 7, 21, 6),(0, 15, 7, 21, 8),(0, 15, 7, 21, 13),(0, 15, 7, 21, 14),(0, 15, 7, 21, 20),(0, 15, 7, 21, 22),(0, 15, 7, 21, 27),(0, 15, 7, 22, 1),(0, 15, 7, 22, 8),(0, 15, 7, 22, 14),(0, 15, 7, 22, 21),(0, 15, 7, 27, 6),(0, 15, 7, 27, 13),(0, 15, 7, 27, 14),(0, 15, 7, 27, 20),(0, 15, 7, 27, 21),(0, 15, 8, 1, 7),(0, 15, 8, 1, 14),(0, 15, 8, 1, 21),(0, 15, 8, 1, 22),(0, 15, 8, 2, 1),(0, 15, 8, 2, 22),(0, 15, 8, 7, 1),(0, 15, 8, 7, 6),(0, 15, 8, 7, 13),(0, 15, 8, 7, 14),(0, 15, 8, 7, 20),(0, 15, 8, 7, 21),(0, 15, 8, 7, 22),(0, 15, 8, 7, 27),(0, 15, 8, 9, 1),(0, 15, 8, 9, 22),(0, 15, 8, 14, 1),(0, 15, 8, 14, 6),(0, 15, 8, 14, 7),(0, 15, 8, 14, 13),(0, 15, 8, 14, 20),(0, 15, 8, 14, 21),(0, 15, 8, 14, 22),(0, 15, 8, 14, 27),(0, 15, 8, 16, 1),(0, 15, 8, 16, 22),(0, 15, 8, 21, 1),(0, 15, 8, 21, 6),(0, 15, 8, 21, 7),(0, 15, 8, 21, 13),(0, 15, 8, 21, 14),(0, 15, 8, 21, 20),(0, 15, 8, 21, 22),(0, 15, 8, 21, 27),(0, 15, 8, 22, 1),(0, 15, 8, 22, 7),(0, 15, 8, 22, 14),(0, 15, 8, 22, 21),(0, 15, 8, 23, 1),(0, 15, 8, 23, 22),(0, 15, 9, 1, 7),(0, 15, 9, 1, 8),(0, 15, 9, 1, 14),(0, 15, 9, 1, 21),(0, 15, 9, 1, 22),(0, 15, 9, 2, 1),(0, 15, 9, 2, 8),(0, 15, 9, 2, 22),(0, 15, 9, 8, 1),(0, 15, 9, 8, 7),(0, 15, 9, 8, 14),(0, 15, 9, 8, 21),(0, 15, 9, 8, 22),(0, 15, 9, 16, 1),(0, 15, 9, 16, 8),(0, 15, 9, 16, 22),(0, 15, 9, 22, 1),(0, 15, 9, 22, 7),(0, 15, 9, 22, 8),(0, 15, 9, 22, 14),(0, 15, 9, 22, 21),(0, 15, 9, 23, 1),(0, 15, 9, 23, 8),(0, 15, 9, 23, 22),(0, 15, 14, 1, 7),(0, 15, 14, 1, 8),(0, 15, 14, 1, 21),(0, 15, 14, 1, 22),(0, 15, 14, 6, 7),(0, 15, 14, 6, 13),(0, 15, 14, 6, 20),(0, 15, 14, 6, 21),(0, 15, 14, 6, 27),(0, 15, 14, 7, 1),(0, 15, 14, 7, 6),(0, 15, 14, 7, 8),(0, 15, 14, 7, 13),(0, 15, 14, 7, 20),(0, 15, 14, 7, 21),(0, 15, 14, 7, 22),(0, 15, 14, 7, 27),(0, 15, 14, 8, 1),(0, 15, 14, 8, 7),(0, 15, 14, 8, 21),(0, 15, 14, 8, 22),(0, 15, 14, 13, 6),(0, 15, 14, 13, 7),(0, 15, 14, 13, 20),(0, 15, 14, 13, 21),(0, 15, 14, 13, 27),(0, 15, 14, 20, 6),(0, 15, 14, 20, 7),(0, 15, 14, 20, 13),(0, 15, 14, 20, 21),(0, 15, 14, 20, 27),(0, 15, 14, 21, 1),(0, 15, 14, 21, 6),(0, 15, 14, 21, 7),(0, 15, 14, 21, 8),(0, 15, 14, 21, 13),(0, 15, 14, 21, 20),(0, 15, 14, 21, 22),(0, 15, 14, 21, 27),(0, 15, 14, 22, 1),(0, 15, 14, 22, 7),(0, 15, 14, 22, 8),(0, 15, 14, 22, 21),(0, 15, 14, 27, 6),(0, 15, 14, 27, 7),(0, 15, 14, 27, 13),(0, 15, 14, 27, 20),(0, 15, 14, 27, 21),(0, 15, 16, 1, 7),(0, 15, 16, 1, 8),(0, 15, 16, 1, 14),(0, 15, 16, 1, 21),(0, 15, 16, 1, 22),(0, 15, 16, 2, 1),(0, 15, 16, 2, 8),(0, 15, 16, 2, 22),(0, 15, 16, 8, 1),(0, 15, 16, 8, 7),(0, 15, 16, 8, 14),(0, 15, 16, 8, 21),(0, 15, 16, 8, 22),(0, 15, 16, 9, 1),(0, 15, 16, 9, 8),(0, 15, 16, 9, 22),(0, 15, 16, 22, 1),(0, 15, 16, 22, 7),(0, 15, 16, 22, 8),(0, 15, 16, 22, 14),(0, 15, 16, 22, 21),(0, 15, 16, 23, 1),(0, 15, 16, 23, 8),(0, 15, 16, 23, 22),(0, 15, 21, 1, 7),(0, 15, 21, 1, 8),(0, 15, 21, 1, 14),(0, 15, 21, 1, 22),(0, 15, 21, 6, 7),(0, 15, 21, 6, 13),(0, 15, 21, 6, 14),(0, 15, 21, 6, 20),(0, 15, 21, 6, 27),(0, 15, 21, 7, 1),(0, 15, 21, 7, 6),(0, 15, 21, 7, 8),(0, 15, 21, 7, 13),(0, 15, 21, 7, 14),(0, 15, 21, 7, 20),(0, 15, 21, 7, 22),(0, 15, 21, 7, 27),(0, 15, 21, 8, 1),(0, 15, 21, 8, 7),(0, 15, 21, 8, 14),(0, 15, 21, 8, 22),(0, 15, 21, 13, 6),(0, 15, 21, 13, 7),(0, 15, 21, 13, 14),(0, 15, 21, 13, 20),(0, 15, 21, 13, 27),(0, 15, 21, 14, 1),(0, 15, 21, 14, 6),(0, 15, 21, 14, 7),(0, 15, 21, 14, 8),(0, 15, 21, 14, 13),(0, 15, 21, 14, 20),(0, 15, 21, 14, 22),(0, 15, 21, 14, 27),(0, 15, 21, 20, 6),(0, 15, 21, 20, 7),(0, 15, 21, 20, 13),(0, 15, 21, 20, 14),(0, 15, 21, 20, 27),(0, 15, 21, 22, 1),(0, 15, 21, 22, 7),(0, 15, 21, 22, 8),(0, 15, 21, 22, 14),(0, 15, 21, 27, 6),(0, 15, 21, 27, 7),(0, 15, 21, 27, 13),(0, 15, 21, 27, 14),(0, 15, 21, 27, 20),(0, 15, 22, 1, 7),(0, 15, 22, 1, 8),(0, 15, 22, 1, 14),(0, 15, 22, 1, 21),(0, 15, 22, 2, 1),(0, 15, 22, 2, 8),(0, 15, 22, 7, 1),(0, 15, 22, 7, 6),(0, 15, 22, 7, 8),(0, 15, 22, 7, 13),(0, 15, 22, 7, 14),(0, 15, 22, 7, 20),(0, 15, 22, 7, 21),(0, 15, 22, 7, 27),(0, 15, 22, 8, 1),(0, 15, 22, 8, 7),(0, 15, 22, 8, 14),(0, 15, 22, 8, 21),(0, 15, 22, 9, 1),(0, 15, 22, 9, 8),(0, 15, 22, 14, 1),(0, 15, 22, 14, 6),(0, 15, 22, 14, 7),(0, 15, 22, 14, 8),(0, 15, 22, 14, 13),(0, 15, 22, 14, 20),(0, 15, 22, 14, 21),(0, 15, 22, 14, 27),(0, 15, 22, 16, 1),(0, 15, 22, 16, 8),(0, 15, 22, 21, 1),(0, 15, 22, 21, 6),(0, 15, 22, 21, 7),(0, 15, 22, 21, 8),(0, 15, 22, 21, 13),(0, 15, 22, 21, 14),(0, 15, 22, 21, 20),(0, 15, 22, 21, 27),(0, 15, 22, 23, 1),(0, 15, 22, 23, 8),(0, 15, 23, 1, 7),(0, 15, 23, 1, 8),(0, 15, 23, 1, 14),(0, 15, 23, 1, 21),(0, 15, 23, 1, 22),(0, 15, 23, 2, 1),(0, 15, 23, 2, 8),(0, 15, 23, 2, 22),(0, 15, 23, 8, 1),(0, 15, 23, 8, 7),(0, 15, 23, 8, 14),(0, 15, 23, 8, 21),(0, 15, 23, 8, 22),(0, 15, 23, 9, 1),(0, 15, 23, 9, 8),(0, 15, 23, 9, 22),(0, 15, 23, 16, 1),(0, 15, 23, 16, 8),(0, 15, 23, 16, 22),(0, 15, 23, 22, 1),(0, 15, 23, 22, 7),(0, 15, 23, 22, 8),(0, 15, 23, 22, 14),(0, 15, 23, 22, 21),(0, 20, 5, 6, 7),(0, 20, 5, 6, 13),(0, 20, 5, 6, 14),(0, 20, 5, 6, 21),(0, 20, 5, 6, 27),(0, 20, 5, 12, 6),(0, 20, 5, 12, 13),(0, 20, 5, 12, 27),(0, 20, 5, 13, 6),(0, 20, 5, 13, 7),(0, 20, 5, 13, 14),(0, 20, 5, 13, 21),(0, 20, 5, 13, 27),(0, 20, 5, 19, 6),(0, 20, 5, 19, 13),(0, 20, 5, 19, 27),(0, 20, 5, 26, 6),(0, 20, 5, 26, 13),(0, 20, 5, 26, 27),(0, 20, 5, 27, 6),(0, 20, 5, 27, 7),(0, 20, 5, 27, 13),(0, 20, 5, 27, 14),(0, 20, 5, 27, 21),(0, 20, 6, 5, 13),(0, 20, 6, 5, 27),(0, 20, 6, 7, 1),(0, 20, 6, 7, 8),(0, 20, 6, 7, 13),(0, 20, 6, 7, 14),(0, 20, 6, 7, 15),(0, 20, 6, 7, 21),(0, 20, 6, 7, 22),(0, 20, 6, 7, 27),(0, 20, 6, 12, 13),(0, 20, 6, 12, 27),(0, 20, 6, 13, 7),(0, 20, 6, 13, 14),(0, 20, 6, 13, 21),(0, 20, 6, 13, 27),(0, 20, 6, 14, 1),(0, 20, 6, 14, 7),(0, 20, 6, 14, 8),(0, 20, 6, 14, 13),(0, 20, 6, 14, 15),(0, 20, 6, 14, 21),(0, 20, 6, 14, 22),(0, 20, 6, 14, 27),(0, 20, 6, 19, 13),(0, 20, 6, 19, 27),(0, 20, 6, 21, 1),(0, 20, 6, 21, 7),(0, 20, 6, 21, 8),(0, 20, 6, 21, 13),(0, 20, 6, 21, 14),(0, 20, 6, 21, 15),(0, 20, 6, 21, 22),(0, 20, 6, 21, 27),(0, 20, 6, 26, 13),(0, 20, 6, 26, 27),(0, 20, 6, 27, 7),(0, 20, 6, 27, 13),(0, 20, 6, 27, 14),(0, 20, 6, 27, 21),(0, 20, 7, 1, 8),(0, 20, 7, 1, 14),(0, 20, 7, 1, 15),(0, 20, 7, 1, 21),(0, 20, 7, 1, 22),(0, 20, 7, 6, 13),(0, 20, 7, 6, 14),(0, 20, 7, 6, 21),(0, 20, 7, 6, 27),(0, 20, 7, 8, 1),(0, 20, 7, 8, 14),(0, 20, 7, 8, 15),(0, 20, 7, 8, 21),(0, 20, 7, 8, 22),(0, 20, 7, 13, 6),(0, 20, 7, 13, 14),(0, 20, 7, 13, 21),(0, 20, 7, 13, 27),(0, 20, 7, 14, 1),(0, 20, 7, 14, 6),(0, 20, 7, 14, 8),(0, 20, 7, 14, 13),(0, 20, 7, 14, 15),(0, 20, 7, 14, 21),(0, 20, 7, 14, 22),(0, 20, 7, 14, 27),(0, 20, 7, 15, 1),(0, 20, 7, 15, 8),(0, 20, 7, 15, 14),(0, 20, 7, 15, 21),(0, 20, 7, 15, 22),(0, 20, 7, 21, 1),(0, 20, 7, 21, 6),(0, 20, 7, 21, 8),(0, 20, 7, 21, 13),(0, 20, 7, 21, 14),(0, 20, 7, 21, 15),(0, 20, 7, 21, 22),(0, 20, 7, 21, 27),(0, 20, 7, 22, 1),(0, 20, 7, 22, 8),(0, 20, 7, 22, 14),(0, 20, 7, 22, 15),(0, 20, 7, 22, 21),(0, 20, 7, 27, 6),(0, 20, 7, 27, 13),(0, 20, 7, 27, 14),(0, 20, 7, 27, 21),(0, 20, 12, 5, 6),(0, 20, 12, 5, 13),(0, 20, 12, 5, 27),(0, 20, 12, 6, 7),(0, 20, 12, 6, 13),(0, 20, 12, 6, 14),(0, 20, 12, 6, 21),(0, 20, 12, 6, 27),(0, 20, 12, 13, 6),(0, 20, 12, 13, 7),(0, 20, 12, 13, 14),(0, 20, 12, 13, 21),(0, 20, 12, 13, 27),(0, 20, 12, 19, 6),(0, 20, 12, 19, 13),(0, 20, 12, 19, 27),(0, 20, 12, 26, 6),(0, 20, 12, 26, 13),(0, 20, 12, 26, 27),(0, 20, 12, 27, 6),(0, 20, 12, 27, 7),(0, 20, 12, 27, 13),(0, 20, 12, 27, 14),(0, 20, 12, 27, 21),(0, 20, 13, 5, 6),(0, 20, 13, 5, 27),(0, 20, 13, 6, 7),(0, 20, 13, 6, 14),(0, 20, 13, 6, 21),(0, 20, 13, 6, 27),(0, 20, 13, 7, 1),(0, 20, 13, 7, 6),(0, 20, 13, 7, 8),(0, 20, 13, 7, 14),(0, 20, 13, 7, 15),(0, 20, 13, 7, 21),(0, 20, 13, 7, 22),(0, 20, 13, 7, 27),(0, 20, 13, 12, 6),(0, 20, 13, 12, 27),(0, 20, 13, 14, 1),(0, 20, 13, 14, 6),(0, 20, 13, 14, 7),(0, 20, 13, 14, 8),(0, 20, 13, 14, 15),(0, 20, 13, 14, 21),(0, 20, 13, 14, 22),(0, 20, 13, 14, 27),(0, 20, 13, 19, 6),(0, 20, 13, 19, 27),(0, 20, 13, 21, 1),(0, 20, 13, 21, 6),(0, 20, 13, 21, 7),(0, 20, 13, 21, 8),(0, 20, 13, 21, 14),(0, 20, 13, 21, 15),(0, 20, 13, 21, 22),(0, 20, 13, 21, 27),(0, 20, 13, 26, 6),(0, 20, 13, 26, 27),(0, 20, 13, 27, 6),(0, 20, 13, 27, 7),(0, 20, 13, 27, 14),(0, 20, 13, 27, 21),(0, 20, 14, 1, 7),(0, 20, 14, 1, 8),(0, 20, 14, 1, 15),(0, 20, 14, 1, 21),(0, 20, 14, 1, 22),(0, 20, 14, 6, 7),(0, 20, 14, 6, 13),(0, 20, 14, 6, 21),(0, 20, 14, 6, 27),(0, 20, 14, 7, 1),(0, 20, 14, 7, 6),(0, 20, 14, 7, 8),(0, 20, 14, 7, 13),(0, 20, 14, 7, 15),(0, 20, 14, 7, 21),(0, 20, 14, 7, 22),(0, 20, 14, 7, 27),(0, 20, 14, 8, 1),(0, 20, 14, 8, 7),(0, 20, 14, 8, 15),(0, 20, 14, 8, 21),(0, 20, 14, 8, 22),(0, 20, 14, 13, 6),(0, 20, 14, 13, 7),(0, 20, 14, 13, 21),(0, 20, 14, 13, 27),(0, 20, 14, 15, 1),(0, 20, 14, 15, 7),(0, 20, 14, 15, 8),(0, 20, 14, 15, 21),(0, 20, 14, 15, 22),(0, 20, 14, 21, 1),(0, 20, 14, 21, 6),(0, 20, 14, 21, 7),(0, 20, 14, 21, 8),(0, 20, 14, 21, 13),(0, 20, 14, 21, 15),(0, 20, 14, 21, 22),(0, 20, 14, 21, 27),(0, 20, 14, 22, 1),(0, 20, 14, 22, 7),(0, 20, 14, 22, 8),(0, 20, 14, 22, 15),(0, 20, 14, 22, 21),(0, 20, 14, 27, 6),(0, 20, 14, 27, 7),(0, 20, 14, 27, 13),(0, 20, 14, 27, 21),(0, 20, 19, 5, 6),(0, 20, 19, 5, 13),(0, 20, 19, 5, 27),(0, 20, 19, 6, 7),(0, 20, 19, 6, 13),(0, 20, 19, 6, 14),(0, 20, 19, 6, 21),(0, 20, 19, 6, 27),(0, 20, 19, 12, 6),(0, 20, 19, 12, 13),(0, 20, 19, 12, 27),(0, 20, 19, 13, 6),(0, 20, 19, 13, 7),(0, 20, 19, 13, 14),(0, 20, 19, 13, 21),(0, 20, 19, 13, 27),(0, 20, 19, 26, 6),(0, 20, 19, 26, 13),(0, 20, 19, 26, 27),(0, 20, 19, 27, 6),(0, 20, 19, 27, 7),(0, 20, 19, 27, 13),(0, 20, 19, 27, 14),(0, 20, 19, 27, 21),(0, 20, 21, 1, 7),(0, 20, 21, 1, 8),(0, 20, 21, 1, 14),(0, 20, 21, 1, 15),(0, 20, 21, 1, 22),(0, 20, 21, 6, 7),(0, 20, 21, 6, 13),(0, 20, 21, 6, 14),(0, 20, 21, 6, 27),(0, 20, 21, 7, 1),(0, 20, 21, 7, 6),(0, 20, 21, 7, 8),(0, 20, 21, 7, 13),(0, 20, 21, 7, 14),(0, 20, 21, 7, 15),(0, 20, 21, 7, 22),(0, 20, 21, 7, 27),(0, 20, 21, 8, 1),(0, 20, 21, 8, 7),(0, 20, 21, 8, 14),(0, 20, 21, 8, 15),(0, 20, 21, 8, 22),(0, 20, 21, 13, 6),(0, 20, 21, 13, 7),(0, 20, 21, 13, 14),(0, 20, 21, 13, 27),(0, 20, 21, 14, 1),(0, 20, 21, 14, 6),(0, 20, 21, 14, 7),(0, 20, 21, 14, 8),(0, 20, 21, 14, 13),(0, 20, 21, 14, 15),(0, 20, 21, 14, 22),(0, 20, 21, 14, 27),(0, 20, 21, 15, 1),(0, 20, 21, 15, 7),(0, 20, 21, 15, 8),(0, 20, 21, 15, 14),(0, 20, 21, 15, 22),(0, 20, 21, 22, 1),(0, 20, 21, 22, 7),(0, 20, 21, 22, 8),(0, 20, 21, 22, 14),(0, 20, 21, 22, 15),(0, 20, 21, 27, 6),(0, 20, 21, 27, 7),(0, 20, 21, 27, 13),(0, 20, 21, 27, 14),(0, 20, 26, 5, 6),(0, 20, 26, 5, 13),(0, 20, 26, 5, 27),(0, 20, 26, 6, 7),(0, 20, 26, 6, 13),(0, 20, 26, 6, 14),(0, 20, 26, 6, 21),(0, 20, 26, 6, 27),(0, 20, 26, 12, 6),(0, 20, 26, 12, 13),(0, 20, 26, 12, 27),(0, 20, 26, 13, 6),(0, 20, 26, 13, 7),(0, 20, 26, 13, 14),(0, 20, 26, 13, 21),(0, 20, 26, 13, 27),(0, 20, 26, 19, 6),(0, 20, 26, 19, 13),(0, 20, 26, 19, 27),(0, 20, 26, 27, 6),(0, 20, 26, 27, 7),(0, 20, 26, 27, 13),(0, 20, 26, 27, 14),(0, 20, 26, 27, 21),(0, 20, 27, 5, 6),(0, 20, 27, 5, 13),(0, 20, 27, 6, 7),(0, 20, 27, 6, 13),(0, 20, 27, 6, 14),(0, 20, 27, 6, 21),(0, 20, 27, 7, 1),(0, 20, 27, 7, 6),(0, 20, 27, 7, 8),(0, 20, 27, 7, 13),(0, 20, 27, 7, 14),(0, 20, 27, 7, 15),(0, 20, 27, 7, 21),(0, 20, 27, 7, 22),(0, 20, 27, 12, 6),(0, 20, 27, 12, 13),(0, 20, 27, 13, 6),(0, 20, 27, 13, 7),(0, 20, 27, 13, 14),(0, 20, 27, 13, 21),(0, 20, 27, 14, 1),(0, 20, 27, 14, 6),(0, 20, 27, 14, 7),(0, 20, 27, 14, 8),(0, 20, 27, 14, 13),(0, 20, 27, 14, 15),(0, 20, 27, 14, 21),(0, 20, 27, 14, 22),(0, 20, 27, 19, 6),(0, 20, 27, 19, 13),(0, 20, 27, 21, 1),(0, 20, 27, 21, 6),(0, 20, 27, 21, 7),(0, 20, 27, 21, 8),(0, 20, 27, 21, 13),(0, 20, 27, 21, 14),(0, 20, 27, 21, 15),(0, 20, 27, 21, 22),(0, 20, 27, 26, 6),(0, 20, 27, 26, 13),(0, 21, 1, 2, 8),(0, 21, 1, 2, 15),(0, 21, 1, 2, 22),(0, 21, 1, 7, 6),(0, 21, 1, 7, 8),(0, 21, 1, 7, 13),(0, 21, 1, 7, 14),(0, 21, 1, 7, 15),(0, 21, 1, 7, 20),(0, 21, 1, 7, 22),(0, 21, 1, 7, 27),(0, 21, 1, 8, 7),(0, 21, 1, 8, 14),(0, 21, 1, 8, 15),(0, 21, 1, 8, 22),(0, 21, 1, 9, 8),(0, 21, 1, 9, 15),(0, 21, 1, 9, 22),(0, 21, 1, 14, 6),(0, 21, 1, 14, 7),(0, 21, 1, 14, 8),(0, 21, 1, 14, 13),(0, 21, 1, 14, 15),(0, 21, 1, 14, 20),(0, 21, 1, 14, 22),(0, 21, 1, 14, 27),(0, 21, 1, 15, 7),(0, 21, 1, 15, 8),(0, 21, 1, 15, 14),(0, 21, 1, 15, 22),(0, 21, 1, 16, 8),(0, 21, 1, 16, 15),(0, 21, 1, 16, 22),(0, 21, 1, 22, 7),(0, 21, 1, 22, 8),(0, 21, 1, 22, 14),(0, 21, 1, 22, 15),(0, 21, 1, 23, 8),(0, 21, 1, 23, 15),(0, 21, 1, 23, 22),(0, 21, 6, 5, 13),(0, 21, 6, 5, 20),(0, 21, 6, 5, 27),(0, 21, 6, 7, 1),(0, 21, 6, 7, 8),(0, 21, 6, 7, 13),(0, 21, 6, 7, 14),(0, 21, 6, 7, 15),(0, 21, 6, 7, 20),(0, 21, 6, 7, 22),(0, 21, 6, 7, 27),(0, 21, 6, 12, 13),(0, 21, 6, 12, 20),(0, 21, 6, 12, 27),(0, 21, 6, 13, 7),(0, 21, 6, 13, 14),(0, 21, 6, 13, 20),(0, 21, 6, 13, 27),(0, 21, 6, 14, 1),(0, 21, 6, 14, 7),(0, 21, 6, 14, 8),(0, 21, 6, 14, 13),(0, 21, 6, 14, 15),(0, 21, 6, 14, 20),(0, 21, 6, 14, 22),(0, 21, 6, 14, 27),(0, 21, 6, 19, 13),(0, 21, 6, 19, 20),(0, 21, 6, 19, 27),(0, 21, 6, 20, 7),(0, 21, 6, 20, 13),(0, 21, 6, 20, 14),(0, 21, 6, 20, 27),(0, 21, 6, 26, 13),(0, 21, 6, 26, 20),(0, 21, 6, 26, 27),(0, 21, 6, 27, 7),(0, 21, 6, 27, 13),(0, 21, 6, 27, 14),(0, 21, 6, 27, 20),(0, 21, 7, 1, 8),(0, 21, 7, 1, 14),(0, 21, 7, 1, 15),(0, 21, 7, 1, 22),(0, 21, 7, 6, 13),(0, 21, 7, 6, 14),(0, 21, 7, 6, 20),(0, 21, 7, 6, 27),(0, 21, 7, 8, 1),(0, 21, 7, 8, 14),(0, 21, 7, 8, 15),(0, 21, 7, 8, 22),(0, 21, 7, 13, 6),(0, 21, 7, 13, 14),(0, 21, 7, 13, 20),(0, 21, 7, 13, 27),(0, 21, 7, 14, 1),(0, 21, 7, 14, 6),(0, 21, 7, 14, 8),(0, 21, 7, 14, 13),(0, 21, 7, 14, 15),(0, 21, 7, 14, 20),(0, 21, 7, 14, 22),(0, 21, 7, 14, 27),(0, 21, 7, 15, 1),(0, 21, 7, 15, 8),(0, 21, 7, 15, 14),(0, 21, 7, 15, 22),(0, 21, 7, 20, 6),(0, 21, 7, 20, 13),(0, 21, 7, 20, 14),(0, 21, 7, 20, 27),(0, 21, 7, 22, 1),(0, 21, 7, 22, 8),(0, 21, 7, 22, 14),(0, 21, 7, 22, 15),(0, 21, 7, 27, 6),(0, 21, 7, 27, 13),(0, 21, 7, 27, 14),(0, 21, 7, 27, 20),(0, 21, 8, 1, 7),(0, 21, 8, 1, 14),(0, 21, 8, 1, 15),(0, 21, 8, 1, 22),(0, 21, 8, 2, 1),(0, 21, 8, 2, 15),(0, 21, 8, 2, 22),(0, 21, 8, 7, 1),(0, 21, 8, 7, 6),(0, 21, 8, 7, 13),(0, 21, 8, 7, 14),(0, 21, 8, 7, 15),(0, 21, 8, 7, 20),(0, 21, 8, 7, 22),(0, 21, 8, 7, 27),(0, 21, 8, 9, 1),(0, 21, 8, 9, 15),(0, 21, 8, 9, 22),(0, 21, 8, 14, 1),(0, 21, 8, 14, 6),(0, 21, 8, 14, 7),(0, 21, 8, 14, 13),(0, 21, 8, 14, 15),(0, 21, 8, 14, 20),(0, 21, 8, 14, 22),(0, 21, 8, 14, 27),(0, 21, 8, 15, 1),(0, 21, 8, 15, 7),(0, 21, 8, 15, 14),(0, 21, 8, 15, 22),(0, 21, 8, 16, 1),(0, 21, 8, 16, 15),(0, 21, 8, 16, 22),(0, 21, 8, 22, 1),(0, 21, 8, 22, 7),(0, 21, 8, 22, 14),(0, 21, 8, 22, 15),(0, 21, 8, 23, 1),(0, 21, 8, 23, 15),(0, 21, 8, 23, 22),(0, 21, 13, 5, 6),(0, 21, 13, 5, 20),(0, 21, 13, 5, 27),(0, 21, 13, 6, 7),(0, 21, 13, 6, 14),(0, 21, 13, 6, 20),(0, 21, 13, 6, 27),(0, 21, 13, 7, 1),(0, 21, 13, 7, 6),(0, 21, 13, 7, 8),(0, 21, 13, 7, 14),(0, 21, 13, 7, 15),(0, 21, 13, 7, 20),(0, 21, 13, 7, 22),(0, 21, 13, 7, 27),(0, 21, 13, 12, 6),(0, 21, 13, 12, 20),(0, 21, 13, 12, 27),(0, 21, 13, 14, 1),(0, 21, 13, 14, 6),(0, 21, 13, 14, 7),(0, 21, 13, 14, 8),(0, 21, 13, 14, 15),(0, 21, 13, 14, 20),(0, 21, 13, 14, 22),(0, 21, 13, 14, 27),(0, 21, 13, 19, 6),(0, 21, 13, 19, 20),(0, 21, 13, 19, 27),(0, 21, 13, 20, 6),(0, 21, 13, 20, 7),(0, 21, 13, 20, 14),(0, 21, 13, 20, 27),(0, 21, 13, 26, 6),(0, 21, 13, 26, 20),(0, 21, 13, 26, 27),(0, 21, 13, 27, 6),(0, 21, 13, 27, 7),(0, 21, 13, 27, 14),(0, 21, 13, 27, 20),(0, 21, 14, 1, 7),(0, 21, 14, 1, 8),(0, 21, 14, 1, 15),(0, 21, 14, 1, 22),(0, 21, 14, 6, 7),(0, 21, 14, 6, 13),(0, 21, 14, 6, 20),(0, 21, 14, 6, 27),(0, 21, 14, 7, 1),(0, 21, 14, 7, 6),(0, 21, 14, 7, 8),(0, 21, 14, 7, 13),(0, 21, 14, 7, 15),(0, 21, 14, 7, 20),(0, 21, 14, 7, 22),(0, 21, 14, 7, 27),(0, 21, 14, 8, 1),(0, 21, 14, 8, 7),(0, 21, 14, 8, 15),(0, 21, 14, 8, 22),(0, 21, 14, 13, 6),(0, 21, 14, 13, 7),(0, 21, 14, 13, 20),(0, 21, 14, 13, 27),(0, 21, 14, 15, 1),(0, 21, 14, 15, 7),(0, 21, 14, 15, 8),(0, 21, 14, 15, 22),(0, 21, 14, 20, 6),(0, 21, 14, 20, 7),(0, 21, 14, 20, 13),(0, 21, 14, 20, 27),(0, 21, 14, 22, 1),(0, 21, 14, 22, 7),(0, 21, 14, 22, 8),(0, 21, 14, 22, 15),(0, 21, 14, 27, 6),(0, 21, 14, 27, 7),(0, 21, 14, 27, 13),(0, 21, 14, 27, 20),(0, 21, 15, 1, 7),(0, 21, 15, 1, 8),(0, 21, 15, 1, 14),(0, 21, 15, 1, 22),(0, 21, 15, 2, 1),(0, 21, 15, 2, 8),(0, 21, 15, 2, 22),(0, 21, 15, 7, 1),(0, 21, 15, 7, 6),(0, 21, 15, 7, 8),(0, 21, 15, 7, 13),(0, 21, 15, 7, 14),(0, 21, 15, 7, 20),(0, 21, 15, 7, 22),(0, 21, 15, 7, 27),(0, 21, 15, 8, 1),(0, 21, 15, 8, 7),(0, 21, 15, 8, 14),(0, 21, 15, 8, 22),(0, 21, 15, 9, 1),(0, 21, 15, 9, 8),(0, 21, 15, 9, 22),(0, 21, 15, 14, 1),(0, 21, 15, 14, 6),(0, 21, 15, 14, 7),(0, 21, 15, 14, 8),(0, 21, 15, 14, 13),(0, 21, 15, 14, 20),(0, 21, 15, 14, 22),(0, 21, 15, 14, 27),(0, 21, 15, 16, 1),(0, 21, 15, 16, 8),(0, 21, 15, 16, 22),(0, 21, 15, 22, 1),(0, 21, 15, 22, 7),(0, 21, 15, 22, 8),(0, 21, 15, 22, 14),(0, 21, 15, 23, 1),(0, 21, 15, 23, 8),(0, 21, 15, 23, 22),(0, 21, 20, 5, 6),(0, 21, 20, 5, 13),(0, 21, 20, 5, 27),(0, 21, 20, 6, 7),(0, 21, 20, 6, 13),(0, 21, 20, 6, 14),(0, 21, 20, 6, 27),(0, 21, 20, 7, 1),(0, 21, 20, 7, 6),(0, 21, 20, 7, 8),(0, 21, 20, 7, 13),(0, 21, 20, 7, 14),(0, 21, 20, 7, 15),(0, 21, 20, 7, 22),(0, 21, 20, 7, 27),(0, 21, 20, 12, 6),(0, 21, 20, 12, 13),(0, 21, 20, 12, 27),(0, 21, 20, 13, 6),(0, 21, 20, 13, 7),(0, 21, 20, 13, 14),(0, 21, 20, 13, 27),(0, 21, 20, 14, 1),(0, 21, 20, 14, 6),(0, 21, 20, 14, 7),(0, 21, 20, 14, 8),(0, 21, 20, 14, 13),(0, 21, 20, 14, 15),(0, 21, 20, 14, 22),(0, 21, 20, 14, 27),(0, 21, 20, 19, 6),(0, 21, 20, 19, 13),(0, 21, 20, 19, 27),(0, 21, 20, 26, 6),(0, 21, 20, 26, 13),(0, 21, 20, 26, 27),(0, 21, 20, 27, 6),(0, 21, 20, 27, 7),(0, 21, 20, 27, 13),(0, 21, 20, 27, 14),(0, 21, 22, 1, 7),(0, 21, 22, 1, 8),(0, 21, 22, 1, 14),(0, 21, 22, 1, 15),(0, 21, 22, 2, 1),(0, 21, 22, 2, 8),(0, 21, 22, 2, 15),(0, 21, 22, 7, 1),(0, 21, 22, 7, 6),(0, 21, 22, 7, 8),(0, 21, 22, 7, 13),(0, 21, 22, 7, 14),(0, 21, 22, 7, 15),(0, 21, 22, 7, 20),(0, 21, 22, 7, 27),(0, 21, 22, 8, 1),(0, 21, 22, 8, 7),(0, 21, 22, 8, 14),(0, 21, 22, 8, 15),(0, 21, 22, 9, 1),(0, 21, 22, 9, 8),(0, 21, 22, 9, 15),(0, 21, 22, 14, 1),(0, 21, 22, 14, 6),(0, 21, 22, 14, 7),(0, 21, 22, 14, 8),(0, 21, 22, 14, 13),(0, 21, 22, 14, 15),(0, 21, 22, 14, 20),(0, 21, 22, 14, 27),(0, 21, 22, 15, 1),(0, 21, 22, 15, 7),(0, 21, 22, 15, 8),(0, 21, 22, 15, 14),(0, 21, 22, 16, 1),(0, 21, 22, 16, 8),(0, 21, 22, 16, 15),(0, 21, 22, 23, 1),(0, 21, 22, 23, 8),(0, 21, 22, 23, 15),(0, 21, 27, 5, 6),(0, 21, 27, 5, 13),(0, 21, 27, 5, 20),(0, 21, 27, 6, 7),(0, 21, 27, 6, 13),(0, 21, 27, 6, 14),(0, 21, 27, 6, 20),(0, 21, 27, 7, 1),(0, 21, 27, 7, 6),(0, 21, 27, 7, 8),(0, 21, 27, 7, 13),(0, 21, 27, 7, 14),(0, 21, 27, 7, 15),(0, 21, 27, 7, 20),(0, 21, 27, 7, 22),(0, 21, 27, 12, 6),(0, 21, 27, 12, 13),(0, 21, 27, 12, 20),(0, 21, 27, 13, 6),(0, 21, 27, 13, 7),(0, 21, 27, 13, 14),(0, 21, 27, 13, 20),(0, 21, 27, 14, 1),(0, 21, 27, 14, 6),(0, 21, 27, 14, 7),(0, 21, 27, 14, 8),(0, 21, 27, 14, 13),(0, 21, 27, 14, 15),(0, 21, 27, 14, 20),(0, 21, 27, 14, 22),(0, 21, 27, 19, 6),(0, 21, 27, 19, 13),(0, 21, 27, 19, 20),(0, 21, 27, 20, 6),(0, 21, 27, 20, 7),(0, 21, 27, 20, 13),(0, 21, 27, 20, 14),(0, 21, 27, 26, 6),(0, 21, 27, 26, 13),(0, 21, 27, 26, 20),(0, 22, 1, 2, 8),(0, 22, 1, 2, 15),(0, 22, 1, 7, 6),(0, 22, 1, 7, 8),(0, 22, 1, 7, 13),(0, 22, 1, 7, 14),(0, 22, 1, 7, 15),(0, 22, 1, 7, 20),(0, 22, 1, 7, 21),(0, 22, 1, 7, 27),(0, 22, 1, 8, 7),(0, 22, 1, 8, 14),(0, 22, 1, 8, 15),(0, 22, 1, 8, 21),(0, 22, 1, 9, 8),(0, 22, 1, 9, 15),(0, 22, 1, 14, 6),(0, 22, 1, 14, 7),(0, 22, 1, 14, 8),(0, 22, 1, 14, 13),(0, 22, 1, 14, 15),(0, 22, 1, 14, 20),(0, 22, 1, 14, 21),(0, 22, 1, 14, 27),(0, 22, 1, 15, 7),(0, 22, 1, 15, 8),(0, 22, 1, 15, 14),(0, 22, 1, 15, 21),(0, 22, 1, 16, 8),(0, 22, 1, 16, 15),(0, 22, 1, 21, 6),(0, 22, 1, 21, 7),(0, 22, 1, 21, 8),(0, 22, 1, 21, 13),(0, 22, 1, 21, 14),(0, 22, 1, 21, 15),(0, 22, 1, 21, 20),(0, 22, 1, 21, 27),(0, 22, 1, 23, 8),(0, 22, 1, 23, 15),(0, 22, 2, 1, 7),(0, 22, 2, 1, 8),(0, 22, 2, 1, 14),(0, 22, 2, 1, 15),(0, 22, 2, 1, 21),(0, 22, 2, 8, 1),(0, 22, 2, 8, 7),(0, 22, 2, 8, 14),(0, 22, 2, 8, 15),(0, 22, 2, 8, 21),(0, 22, 2, 9, 1),(0, 22, 2, 9, 8),(0, 22, 2, 9, 15),(0, 22, 2, 15, 1),(0, 22, 2, 15, 7),(0, 22, 2, 15, 8),(0, 22, 2, 15, 14),(0, 22, 2, 15, 21),(0, 22, 2, 16, 1),(0, 22, 2, 16, 8),(0, 22, 2, 16, 15),(0, 22, 2, 23, 1),(0, 22, 2, 23, 8),(0, 22, 2, 23, 15),(0, 22, 7, 1, 8),(0, 22, 7, 1, 14),(0, 22, 7, 1, 15),(0, 22, 7, 1, 21),(0, 22, 7, 6, 13),(0, 22, 7, 6, 14),(0, 22, 7, 6, 20),(0, 22, 7, 6, 21),(0, 22, 7, 6, 27),(0, 22, 7, 8, 1),(0, 22, 7, 8, 14),(0, 22, 7, 8, 15),(0, 22, 7, 8, 21),(0, 22, 7, 13, 6),(0, 22, 7, 13, 14),(0, 22, 7, 13, 20),(0, 22, 7, 13, 21),(0, 22, 7, 13, 27),(0, 22, 7, 14, 1),(0, 22, 7, 14, 6),(0, 22, 7, 14, 8),(0, 22, 7, 14, 13),(0, 22, 7, 14, 15),(0, 22, 7, 14, 20),(0, 22, 7, 14, 21),(0, 22, 7, 14, 27),(0, 22, 7, 15, 1),(0, 22, 7, 15, 8),(0, 22, 7, 15, 14),(0, 22, 7, 15, 21),(0, 22, 7, 20, 6),(0, 22, 7, 20, 13),(0, 22, 7, 20, 14),(0, 22, 7, 20, 21),(0, 22, 7, 20, 27),(0, 22, 7, 21, 1),(0, 22, 7, 21, 6),(0, 22, 7, 21, 8),(0, 22, 7, 21, 13),(0, 22, 7, 21, 14),(0, 22, 7, 21, 15),(0, 22, 7, 21, 20),(0, 22, 7, 21, 27),(0, 22, 7, 27, 6),(0, 22, 7, 27, 13),(0, 22, 7, 27, 14),(0, 22, 7, 27, 20),(0, 22, 7, 27, 21),(0, 22, 8, 1, 7),(0, 22, 8, 1, 14),(0, 22, 8, 1, 15),(0, 22, 8, 1, 21),(0, 22, 8, 2, 1),(0, 22, 8, 2, 15),(0, 22, 8, 7, 1),(0, 22, 8, 7, 6),(0, 22, 8, 7, 13),(0, 22, 8, 7, 14),(0, 22, 8, 7, 15),(0, 22, 8, 7, 20),(0, 22, 8, 7, 21),(0, 22, 8, 7, 27),(0, 22, 8, 9, 1),(0, 22, 8, 9, 15),(0, 22, 8, 14, 1),(0, 22, 8, 14, 6),(0, 22, 8, 14, 7),(0, 22, 8, 14, 13),(0, 22, 8, 14, 15),(0, 22, 8, 14, 20),(0, 22, 8, 14, 21),(0, 22, 8, 14, 27),(0, 22, 8, 15, 1),(0, 22, 8, 15, 7),(0, 22, 8, 15, 14),(0, 22, 8, 15, 21),(0, 22, 8, 16, 1),(0, 22, 8, 16, 15),(0, 22, 8, 21, 1),(0, 22, 8, 21, 6),(0, 22, 8, 21, 7),(0, 22, 8, 21, 13),(0, 22, 8, 21, 14),(0, 22, 8, 21, 15),(0, 22, 8, 21, 20),(0, 22, 8, 21, 27),(0, 22, 8, 23, 1),(0, 22, 8, 23, 15),(0, 22, 9, 1, 7),(0, 22, 9, 1, 8),(0, 22, 9, 1, 14),(0, 22, 9, 1, 15),(0, 22, 9, 1, 21),(0, 22, 9, 2, 1),(0, 22, 9, 2, 8),(0, 22, 9, 2, 15),(0, 22, 9, 8, 1),(0, 22, 9, 8, 7),(0, 22, 9, 8, 14),(0, 22, 9, 8, 15),(0, 22, 9, 8, 21),(0, 22, 9, 15, 1),(0, 22, 9, 15, 7),(0, 22, 9, 15, 8),(0, 22, 9, 15, 14),(0, 22, 9, 15, 21),(0, 22, 9, 16, 1),(0, 22, 9, 16, 8),(0, 22, 9, 16, 15),(0, 22, 9, 23, 1),(0, 22, 9, 23, 8),(0, 22, 9, 23, 15),(0, 22, 14, 1, 7),(0, 22, 14, 1, 8),(0, 22, 14, 1, 15),(0, 22, 14, 1, 21),(0, 22, 14, 6, 7),(0, 22, 14, 6, 13),(0, 22, 14, 6, 20),(0, 22, 14, 6, 21),(0, 22, 14, 6, 27),(0, 22, 14, 7, 1),(0, 22, 14, 7, 6),(0, 22, 14, 7, 8),(0, 22, 14, 7, 13),(0, 22, 14, 7, 15),(0, 22, 14, 7, 20),(0, 22, 14, 7, 21),(0, 22, 14, 7, 27),(0, 22, 14, 8, 1),(0, 22, 14, 8, 7),(0, 22, 14, 8, 15),(0, 22, 14, 8, 21),(0, 22, 14, 13, 6),(0, 22, 14, 13, 7),(0, 22, 14, 13, 20),(0, 22, 14, 13, 21),(0, 22, 14, 13, 27),(0, 22, 14, 15, 1),(0, 22, 14, 15, 7),(0, 22, 14, 15, 8),(0, 22, 14, 15, 21),(0, 22, 14, 20, 6),(0, 22, 14, 20, 7),(0, 22, 14, 20, 13),(0, 22, 14, 20, 21),(0, 22, 14, 20, 27),(0, 22, 14, 21, 1),(0, 22, 14, 21, 6),(0, 22, 14, 21, 7),(0, 22, 14, 21, 8),(0, 22, 14, 21, 13),(0, 22, 14, 21, 15),(0, 22, 14, 21, 20),(0, 22, 14, 21, 27),(0, 22, 14, 27, 6),(0, 22, 14, 27, 7),(0, 22, 14, 27, 13),(0, 22, 14, 27, 20),(0, 22, 14, 27, 21),(0, 22, 15, 1, 7),(0, 22, 15, 1, 8),(0, 22, 15, 1, 14),(0, 22, 15, 1, 21),(0, 22, 15, 2, 1),(0, 22, 15, 2, 8),(0, 22, 15, 7, 1),(0, 22, 15, 7, 6),(0, 22, 15, 7, 8),(0, 22, 15, 7, 13),(0, 22, 15, 7, 14),(0, 22, 15, 7, 20),(0, 22, 15, 7, 21),(0, 22, 15, 7, 27),(0, 22, 15, 8, 1),(0, 22, 15, 8, 7),(0, 22, 15, 8, 14),(0, 22, 15, 8, 21),(0, 22, 15, 9, 1),(0, 22, 15, 9, 8),(0, 22, 15, 14, 1),(0, 22, 15, 14, 6),(0, 22, 15, 14, 7),(0, 22, 15, 14, 8),(0, 22, 15, 14, 13),(0, 22, 15, 14, 20),(0, 22, 15, 14, 21),(0, 22, 15, 14, 27),(0, 22, 15, 16, 1),(0, 22, 15, 16, 8),(0, 22, 15, 21, 1),(0, 22, 15, 21, 6),(0, 22, 15, 21, 7),(0, 22, 15, 21, 8),(0, 22, 15, 21, 13),(0, 22, 15, 21, 14),(0, 22, 15, 21, 20),(0, 22, 15, 21, 27),(0, 22, 15, 23, 1),(0, 22, 15, 23, 8),(0, 22, 16, 1, 7),(0, 22, 16, 1, 8),(0, 22, 16, 1, 14),(0, 22, 16, 1, 15),(0, 22, 16, 1, 21),(0, 22, 16, 2, 1),(0, 22, 16, 2, 8),(0, 22, 16, 2, 15),(0, 22, 16, 8, 1),(0, 22, 16, 8, 7),(0, 22, 16, 8, 14),(0, 22, 16, 8, 15),(0, 22, 16, 8, 21),(0, 22, 16, 9, 1),(0, 22, 16, 9, 8),(0, 22, 16, 9, 15),(0, 22, 16, 15, 1),(0, 22, 16, 15, 7),(0, 22, 16, 15, 8),(0, 22, 16, 15, 14),(0, 22, 16, 15, 21),(0, 22, 16, 23, 1),(0, 22, 16, 23, 8),(0, 22, 16, 23, 15),(0, 22, 21, 1, 7),(0, 22, 21, 1, 8),(0, 22, 21, 1, 14),(0, 22, 21, 1, 15),(0, 22, 21, 6, 7),(0, 22, 21, 6, 13),(0, 22, 21, 6, 14),(0, 22, 21, 6, 20),(0, 22, 21, 6, 27),(0, 22, 21, 7, 1),(0, 22, 21, 7, 6),(0, 22, 21, 7, 8),(0, 22, 21, 7, 13),(0, 22, 21, 7, 14),(0, 22, 21, 7, 15),(0, 22, 21, 7, 20),(0, 22, 21, 7, 27),(0, 22, 21, 8, 1),(0, 22, 21, 8, 7),(0, 22, 21, 8, 14),(0, 22, 21, 8, 15),(0, 22, 21, 13, 6),(0, 22, 21, 13, 7),(0, 22, 21, 13, 14),(0, 22, 21, 13, 20),(0, 22, 21, 13, 27),(0, 22, 21, 14, 1),(0, 22, 21, 14, 6),(0, 22, 21, 14, 7),(0, 22, 21, 14, 8),(0, 22, 21, 14, 13),(0, 22, 21, 14, 15),(0, 22, 21, 14, 20),(0, 22, 21, 14, 27),(0, 22, 21, 15, 1),(0, 22, 21, 15, 7),(0, 22, 21, 15, 8),(0, 22, 21, 15, 14),(0, 22, 21, 20, 6),(0, 22, 21, 20, 7),(0, 22, 21, 20, 13),(0, 22, 21, 20, 14),(0, 22, 21, 20, 27),(0, 22, 21, 27, 6),(0, 22, 21, 27, 7),(0, 22, 21, 27, 13),(0, 22, 21, 27, 14),(0, 22, 21, 27, 20),(0, 22, 23, 1, 7),(0, 22, 23, 1, 8),(0, 22, 23, 1, 14),(0, 22, 23, 1, 15),(0, 22, 23, 1, 21),(0, 22, 23, 2, 1),(0, 22, 23, 2, 8),(0, 22, 23, 2, 15),(0, 22, 23, 8, 1),(0, 22, 23, 8, 7),(0, 22, 23, 8, 14),(0, 22, 23, 8, 15),(0, 22, 23, 8, 21),(0, 22, 23, 9, 1),(0, 22, 23, 9, 8),(0, 22, 23, 9, 15),(0, 22, 23, 15, 1),(0, 22, 23, 15, 7),(0, 22, 23, 15, 8),(0, 22, 23, 15, 14),(0, 22, 23, 15, 21),(0, 22, 23, 16, 1),(0, 22, 23, 16, 8),(0, 22, 23, 16, 15),(0, 27, 5, 6, 7),(0, 27, 5, 6, 13),(0, 27, 5, 6, 14),(0, 27, 5, 6, 20),(0, 27, 5, 6, 21),(0, 27, 5, 12, 6),(0, 27, 5, 12, 13),(0, 27, 5, 12, 20),(0, 27, 5, 13, 6),(0, 27, 5, 13, 7),(0, 27, 5, 13, 14),(0, 27, 5, 13, 20),(0, 27, 5, 13, 21),(0, 27, 5, 19, 6),(0, 27, 5, 19, 13),(0, 27, 5, 19, 20),(0, 27, 5, 20, 6),(0, 27, 5, 20, 7),(0, 27, 5, 20, 13),(0, 27, 5, 20, 14),(0, 27, 5, 20, 21),(0, 27, 5, 26, 6),(0, 27, 5, 26, 13),(0, 27, 5, 26, 20),(0, 27, 6, 5, 13),(0, 27, 6, 5, 20),(0, 27, 6, 7, 1),(0, 27, 6, 7, 8),(0, 27, 6, 7, 13),(0, 27, 6, 7, 14),(0, 27, 6, 7, 15),(0, 27, 6, 7, 20),(0, 27, 6, 7, 21),(0, 27, 6, 7, 22),(0, 27, 6, 12, 13),(0, 27, 6, 12, 20),(0, 27, 6, 13, 7),(0, 27, 6, 13, 14),(0, 27, 6, 13, 20),(0, 27, 6, 13, 21),(0, 27, 6, 14, 1),(0, 27, 6, 14, 7),(0, 27, 6, 14, 8),(0, 27, 6, 14, 13),(0, 27, 6, 14, 15),(0, 27, 6, 14, 20),(0, 27, 6, 14, 21),(0, 27, 6, 14, 22),(0, 27, 6, 19, 13),(0, 27, 6, 19, 20),(0, 27, 6, 20, 7),(0, 27, 6, 20, 13),(0, 27, 6, 20, 14),(0, 27, 6, 20, 21),(0, 27, 6, 21, 1),(0, 27, 6, 21, 7),(0, 27, 6, 21, 8),(0, 27, 6, 21, 13),(0, 27, 6, 21, 14),(0, 27, 6, 21, 15),(0, 27, 6, 21, 20),(0, 27, 6, 21, 22),(0, 27, 6, 26, 13),(0, 27, 6, 26, 20),(0, 27, 7, 1, 8),(0, 27, 7, 1, 14),(0, 27, 7, 1, 15),(0, 27, 7, 1, 21),(0, 27, 7, 1, 22),(0, 27, 7, 6, 13),(0, 27, 7, 6, 14),(0, 27, 7, 6, 20),(0, 27, 7, 6, 21),(0, 27, 7, 8, 1),(0, 27, 7, 8, 14),(0, 27, 7, 8, 15),(0, 27, 7, 8, 21),(0, 27, 7, 8, 22),(0, 27, 7, 13, 6),(0, 27, 7, 13, 14),(0, 27, 7, 13, 20),(0, 27, 7, 13, 21),(0, 27, 7, 14, 1),(0, 27, 7, 14, 6),(0, 27, 7, 14, 8),(0, 27, 7, 14, 13),(0, 27, 7, 14, 15),(0, 27, 7, 14, 20),(0, 27, 7, 14, 21),(0, 27, 7, 14, 22),(0, 27, 7, 15, 1),(0, 27, 7, 15, 8),(0, 27, 7, 15, 14),(0, 27, 7, 15, 21),(0, 27, 7, 15, 22),(0, 27, 7, 20, 6),(0, 27, 7, 20, 13),(0, 27, 7, 20, 14),(0, 27, 7, 20, 21),(0, 27, 7, 21, 1),(0, 27, 7, 21, 6),(0, 27, 7, 21, 8),(0, 27, 7, 21, 13),(0, 27, 7, 21, 14),(0, 27, 7, 21, 15),(0, 27, 7, 21, 20),(0, 27, 7, 21, 22),(0, 27, 7, 22, 1),(0, 27, 7, 22, 8),(0, 27, 7, 22, 14),(0, 27, 7, 22, 15),(0, 27, 7, 22, 21),(0, 27, 12, 5, 6),(0, 27, 12, 5, 13),(0, 27, 12, 5, 20),(0, 27, 12, 6, 7),(0, 27, 12, 6, 13),(0, 27, 12, 6, 14),(0, 27, 12, 6, 20),(0, 27, 12, 6, 21),(0, 27, 12, 13, 6),(0, 27, 12, 13, 7),(0, 27, 12, 13, 14),(0, 27, 12, 13, 20),(0, 27, 12, 13, 21),(0, 27, 12, 19, 6),(0, 27, 12, 19, 13),(0, 27, 12, 19, 20),(0, 27, 12, 20, 6),(0, 27, 12, 20, 7),(0, 27, 12, 20, 13),(0, 27, 12, 20, 14),(0, 27, 12, 20, 21),(0, 27, 12, 26, 6),(0, 27, 12, 26, 13),(0, 27, 12, 26, 20),(0, 27, 13, 5, 6),(0, 27, 13, 5, 20),(0, 27, 13, 6, 7),(0, 27, 13, 6, 14),(0, 27, 13, 6, 20),(0, 27, 13, 6, 21),(0, 27, 13, 7, 1),(0, 27, 13, 7, 6),(0, 27, 13, 7, 8),(0, 27, 13, 7, 14),(0, 27, 13, 7, 15),(0, 27, 13, 7, 20),(0, 27, 13, 7, 21),(0, 27, 13, 7, 22),(0, 27, 13, 12, 6),(0, 27, 13, 12, 20),(0, 27, 13, 14, 1),(0, 27, 13, 14, 6),(0, 27, 13, 14, 7),(0, 27, 13, 14, 8),(0, 27, 13, 14, 15),(0, 27, 13, 14, 20),(0, 27, 13, 14, 21),(0, 27, 13, 14, 22),(0, 27, 13, 19, 6),(0, 27, 13, 19, 20),(0, 27, 13, 20, 6),(0, 27, 13, 20, 7),(0, 27, 13, 20, 14),(0, 27, 13, 20, 21),(0, 27, 13, 21, 1),(0, 27, 13, 21, 6),(0, 27, 13, 21, 7),(0, 27, 13, 21, 8),(0, 27, 13, 21, 14),(0, 27, 13, 21, 15),(0, 27, 13, 21, 20),(0, 27, 13, 21, 22),(0, 27, 13, 26, 6),(0, 27, 13, 26, 20),(0, 27, 14, 1, 7),(0, 27, 14, 1, 8),(0, 27, 14, 1, 15),(0, 27, 14, 1, 21),(0, 27, 14, 1, 22),(0, 27, 14, 6, 7),(0, 27, 14, 6, 13),(0, 27, 14, 6, 20),(0, 27, 14, 6, 21),(0, 27, 14, 7, 1),(0, 27, 14, 7, 6),(0, 27, 14, 7, 8),(0, 27, 14, 7, 13),(0, 27, 14, 7, 15),(0, 27, 14, 7, 20),(0, 27, 14, 7, 21),(0, 27, 14, 7, 22),(0, 27, 14, 8, 1),(0, 27, 14, 8, 7),(0, 27, 14, 8, 15),(0, 27, 14, 8, 21),(0, 27, 14, 8, 22),(0, 27, 14, 13, 6),(0, 27, 14, 13, 7),(0, 27, 14, 13, 20),(0, 27, 14, 13, 21),(0, 27, 14, 15, 1),(0, 27, 14, 15, 7),(0, 27, 14, 15, 8),(0, 27, 14, 15, 21),(0, 27, 14, 15, 22),(0, 27, 14, 20, 6),(0, 27, 14, 20, 7),(0, 27, 14, 20, 13),(0, 27, 14, 20, 21),(0, 27, 14, 21, 1),(0, 27, 14, 21, 6),(0, 27, 14, 21, 7),(0, 27, 14, 21, 8),(0, 27, 14, 21, 13),(0, 27, 14, 21, 15),(0, 27, 14, 21, 20),(0, 27, 14, 21, 22),(0, 27, 14, 22, 1),(0, 27, 14, 22, 7),(0, 27, 14, 22, 8),(0, 27, 14, 22, 15),(0, 27, 14, 22, 21),(0, 27, 19, 5, 6),(0, 27, 19, 5, 13),(0, 27, 19, 5, 20),(0, 27, 19, 6, 7),(0, 27, 19, 6, 13),(0, 27, 19, 6, 14),(0, 27, 19, 6, 20),(0, 27, 19, 6, 21),(0, 27, 19, 12, 6),(0, 27, 19, 12, 13),(0, 27, 19, 12, 20),(0, 27, 19, 13, 6),(0, 27, 19, 13, 7),(0, 27, 19, 13, 14),(0, 27, 19, 13, 20),(0, 27, 19, 13, 21),(0, 27, 19, 20, 6),(0, 27, 19, 20, 7),(0, 27, 19, 20, 13),(0, 27, 19, 20, 14),(0, 27, 19, 20, 21),(0, 27, 19, 26, 6),(0, 27, 19, 26, 13),(0, 27, 19, 26, 20),(0, 27, 20, 5, 6),(0, 27, 20, 5, 13),(0, 27, 20, 6, 7),(0, 27, 20, 6, 13),(0, 27, 20, 6, 14),(0, 27, 20, 6, 21),(0, 27, 20, 7, 1),(0, 27, 20, 7, 6),(0, 27, 20, 7, 8),(0, 27, 20, 7, 13),(0, 27, 20, 7, 14),(0, 27, 20, 7, 15),(0, 27, 20, 7, 21),(0, 27, 20, 7, 22),(0, 27, 20, 12, 6),(0, 27, 20, 12, 13),(0, 27, 20, 13, 6),(0, 27, 20, 13, 7),(0, 27, 20, 13, 14),(0, 27, 20, 13, 21),(0, 27, 20, 14, 1),(0, 27, 20, 14, 6),(0, 27, 20, 14, 7),(0, 27, 20, 14, 8),(0, 27, 20, 14, 13),(0, 27, 20, 14, 15),(0, 27, 20, 14, 21),(0, 27, 20, 14, 22),(0, 27, 20, 19, 6),(0, 27, 20, 19, 13),(0, 27, 20, 21, 1),(0, 27, 20, 21, 6),(0, 27, 20, 21, 7),(0, 27, 20, 21, 8),(0, 27, 20, 21, 13),(0, 27, 20, 21, 14),(0, 27, 20, 21, 15),(0, 27, 20, 21, 22),(0, 27, 20, 26, 6),(0, 27, 20, 26, 13),(0, 27, 21, 1, 7),(0, 27, 21, 1, 8),(0, 27, 21, 1, 14),(0, 27, 21, 1, 15),(0, 27, 21, 1, 22),(0, 27, 21, 6, 7),(0, 27, 21, 6, 13),(0, 27, 21, 6, 14),(0, 27, 21, 6, 20),(0, 27, 21, 7, 1),(0, 27, 21, 7, 6),(0, 27, 21, 7, 8),(0, 27, 21, 7, 13),(0, 27, 21, 7, 14),(0, 27, 21, 7, 15),(0, 27, 21, 7, 20),(0, 27, 21, 7, 22),(0, 27, 21, 8, 1),(0, 27, 21, 8, 7),(0, 27, 21, 8, 14),(0, 27, 21, 8, 15),(0, 27, 21, 8, 22),(0, 27, 21, 13, 6),(0, 27, 21, 13, 7),(0, 27, 21, 13, 14),(0, 27, 21, 13, 20),(0, 27, 21, 14, 1),(0, 27, 21, 14, 6),(0, 27, 21, 14, 7),(0, 27, 21, 14, 8),(0, 27, 21, 14, 13),(0, 27, 21, 14, 15),(0, 27, 21, 14, 20),(0, 27, 21, 14, 22),(0, 27, 21, 15, 1),(0, 27, 21, 15, 7),(0, 27, 21, 15, 8),(0, 27, 21, 15, 14),(0, 27, 21, 15, 22),(0, 27, 21, 20, 6),(0, 27, 21, 20, 7),(0, 27, 21, 20, 13),(0, 27, 21, 20, 14),(0, 27, 21, 22, 1),(0, 27, 21, 22, 7),(0, 27, 21, 22, 8),(0, 27, 21, 22, 14),(0, 27, 21, 22, 15),(0, 27, 26, 5, 6),(0, 27, 26, 5, 13),(0, 27, 26, 5, 20),(0, 27, 26, 6, 7),(0, 27, 26, 6, 13),(0, 27, 26, 6, 14),(0, 27, 26, 6, 20),(0, 27, 26, 6, 21),(0, 27, 26, 12, 6),(0, 27, 26, 12, 13),(0, 27, 26, 12, 20),(0, 27, 26, 13, 6),(0, 27, 26, 13, 7),(0, 27, 26, 13, 14),(0, 27, 26, 13, 20),(0, 27, 26, 13, 21),(0, 27, 26, 19, 6),(0, 27, 26, 19, 13),(0, 27, 26, 19, 20),(0, 27, 26, 20, 6),(0, 27, 26, 20, 7),(0, 27, 26, 20, 13),(0, 27, 26, 20, 14),(0, 27, 26, 20, 21)]
    
    #a list of Sage graphs being 'accumulated'
    graphs = []

    f_results = open("EP: FiveCycle_Graph_triangle_inserted_FINDING_Results.txt", "a")
    f_times = open("EP: FiveCycle_Graph_triangle_inserted_FINDING_Times.txt", "a")

    count = 0

    #double checked with previous result
    five_cycles_one_vertex_inserted_dict = [{0: [1, 20, 21, 6, 22, 7, 8, 27, 28, 13, 14, 15], 1: [0, 16, 2, 21, 22, 7, 23, 8, 9, 28, 14, 15], 2: [16, 1, 17, 3, 22, 23, 8, 24, 9, 10, 28, 15], 3: [16, 17, 2, 18, 4, 23, 24, 9, 25, 10, 11], 4: [17, 18, 3, 19, 5, 24, 25, 10, 26, 11, 12], 5: [18, 19, 4, 20, 6, 25, 26, 11, 27, 12, 13], 6: [0, 19, 20, 5, 21, 7, 26, 27, 12, 13, 14], 7: [0, 1, 20, 21, 6, 22, 8, 27, 28, 13, 14, 15], 8: [0, 16, 1, 2, 21, 22, 7, 23, 9, 28, 14, 15], 9: [16, 1, 17, 2, 3, 22, 23, 8, 24, 10, 15], 10: [16, 17, 2, 18, 3, 4, 23, 24, 9, 25, 11], 11: [17, 18, 3, 19, 4, 5, 24, 25, 10, 26, 12], 12: [18, 19, 4, 20, 5, 6, 25, 26, 11, 27, 13], 13: [0, 19, 20, 5, 21, 6, 7, 26, 27, 12, 14], 14: [0, 1, 20, 21, 6, 22, 7, 8, 27, 13, 15], 15: [0, 16, 1, 2, 21, 22, 7, 23, 8, 9, 14], 16: [1, 17, 2, 3, 22, 23, 8, 24, 9, 10, 15], 17: [16, 2, 18, 3, 4, 23, 24, 9, 25, 10, 11], 18: [17, 3, 19, 4, 5, 24, 25, 10, 26, 11, 12], 19: [18, 4, 20, 5, 6, 25, 26, 11, 27, 12, 13], 20: [0, 19, 5, 21, 6, 7, 26, 27, 12, 13, 14], 21: [0, 1, 20, 6, 22, 7, 8, 27, 13, 14, 15], 22: [0, 16, 1, 2, 21, 7, 23, 8, 9, 14, 15], 23: [16, 1, 17, 2, 3, 22, 8, 24, 9, 10, 15], 24: [16, 17, 2, 18, 3, 4, 23, 9, 25, 10, 11], 25: [17, 18, 3, 19, 4, 5, 24, 10, 26, 11, 12], 26: [18, 19, 4, 20, 5, 6, 25, 11, 27, 12, 13], 27: [0, 19, 20, 5, 21, 6, 7, 26, 12, 13, 14], 28: [0, 1, 2, 7, 8]},
        {0: [1, 20, 21, 6, 22, 7, 8, 27, 28, 13, 14, 15], 1: [0, 16, 2, 21, 22, 7, 23, 8, 9, 28, 14, 15], 2: [16, 1, 17, 3, 22, 23, 8, 24, 9, 10, 28, 15], 3: [16, 17, 2, 18, 4, 23, 24, 9, 25, 10, 11], 4: [17, 18, 3, 19, 5, 24, 25, 10, 26, 11, 12], 5: [18, 19, 4, 20, 6, 25, 26, 11, 27, 12, 13], 6: [0, 19, 20, 5, 21, 7, 26, 27, 12, 13, 14], 7: [0, 1, 20, 21, 6, 22, 8, 27, 13, 14, 15], 8: [0, 16, 1, 2, 21, 22, 7, 23, 9, 28, 14, 15], 9: [16, 1, 17, 2, 3, 22, 23, 8, 24, 10, 15], 10: [16, 17, 2, 18, 3, 4, 23, 24, 9, 25, 11], 11: [17, 18, 3, 19, 4, 5, 24, 25, 10, 26, 12], 12: [18, 19, 4, 20, 5, 6, 25, 26, 11, 27, 13], 13: [0, 19, 20, 5, 21, 6, 7, 26, 27, 12, 14], 14: [0, 1, 20, 21, 6, 22, 7, 8, 27, 13, 15], 15: [0, 16, 1, 2, 21, 22, 7, 23, 8, 9, 28, 14], 16: [1, 17, 2, 3, 22, 23, 8, 24, 9, 10, 15], 17: [16, 2, 18, 3, 4, 23, 24, 9, 25, 10, 11], 18: [17, 3, 19, 4, 5, 24, 25, 10, 26, 11, 12], 19: [18, 4, 20, 5, 6, 25, 26, 11, 27, 12, 13], 20: [0, 19, 5, 21, 6, 7, 26, 27, 12, 13, 14], 21: [0, 1, 20, 6, 22, 7, 8, 27, 13, 14, 15], 22: [0, 16, 1, 2, 21, 7, 23, 8, 9, 14, 15], 23: [16, 1, 17, 2, 3, 22, 8, 24, 9, 10, 15], 24: [16, 17, 2, 18, 3, 4, 23, 9, 25, 10, 11], 25: [17, 18, 3, 19, 4, 5, 24, 10, 26, 11, 12], 26: [18, 19, 4, 20, 5, 6, 25, 11, 27, 12, 13], 27: [0, 19, 20, 5, 21, 6, 7, 26, 12, 13, 14], 28: [0, 1, 2, 8, 15]},
        {0: [1, 20, 21, 6, 22, 7, 8, 27, 28, 13, 14, 15], 1: [0, 16, 2, 21, 22, 7, 23, 8, 9, 28, 14, 15], 2: [16, 1, 17, 3, 22, 23, 8, 24, 9, 10, 15], 3: [16, 17, 2, 18, 4, 23, 24, 9, 25, 10, 11], 4: [17, 18, 3, 19, 5, 24, 25, 10, 26, 11, 12], 5: [18, 19, 4, 20, 6, 25, 26, 11, 27, 12, 13], 6: [0, 19, 20, 5, 21, 7, 26, 27, 12, 13, 14], 7: [0, 1, 20, 21, 6, 22, 8, 27, 28, 13, 14, 15], 8: [0, 16, 1, 2, 21, 22, 7, 23, 9, 28, 14, 15], 9: [16, 1, 17, 2, 3, 22, 23, 8, 24, 10, 15], 10: [16, 17, 2, 18, 3, 4, 23, 24, 9, 25, 11], 11: [17, 18, 3, 19, 4, 5, 24, 25, 10, 26, 12], 12: [18, 19, 4, 20, 5, 6, 25, 26, 11, 27, 13], 13: [0, 19, 20, 5, 21, 6, 7, 26, 27, 12, 14], 14: [0, 1, 20, 21, 6, 22, 7, 8, 27, 28, 13, 15], 15: [0, 16, 1, 2, 21, 22, 7, 23, 8, 9, 14], 16: [1, 17, 2, 3, 22, 23, 8, 24, 9, 10, 15], 17: [16, 2, 18, 3, 4, 23, 24, 9, 25, 10, 11], 18: [17, 3, 19, 4, 5, 24, 25, 10, 26, 11, 12], 19: [18, 4, 20, 5, 6, 25, 26, 11, 27, 12, 13], 20: [0, 19, 5, 21, 6, 7, 26, 27, 12, 13, 14], 21: [0, 1, 20, 6, 22, 7, 8, 27, 13, 14, 15], 22: [0, 16, 1, 2, 21, 7, 23, 8, 9, 14, 15], 23: [16, 1, 17, 2, 3, 22, 8, 24, 9, 10, 15], 24: [16, 17, 2, 18, 3, 4, 23, 9, 25, 10, 11], 25: [17, 18, 3, 19, 4, 5, 24, 10, 26, 11, 12], 26: [18, 19, 4, 20, 5, 6, 25, 11, 27, 12, 13], 27: [0, 19, 20, 5, 21, 6, 7, 26, 12, 13, 14], 28: [0, 1, 7, 8, 14]},
        {0: [1, 20, 21, 6, 22, 7, 8, 27, 28, 13, 14, 15], 1: [0, 16, 2, 21, 22, 7, 23, 8, 9, 28, 14, 15], 2: [16, 1, 17, 3, 22, 23, 8, 24, 9, 10, 15], 3: [16, 17, 2, 18, 4, 23, 24, 9, 25, 10, 11], 4: [17, 18, 3, 19, 5, 24, 25, 10, 26, 11, 12], 5: [18, 19, 4, 20, 6, 25, 26, 11, 27, 12, 13], 6: [0, 19, 20, 5, 21, 7, 26, 27, 12, 13, 14], 7: [0, 1, 20, 21, 6, 22, 8, 27, 28, 13, 14, 15], 8: [0, 16, 1, 2, 21, 22, 7, 23, 9, 14, 15], 9: [16, 1, 17, 2, 3, 22, 23, 8, 24, 10, 15], 10: [16, 17, 2, 18, 3, 4, 23, 24, 9, 25, 11], 11: [17, 18, 3, 19, 4, 5, 24, 25, 10, 26, 12], 12: [18, 19, 4, 20, 5, 6, 25, 26, 11, 27, 13], 13: [0, 19, 20, 5, 21, 6, 7, 26, 27, 12, 14], 14: [0, 1, 20, 21, 6, 22, 7, 8, 27, 28, 13, 15], 15: [0, 16, 1, 2, 21, 22, 7, 23, 8, 9, 14], 16: [1, 17, 2, 3, 22, 23, 8, 24, 9, 10, 15], 17: [16, 2, 18, 3, 4, 23, 24, 9, 25, 10, 11], 18: [17, 3, 19, 4, 5, 24, 25, 10, 26, 11, 12], 19: [18, 4, 20, 5, 6, 25, 26, 11, 27, 12, 13], 20: [0, 19, 5, 21, 6, 7, 26, 27, 12, 13, 14], 21: [0, 1, 20, 6, 22, 7, 8, 27, 28, 13, 14, 15], 22: [0, 16, 1, 2, 21, 7, 23, 8, 9, 14, 15], 23: [16, 1, 17, 2, 3, 22, 8, 24, 9, 10, 15], 24: [16, 17, 2, 18, 3, 4, 23, 9, 25, 10, 11], 25: [17, 18, 3, 19, 4, 5, 24, 10, 26, 11, 12], 26: [18, 19, 4, 20, 5, 6, 25, 11, 27, 12, 13], 27: [0, 19, 20, 5, 21, 6, 7, 26, 12, 13, 14], 28: [0, 1, 21, 7, 14]}]
    
    five_cycles_one_vertex_inserted = [dict_to_adj_list(i) for i in five_cycles_one_vertex_inserted_dict]

    five_cycles_triangle_inserted = []

    print("tricycles length: " + str(len(tricycles)))

    '''
    for h in five_cycles_one_vertex_inserted:
        
        for j in tricycles:
            i = copy.deepcopy(h)
            curr_5cycle = i[len(i)-1]

            cycle_overlap = False
            
            for k in curr_5cycle:
                if k in j:
                    cycle_overlap = True
                    break
            
            if cycle_overlap:
                continue

            for n in j:
                curr_5cycle.append(n)
                i[n].append(len(i)-1)
                #print("current 5 cycle: " + str(curr_5cycle) + " i[n]: " + str(i[n]))

            #converts from adj list to dict
            #required by Sage input for new Graph construction
            #recall dicts immutable in Python
            temp_graph = adj_list_to_dict(i)
                
            #for checking
            #print(temp_graph)

            sage_temp = graph.Graph(temp_graph)

            isomorph_seen = False
            i_start_time = time.time()

            for n in five_cycles_triangle_inserted:
                if sage_temp.is_isomorphic(n):
                    isomorph_seen = True
                    break
            
            if not isomorph_seen:
                five_cycles_triangle_inserted.append(sage_temp)
                    # associated_cycles.append(i)

                i_end_time = time.time()
                i_full_time = i_end_time - i_start_time

                i_start_time = time.time()


                f_results.write(str(sage_temp.to_dictionary()) + "," + "\n")
                    
                count += 1
                f_times.write(str(count) + str(datetime.timedelta(seconds=i_full_time)) + "\n")

                print("NON-ISOMORPH 5 CYCLE GRAPH COUNT: " + str(count) + "; TIME: " + str(datetime.timedelta(seconds=i_full_time)))

                #TODO include this?
                #break
    '''

''' 
#first principles finding of 5 cycles
    for i in g[0]:
        for j in g[i]:
            for k in g[j]:
                for l in g[k]:
                    for m in g[l]:
                        if m == 0 and all_different(0, i, j, k, l):
                            cycle = (0, i, j, k, l)
                            fivecycles_list.append(cycle)
                            print(str(cycle) + ",", end='') 

   for i in fivecycles_list:
        h = copy.deepcopy(g)
        new_list = []
        nv1 = len(h)

        for n in i:
            new_list.append(n)
        
        for n in new_list:
            h[n].append(nv1)
        
        h.append(new_list)

        #converts from adj list to dict
        #required by Sage input for new Graph construction
        #recall dicts immutable in Python
        temp_graph = adj_list_to_dict(h)
            
        #for checking
        #print(temp_graph)

        sage_temp = graph.Graph(temp_graph)

        isomorph_seen = False
        i_start_time = time.time()

        for n in five_cycles_one_vertex_inserted:
            if sage_temp.is_isomorphic(n):
                isomorph_seen = True
                break
        
        if not isomorph_seen:
            five_cycles_one_vertex_inserted.append(sage_temp)
                # associated_cycles.append(i)

            i_end_time = time.time()
            i_full_time = i_end_time - i_start_time

            i_start_time = time.time()


            f_results.write(str(sage_temp.to_dictionary()) + "," + "\n")
                
            count += 1
            f_times.write(str(count) + str(datetime.timedelta(seconds=i_full_time)) + "\n")

            print("NON-ISOMORPH 5 CYCLE GRAPH COUNT: " + str(count) + "; TIME: " + str(datetime.timedelta(seconds=i_full_time)))

            #TODO include this?
            #break
'''




if __name__ == "__main__":
    main()

tricycles length: 2184
